## **Imports**

In [ ]:
!python3 --version

Python 3.7.13


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
%cd gdrive/My Drive/BERTweet-ner

Mounted at /content/gdrive
/content/gdrive/My Drive/BERTweet-ner


In [ ]:
!pip3 install datasets
!pip3 install transformers
!pip3 install -U sentence-transformers
!pip3 install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.2 MB/s 
     |████████████████████████████████| 140 kB 78.3 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 1.1 MB 56.5 MB/s 
     |████████████████████████████████| 212 kB 79.1 MB/s 
     |████████████████████████████████| 596 kB 75.7 MB/s 
     |████████████████████████████████| 127 kB 50.2 MB/s 
     |████████████████████████████████| 144 kB 80.2 MB/s 
     |████████████████████████████████| 271 kB 68.3 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dep

In [ ]:
!pip3 install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=2898d376ed080e7643acc0c6d4d3ba9065e22366075808b2d1a759322914a31a
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
# import nltk
# nltk.download('all')

In [ ]:
import pandas as pd
import numpy as np
import re
import string

from datasets import load_dataset, load_metric, ClassLabel, Sequence
import random
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import copy

# import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

import math
import shutil

torch.autograd.set_detect_anomaly(True)
torch.cuda.amp.autocast(enabled=False)

## **Setting some global stuff**

In [ ]:
import torch
# Set device
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

cuda


In [ ]:
# example_2D_list = [[0.2423,-0.6679,0.8277],[0.2423,-0.6679,0.8277],[0.2423,-0.6679,0.8277]]
# twodTensor = torch.tensor(example_2D_list)

# example_1D_list = [0.2245,-2.2268,0.4577]
# origTensor = torch.tensor(example_1D_list)

# # modifiedTensor = origTensor.repeat(3, 1)

# # print(twodTensor.size(),origTensor.size())
# # print(modifiedTensor)

# cosine_layer = torch.nn.CosineSimilarity(dim=1)
# output = cosine_layer(origTensor, twodTensor)
# print(output)
# print(torch.mul(output, -1/100))

In [ ]:
# import pandas as pd
# import numpy as np


class PhraseEmbeddingI(nn.Module):

    def __init__(self,input_size,output_size,device,mentions_dict,index_candidate_dict):
        super(PhraseEmbeddingI, self).__init__()
        self.print_once=True
        self.dense_layer = nn.Linear(input_size,output_size)
        self.batchnorm = nn.BatchNorm1d(output_size)
        self.non_linear_layer = nn.LeakyReLU() #nn.Tanh()
        self.cosine_layer = nn.CosineSimilarity(dim=1)
        self.device = device
        self.mentions_dict_train = mentions_dict
        self.mentions_embedding_dict_train = {}
        self.index_candidate_dict_train = index_candidate_dict
        self.temperature = 2
        return

    def encode(self, input_embedding):

        # print(input_embedding.size())

        #Average Pooling has already been done before
        input_embedding = input_embedding.to(device=self.device)
        x = self.dense_layer(input_embedding)
        x = self.batchnorm(x)
        out = self.non_linear_layer(x)
        # print(out.size())
        return out

    def compute(self, input_tuple):
        # Soft Nearest Neighbours
        # print(input_tuple)
        anchor_mention_index = input_tuple[0]
        input_anchor = self.index_candidate_dict_train[input_tuple[1]]
        input_negative1 = self.index_candidate_dict_train[input_tuple[2]]
        input_negative2 = self.index_candidate_dict_train[input_tuple[3]]
        input_negative3 = self.index_candidate_dict_train[input_tuple[4]]
        input_negative4 = self.index_candidate_dict_train[input_tuple[5]]

        # print(anchor_mention_index,input_anchor,input_negative1,input_negative2,input_negative3,input_negative4)

        positive_embeddings_list = []
        all_embeddings_list = []

        anchor_embedding = self.encode(self.mentions_dict_train[input_anchor][anchor_mention_index])

        positive_embeddings_list = [self.mentions_dict_train[input_anchor][ind] for ind in range(len(self.mentions_dict_train[input_anchor])) if ind != anchor_mention_index]
        
        all_embeddings_list += positive_embeddings_list
        all_embeddings_list += self.mentions_dict_train[input_negative1] + self.mentions_dict_train[input_negative2] + self.mentions_dict_train[input_negative3] + self.mentions_dict_train[input_negative4]

        pos_emb_tensors = self.encode(torch.stack(positive_embeddings_list).to(self.device)) #
        all_emb_tensors = self.encode(torch.stack(all_embeddings_list).to(self.device)) #

        # print(len(positive_embeddings_list),len(all_embeddings_list))
        # print(pos_emb_tensors.size(),all_emb_tensors.size())

        # #Cosine sims
        pos_sim_tensor = torch.mul(1-self.cosine_layer(anchor_embedding, pos_emb_tensors), -1/self.temperature)
        all_sim_tensor = torch.mul(1-self.cosine_layer(anchor_embedding, all_emb_tensors), -1/self.temperature)

        ret = torch.sum(torch.exp(pos_sim_tensor))/torch.sum(torch.exp(all_sim_tensor)).to(self.device)
        # print('==>',output_anchor.size(),output_positive.size(),output_negative.size())
        return ret

    def forward(self,batch_data):
        output_arr = []
        printn=0

        # # print('start')
        # for key in self.mentions_dict_train.keys():
        #     mentions_embedding_list = []
        #     for emb in self.mentions_dict_train[key]:
        #         mentions_embedding_list.append(self.encode(emb))
        #     self.mentions_embedding_dict_train[key] = mentions_embedding_list
        # # print('end')

        anchor_mention_indices = batch_data[0]
        anchor_candidate_indices = batch_data[1]
        neg_candidate1_indices = batch_data[2]
        neg_candidate2_indices = batch_data[3]
        neg_candidate3_indices = batch_data[4]
        neg_candidate4_indices = batch_data[5]

        for index in range(len(anchor_mention_indices)):
            x_tup = (int(anchor_mention_indices[index].item()), int(anchor_candidate_indices[index].item()), int(neg_candidate1_indices[index].item()), int(neg_candidate2_indices[index].item()), int(neg_candidate3_indices[index].item()), int(neg_candidate4_indices[index].item()))
            out = self.compute(x_tup)
            # if(printn%10==0):
            #     print(printn)
            printn+=1
            output_arr.append(out)
        output_tensor = torch.stack(output_arr).to(self.device)
        return output_tensor

    def getEmbedding(self, input_embeddings):
        with torch.no_grad():
            return self.encode(input_embeddings)

In [ ]:
# import pandas as pd
# import numpy as np


class PhraseEmbedding(nn.Module):

    def __init__(self,input_size,output_size,device):
        super(PhraseEmbedding, self).__init__()
        self.print_once=True
        self.dense_layer1 = nn.Linear(input_size,output_size)
        # self.batchnorm1 = nn.BatchNorm1d(350)
        # self.dense_layer2 = nn.Linear(350,output_size)
        self.batchnorm2 = nn.BatchNorm1d(output_size)
        self.non_linear_layer = nn.Tanh()#nn.LeakyReLU() 
        self.cosine_layer = nn.CosineSimilarity(dim=1)
        self.device = device
        return

    

    def encode(self, input_embedding):

        # print(input_embedding.size())

        #Average Pooling has already been done before
        input_embedding = input_embedding.to(device=self.device)
        # print(average_pooled_embedding.size())
        x = self.dense_layer1(input_embedding)
        # x = self.batchnorm1(x)
        # x = self.non_linear_layer(x)
        # x = self.dense_layer2(x)
        x = self.batchnorm2(x)
        out = self.non_linear_layer(x)
        # print(out.size())
        return out

    def forward(self, input_tuple):
        # Triplet
        input_anchor = input_tuple[0]
        input_positive = input_tuple[1]
        input_negative = input_tuple[2]

        output_anchor = self.encode(input_anchor)
        output_positive = self.encode(input_positive)
        output_negative = self.encode(input_negative)

        assert torch.isnan(output_anchor).any() == False
        assert torch.isnan(output_positive).any() == False
        assert torch.isnan(output_negative).any() == False

        # #L2 distances
        # positive_distance = torch.linalg.norm(output_anchor - output_positive)
        # negative_distance = torch.linalg.norm(output_anchor - output_negative)

        #Cosine distances
        positive_distance = 1-self.cosine_layer(output_anchor, output_positive)
        negative_distance = 1-self.cosine_layer(output_anchor, output_negative)

        # print('==>',output_anchor.size(),output_positive.size(),output_negative.size())
        return (positive_distance,negative_distance)

    def getEmbedding(self, input_embeddings):
        with torch.no_grad():
            return self.encode(input_embeddings)

In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
    
    def forward(self, input_tup):
        distance_positive = input_tup[0]
        distance_negative = input_tup[1]
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        # print(distance_positive.size(),losses.size())
        return losses.mean()

In [ ]:
class TupletLoss(nn.Module):
    def __init__(self):
        super(TupletLoss, self).__init__()
    
    def forward(self, input_tup):
        expdiff1 = input_tup[0]
        expdiff2 = input_tup[1]
        expdiff3 = input_tup[2]
        expdiff4 = input_tup[3]
        expdiffsum = expdiff1 + expdiff2 + expdiff3 + expdiff4
        losses = torch.log(torch.add(expdiffsum,1))
        
        # print(expdiff1.size(),expdiffsum.size(),losses.size())
        return losses.mean()

In [ ]:
class SoftKNNLoss(nn.Module):
    def __init__(self):
        super(SoftKNNLoss, self).__init__()
    
    def forward(self, input):
        losses = torch.mul(torch.log(input),-1)
        # print(input.size(),losses.size())
        # final_loss = Variable(final_loss, requires_grad = True)’
        return losses.mean()

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "vinai/bertweet-base"
batch_size = 32
datasets = load_dataset("wnut_17")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

Dataset wnut_17 downloaded and prepared to /root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## **Initialization of some components**

In [ ]:
from os import path

# from entityEmbedding import phraseEmbedding
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer, checkpoint['epoch']

In [ ]:
# Initialize network
output_embedding_size = 300
# output_embedding_size = 768
learning_rate = 0.001

phraseEmbeddingModel = PhraseEmbedding(768, output_embedding_size, device).to(device) #triplet
# phraseEmbeddingModel = PhraseEmbeddingI(768, output_embedding_size, device).to(device) #soft-knn

#Loss and Optimizer
optimizer = optim.Adam(phraseEmbeddingModel.parameters(), lr=learning_rate, weight_decay=0.0001)

In [ ]:
# define checkpoint saved path for entity phrase embedder

# ckp_path = "entityEmbedding/model_checkpoints_ner/checkpoint_model300.pt" #300 triplet
# # ckp_path = "entityEmbedding/model_checkpoints_ner/checkpoint_model300_softknn.pt" #300 tuplet

# if(path.exists(ckp_path)):
#     # load the saved checkpoint
#     entityPhraseEmbedder, optimizer, start_epoch = load_ckp(ckp_path, phraseEmbeddingModel, optimizer)

#     print("starting with model at epoch:", start_epoch)

In [ ]:
import nltk

nltk.download('stopwords')
nltk.download('gutenberg')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

gutenberg_text = ""
for file_id in gutenberg.fileids():
    gutenberg_text += gutenberg.raw(file_id)
tokenizer_trainer = PunktTrainer()
tokenizer_trainer.INCLUDE_ALL_COLLOCS = True
tokenizer_trainer.train(gutenberg_text)

sentence_tokenizer = PunktSentenceTokenizer(tokenizer_trainer.get_params())

sentence_tokenizer._params.abbrev_types.add('dr')
sentence_tokenizer._params.abbrev_types.add('c.j')
sentence_tokenizer._params.abbrev_types.add('u.s')
sentence_tokenizer._params.abbrev_types.add('u.s.a')
sentence_tokenizer._params.abbrev_types.add('ret')
sentence_tokenizer._params.abbrev_types.add('rep')
sentence_tokenizer._params.abbrev_types.add('mr')
sentence_tokenizer._params.abbrev_types.add('ms')
sentence_tokenizer._params.abbrev_types.add('mrs')
sentence_tokenizer._params.abbrev_types.add('v')
sentence_tokenizer._params.abbrev_types.add('vs')

## **External fine-tuning of BERTweet**

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "vinai/bertweet-base"
batch_size = 16

In [ ]:
datasets = load_dataset("wnut_17")

Reusing dataset wnut_17 (/root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
label_all_tokens = True

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
expanded_label_dict={0:'O', 1:'B-corporation', 2:'I-corporation', 3:'B-creative-work', 4:'I-creative-work', 5:'B-group', 6:'I-group', 7:'B-location', 8:'I-location', 9:'B-person', 10:'I-person', 11:'B-product', 12:'I-product'}
BIO_dict={'O':0,'B':1,'I':2}
BIO_type_dict={'O':0, 'B-ORG':1, 'I-ORG':2, 'B-MISC':3, 'I-MISC':4, 'B-LOC':5, 'I-LOC':6, 'B-PER':7, 'I-PER':8}
label_map_dict={'O':'O', 'B-corporation':'B-ORG', 'I-corporation':'I-ORG', 'B-creative-work':'B-MISC', 'I-creative-work':'I-MISC', 'B-group':'B-MISC', 'I-group':'I-MISC', 'B-location':'B-LOC', 'I-location':'I-LOC', 'B-person':'B-PER', 'I-person':'I-PER', 'B-product':'B-MISC', 'I-product':'I-MISC'}

def tokenize_and_align_labels(example):
        
    tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
    inputId_to_token_dict={}
    for index, token in enumerate(example["tokens"]):
        values=tokenizer.encode(token, add_special_tokens=False, truncation=True)
        for value in values:
            try:
                inputId_to_token_dict[value].append(index)
            except KeyError:
                inputId_to_token_dict[value]=[index]
    labels=[]
    for inputID in tokenized_input['input_ids']:
        try:
            index_list=copy.deepcopy(inputId_to_token_dict[inputID])
            index_to_address=index_list.pop(0)

            # label=BIO_dict[expanded_label_dict[example['ner_tags'][index_to_address]][0]] #Just BIO
            # label = example['ner_tags'][index_to_address]
            label = BIO_type_dict[label_map_dict[expanded_label_dict[example['ner_tags'][index_to_address]]]]

            labels.append(label)
            inputId_to_token_dict[inputID]=index_list
        except KeyError:
            labels.append(-100)

    assert (len(tokenized_input['input_ids']) == len(labels))
    tokenized_input['labels']=labels
    
    return tokenized_input

In [ ]:
# tokenize_and_align_labels(datasets['train'][2])
tokenized_datasets = datasets.map(tokenize_and_align_labels)

Parameter 'function'=<function tokenize_and_align_labels at 0x7f42499f1b00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/3394 [00:00<?, ?ex/s]

  0%|          | 0/1009 [00:00<?, ?ex/s]

  0%|          | 0/1287 [00:00<?, ?ex/s]

In [ ]:
tokenized_datasets['train']

Dataset({
    features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3394
})

In [ ]:
!pip3 install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=b91e1e601946d476ca6a18d10abdd97922320deb017a103ea2961e244f696328
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

In [ ]:
# label_list = datasets["train"].features[f"{task}_tags"].feature.names

# label_list = ['O','B','I']

label_list = ['O','B-ORG','I-ORG','B-MISC','I-MISC','B-LOC','I-LOC','B-PER','I-PER']
print(label_list)
print(len(label_list))

['O', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER']
9


In [ ]:
import numpy as np

def compute_metrics(p):
    # print(p.shape)
    output, labels = p

    # print(len(predictions))
    # print(predictions[0].shape)
    # for elem in predictions[1]:
    #   print(elem.shape)

    predictions, _ = output
    
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
alt_model = AutoModelForTokenClassification.from_pretrained("vinai/bertweet-base", output_hidden_states=True, num_labels=len(label_list))
# alt_model.config

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [ ]:
alt_training_args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)
# alt_training_args

In [ ]:
# alt_training_args

In [ ]:
alt_trainer = Trainer(
    alt_model,
    alt_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
alt_trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3394
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 639


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.275247,0.536797,0.412303,0.466385,0.934701
2,No log,0.227843,0.678075,0.527016,0.593078,0.946833
3,0.210900,0.241026,0.683519,0.561929,0.616788,0.948677


The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1009
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1009


TrainOutput(global_step=639, training_loss=0.1856081787969025, metrics={'train_runtime': 411.4005, 'train_samples_per_second': 24.75, 'train_steps_per_second': 1.553, 'total_flos': 196065609633108.0, 'train_loss': 0.1856081787969025, 'epoch': 3.0})

## **Pre-processing on custom test set**

In [ ]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [ ]:
import re
import emoji
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from nltk.tokenize import TweetTokenizer
import string

string.punctuation=string.punctuation+'…‘’'

In [ ]:
def collate_token_labels(tweetWordList, token_dict, prediction_labels):
    #this function is also a valid alt implementation of the one in LocalNER module
    counter=0
    collated_labels=[]
    for word in tweetWordList:
    # for key in token_dict.keys():
        
        vals=token_dict[word]
        if(counter<len(prediction_labels)):
            labels=prediction_labels[counter:counter+len(vals)]
            boundary_labels=[label[0] for label in labels]
            if('I' in boundary_labels):
                label=labels[boundary_labels.index('I')]
                collated_labels.append(label)
            elif('B' in boundary_labels):
                label=labels[boundary_labels.index('B')]
                collated_labels.append(label)
            else:
                collated_labels.append('O')
            counter+=len(vals)
        else:
            collated_labels.append('O')
    return collated_labels

In [ ]:
def get_entities(word_tag_tuples):
    
    mentions=[]
    candidateMention=''
    type_tag=''
    #emoji.get_emoji_regexp().sub(u'', candidateMention)
    for tup in word_tag_tuples:
        candidate=tup[0]
        tag=tup[1]

        if(tag=='O'):
            if(candidateMention):
                if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
                    mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
                    if mention_to_add.endswith("'s"):
                        li = mention_to_add.rsplit("'s", 1)
                        mention_to_add=''.join(li)
                    elif mention_to_add.endswith("’s"):
                        li = mention_to_add.rsplit("’s", 1)
                        mention_to_add=''.join(li)
                    else:
                        mention_to_add=mention_to_add
                    if(mention_to_add!=''):
                        mentions.append((mention_to_add,type_tag))
            candidateMention=''
            type_tag=''
        else:
            boundary_tag = tag.split('-')[0]
            type_tag = tag.split('-')[1]
            if (boundary_tag=='B'):
                if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
                    mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
                    if mention_to_add.endswith("'s"):
                        li = mention_to_add.rsplit("'s", 1)
                        mention_to_add=''.join(li)
                    elif mention_to_add.endswith("’s"):
                        li = mention_to_add.rsplit("’s", 1)
                        mention_to_add=''.join(li)
                    else:
                        mention_to_add=mention_to_add
                    if(mention_to_add!=''):
                        mentions.append((mention_to_add,type_tag))
                candidateMention=candidate
            else:
                candidateMention+=" "+candidate
        # if (tag=='B'):
        #     if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))):
        #         mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
        #         if(mention_to_add):
        #             mentions.append(mention_to_add)
        #     candidateMention=candidate
        # else:
        #     candidateMention+=" "+candidate
    if(emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).strip()):
        if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
            mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
            if(mention_to_add!=''):
                mentions.append((mention_to_add,type_tag))
        # mentions.append(emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip())
    # print('extracted mentions:', mentions)
    return mentions

In [ ]:
# def get_encoding_seq(tweet_word_list, mentions):
#     print(tweet_word_list)
#     print(mentions)
#     tweet_word_index=0
#     encoded_tag_sequence=[]
#     while(mentions):
#         current_mention=[token.strip() for token in mentions.pop(0).split(' ')]
#         while(normalize(current_mention[0])!=normalize(tweet_word_list[tweet_word_index])):
#             encoded_tag_sequence.append('O')
#             tweet_word_index+=1
#         if(normalize(current_mention[0])==normalize(tweet_word_list[tweet_word_index])):
#             for token_index, token in enumerate(current_mention):
#                 if(token_index==0):
#                     encoded_tag_sequence.append('B')
#                 else:
#                     encoded_tag_sequence.append('I')
#                 tweet_word_index+=1
#     while(tweet_word_index<len(tweet_word_list)):
#         encoded_tag_sequence.append('O')
#         tweet_word_index+=1
        
#     print(encoded_tag_sequence)
#     return encoded_tag_sequence

In [ ]:
gutenberg_text = ""
for file_id in gutenberg.fileids():
    gutenberg_text += gutenberg.raw(file_id)
tokenizer_trainer = PunktTrainer()
tokenizer_trainer.INCLUDE_ALL_COLLOCS = True
tokenizer_trainer.train(gutenberg_text)

my_sentence_tokenizer = PunktSentenceTokenizer(tokenizer_trainer.get_params())
my_sentence_tokenizer._params.abbrev_types.add('dr')
my_sentence_tokenizer._params.abbrev_types.add('c.j')
my_sentence_tokenizer._params.abbrev_types.add('u.s')
my_sentence_tokenizer._params.abbrev_types.add('u.s.a')
my_sentence_tokenizer._params.abbrev_types.add('ret')
my_sentence_tokenizer._params.abbrev_types.add('rep')
my_sentence_tokenizer._params.abbrev_types.add('mr')
my_sentence_tokenizer._params.abbrev_types.add('ms')
my_sentence_tokenizer._params.abbrev_types.add('mrs')
my_sentence_tokenizer._params.abbrev_types.add('v')
my_sentence_tokenizer._params.abbrev_types.add('vs')


def normalize_to_sentences(text):
    tweetSentences=list(filter (lambda sentence: len(sentence)>1, text.split('\n')))
    tweetSentenceList_inter=custom_flatten(list(map(lambda sentText: my_sentence_tokenizer.tokenize(sentText.lstrip().rstrip()),tweetSentences)),[])
    tweetSentenceList=list(filter (lambda sentence: len(sentence)>1, tweetSentenceList_inter))
    return tweetSentenceList

def custom_flatten(mylist, outlist,ignore_types=(str, bytes, int)):
    
    if (mylist !=[]):
        for item in mylist:
            #print not isinstance(item, ne.NE_candidate)
            if isinstance(item, list) and not isinstance(item, ignore_types):
                custom_flatten(item, outlist)
            else:
                item=item.strip(' \t\n\r')
                outlist.append(item)
    return outlist

In [ ]:
def preprocess(filename):
    """save a file with token, label and prediction in each row"""
    tweet_to_sentences_w_annotation={}
    sentenceID=0
    test=pd.read_csv("data/"+filename,sep =',',keep_default_na=False)
    # outputfilename="data/covid/covid_2K.txt"
    
    all_annotated_ne=[]
    tweetsentences=[]
    tokenizedsentences=[]
    
    for row in test.itertuples():
        tweetID=str(row.Index)
        text=str(row.TweetText)
        row_sentences = normalize_to_sentences(text)
        tweetsentences += row_sentences
        tokenizedsentences += [tokenizer(sentence, is_split_into_words=True) for sentence in row_sentences]
        # print(tweetID,text)
        
        mentions=[]
        # print(row.mentions_other)
        for sentence_level in str(row.mentions_other).split(';'):
            if(sentence_level):
                for elem in sentence_level.split(','):
                    if(elem):
                        mention_record = elem.split('|')
                        # print(mention_record)
                        mention, entity_type =  mention_record[0], mention_record[1].lower()
                        if(mention):
                            if((mention !='')&(mention !='nan')):
                                mention = mention.lower().strip(string.punctuation).strip()
                                if(mention.startswith('the ')):
                                    mention = mention[4:]
                                mentions.append((mention, entity_type))
        # mentions=list(filter(lambda element: ((element !='')&(element !='nan')), mentions))
        # all_annotated_ne.extend(mentions)
        
        # if(row_sentences):
        tweet_to_sentences_w_annotation[tweetID]=((sentenceID,sentenceID+len(row_sentences)),mentions)
        sentenceID+=len(row_sentences)
        # else:
        #     tweet_to_sentences_w_annotation[tweetID]=((sentenceID,sentenceID+1),mentions)
        #     sentenceID+=1
        # print(sentenceID,len(row_sentences))
    return tweetsentences, tokenizedsentences, tweet_to_sentences_w_annotation

In [ ]:
def calculate_f1_ner(tweet_to_sentences_w_annotation, tweetsentences, true_predictions):
    
    entity_types = ['org','misc', 'loc', 'per']
    confusion_matrices = {entity_type: {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0} for entity_type in entity_types}
    scores = {entity_type: {'p': 0, 'r': 0, 'f1': 0} for entity_type in entity_types}

    ner_arrays=[]
    for n, tweet in enumerate(tweetsentences):
        # tweet_data = list(zip(tweet, true_labels[n], predictions[i:i + len(tweet)]))
        # word_tag_tuples=zip(tweet,predictions[i:i + len(tweet)])
        
        assert (len(tweet)==len(true_predictions[n]))
        word_tag_tuples=zip(tweet,true_predictions[n])
        entities_from_sentence=get_entities(word_tag_tuples)
        ner_arrays.append(entities_from_sentence)

    print('tally:',len(tweetsentences),len(ner_arrays))

    for tweetID in tweet_to_sentences_w_annotation.keys():
        unrecovered_annotated_mention_list=[]
        annotated_mention_list=tweet_to_sentences_w_annotation[tweetID][1]
        output_mentions_list=[]
        true_positive_list=[]
        idRange=tweet_to_sentences_w_annotation[tweetID][0]

        for sentID in range(idRange[0],idRange[1]):
            output_mentions_list+=ner_arrays[sentID]
        output_mentions_list=[(elem[0],elem[1].lower()) for elem in output_mentions_list]
        print(tweetID,annotated_mention_list,output_mentions_list)

        while(annotated_mention_list):
            if(len(output_mentions_list)):
                annotated_candidate= annotated_mention_list.pop()
                if(annotated_candidate in output_mentions_list):
                    output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                    # tp_counter_inner+=1 # for emd
                    true_positive_list.append(annotated_candidate)
                    # #for ner
                    # entity_type=annotated_candidate[1]
                    # confusion_matrices[entity_type]['TP']+=1
                else:
                    unrecovered_annotated_mention_list.append(annotated_candidate)
            else:
                unrecovered_annotated_mention_list.extend(annotated_mention_list)
                break
        # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)

        # fn_counter_inner=len(unrecovered_annotated_mention_list)
        # fp_counter_inner=all_postitive_counter_inner - tp_counter_inner
        # print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

        print('true positives:',true_positive_list)
        print('false positives:',output_mentions_list)
        print('false negatives:',unrecovered_annotated_mention_list)

        for mention_tup in true_positive_list:
            entity_type=mention_tup[1]
            if(mention_tup[0]):
                confusion_matrices[entity_type]['TP']+=1

        for mention_tup in unrecovered_annotated_mention_list:
            entity_type=mention_tup[1]
            if(mention_tup[0]):
                confusion_matrices[entity_type]['FN']+=1

        for mention_tup in output_mentions_list:
            entity_type=mention_tup[1]
            if(mention_tup[0]):
                confusion_matrices[entity_type]['FP']+=1
    
    print('========Named Entity Recognition========')
    for entity_type in entity_types:
        print(entity_type.upper())
        precision = confusion_matrices[entity_type]['TP']/(confusion_matrices[entity_type]['TP']+confusion_matrices[entity_type]['FP'])
        recall = confusion_matrices[entity_type]['TP']/(confusion_matrices[entity_type]['TP']+confusion_matrices[entity_type]['FN'])
        f_measure = 2*precision*recall/(precision+recall)

        scores[entity_type]['p'] = precision
        scores[entity_type]['r'] = recall
        scores[entity_type]['f1'] = f_measure

        print('precision: ',precision)
        print('recall: ',recall)
        print('f_measure: ',f_measure)

        print('------------------------')
        


In [ ]:
def calculate_f1_emd(tweet_to_sentences_w_annotation, tweetsentences, true_predictions):
    
    # dataset, i = [], 0
    ner_arrays=[]
    file_write_text=''
    all_detected_ne=[]
    all_annotated_ne=[]
    
    for n, tweet in enumerate(tweetsentences):
        # tweet_data = list(zip(tweet, true_labels[n], predictions[i:i + len(tweet)]))
        # word_tag_tuples=zip(tweet,predictions[i:i + len(tweet)])
        
        assert (len(tweet)==len(true_predictions[n]))
        word_tag_tuples=zip(tweet,true_predictions[n])
        entities_from_sentence=get_entities(word_tag_tuples)
        # line_text='\t'.join(entities_from_sentence)
        # file_write_text+=line_text+'\n'
        # print(entities_from_sentence)
        all_detected_ne.extend(entities_from_sentence)
        ner_arrays.append(entities_from_sentence)
        # dataset += tweet_data + [()]
    
    print('tally:',len(tweetsentences),len(ner_arrays))
    system_output_mention_list=list(set(all_detected_ne))
    # file_write_text='\n'.join(system_output_mention_list)
    # f1= open(outputfilename, "w")
    # f1.write(file_write_text)
    # f1.close()
    
    true_positive_count=0
    false_positive_count=0
    false_negative_count=0
    total_mentions=0
    total_annotation=0
    
    
    for tweetID in tweet_to_sentences_w_annotation.keys():
        unrecovered_annotated_mention_list=[]
        tp_counter_inner=0
        fp_counter_inner=0
        fn_counter_inner=0
        
        annotated_mention_list=tweet_to_sentences_w_annotation[tweetID][1]
        all_annotated_ne.extend(annotated_mention_list)
        output_mentions_list=[]
        output_mentions_w_type_list=[]
        idRange=tweet_to_sentences_w_annotation[tweetID][0]
        for sentID in range(idRange[0],idRange[1]):
            output_mentions_w_type_list+=ner_arrays[sentID]
            
        print(tweetID,annotated_mention_list,output_mentions_w_type_list)
        output_mentions_list=[elem[0] for elem in output_mentions_w_type_list]
        print(output_mentions_list)
        all_postitive_counter_inner=len(output_mentions_list)
        while(annotated_mention_list):
            if(len(output_mentions_list)):
                annotated_candidate= annotated_mention_list.pop()
                if(annotated_candidate in output_mentions_list):
                    output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                    tp_counter_inner+=1
                else:
                    unrecovered_annotated_mention_list.append(annotated_candidate)
            else:
                unrecovered_annotated_mention_list.extend(annotated_mention_list)
                break
        # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
        fn_counter_inner=len(unrecovered_annotated_mention_list)
        fp_counter_inner=all_postitive_counter_inner - tp_counter_inner
        
        print(tp_counter_inner,fp_counter_inner,fn_counter_inner)
        
        true_positive_count+=tp_counter_inner
        false_positive_count+=fp_counter_inner
        false_negative_count+=fn_counter_inner
        
    print('true_positive_count,false_positive_count,false_negative_count:')
    print(true_positive_count,false_positive_count,false_negative_count)
    
    precision=(true_positive_count)/(true_positive_count+false_positive_count)
    recall=(true_positive_count)/(true_positive_count+false_negative_count)
    f_measure=2*(precision*recall)/(precision+recall)
            
    print('========Entity Mention Detection========')
    print('precision: ',precision)
    print('recall: ',recall)
    print('f_measure: ',f_measure)

    # print('========Entity Detection========')
    # true_positive_entities =  len(list(set(all_detected_ne).intersection(set(all_annotated_ne))))
    # false_positive_entities = len(list(set(all_annotated_ne)-set(all_detected_ne)))
    # false_negative_entities = len(list(set(all_detected_ne)-set(all_annotated_ne)))

    # precision= (true_positive_entities)/(true_positive_entities+false_positive_entities)
    # recall= (true_positive_entities)/(true_positive_entities+false_negative_entities)
    # f_measure = 2*(precision*recall)/(precision+recall)

    # print('precision: ',precision)
    # print('recall: ',recall)
    # print('f_measure: ',f_measure)   

    return

## **Entity Classifier Alt**

In [ ]:
import pandas as pd
import time
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, random_split

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [ ]:
class CandidateDataset(Dataset):
    def __init__(self, df, candidateEmbeddingDict):
        self.samples= []
        self.max_freq=-1
        self.freq_arr = []
        # self.embeddings_repo = []
        self.output = []

        for index, row in df.iterrows():
            candidate_record = row['candidate'].split('||')
            # print(candidate_record)
            candidate, cluster_id = candidate_record[0], candidate_record[1]
            normalized_length = row['normalized_length']
            cumulative= row['cumulative']
            if(cumulative>self.max_freq):
                self.max_freq=cumulative

            output_val= row['class']

            local_embedding_list= candidateEmbeddingDict[(candidate,cluster_id)]
            assert len(local_embedding_list) !=0
            assert cumulative == len(local_embedding_list)

            tup=(candidate,cluster_id,normalized_length,cumulative)

            self.freq_arr.append(cumulative)
            self.samples.append(tup)
            self.output.append(output_val)
        

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        tup2=(self.freq_arr[idx]/self.max_freq,)
        tup=self.samples[idx]+tup2

        return tup,self.output[idx]
        # return self.len_arr[idx],self.cum_freq_arr[idx],self.output[idx]

In [ ]:
# 2 output_classes: 'entity'/'non-entity'; so sigmoid transformation would suffice

class NN(nn.Module):
    def __init__(self,input_size,device):
        super(NN, self).__init__()
        self.device=device
        self.input_size = input_size

        #customized weighted pooling
        self.pooling_layer1 = nn.Linear(input_size,50)
        self.pooling_layer2 = nn.Linear(50,1)

        self.linear1 = nn.Linear(input_size+1,500)
        self.batchnorm = nn.BatchNorm1d(500)
        self.linear2 = nn.Linear(500,200)
        self.linear3 = nn.Linear(200,50)
        self.output_layer = nn.Linear(50,5)
      
    def nxn_cos_sim(A, B, dim=1, eps=1e-8):
      numerator = A @ B.T
      A_l2 = torch.mul(A, A).sum(axis=dim)
      B_l2 = torch.mul(B, B).sum(axis=dim)
      denominator = torch.max(torch.sqrt(torch.outer(A_l2, B_l2)), torch.tensor(eps))
      return torch.div(numerator, denominator)

    
    def compute(self, x_tup):

        x_len=x_tup[0]
        # x_List=x_tup[1]
        local_embedding_list=x_tup[1]
        x_cum_freq=x_tup[2]
        x_normalized_freq=x_tup[3]
        
        # x_tensor = torch.FloatTensor(np.array([x_len,x_normalized_freq],dtype=float)).to(self.device)
        x_tensor = torch.FloatTensor(np.array([x_len],dtype=float)).to(self.device)

        
        all_local_embedding = torch.stack(local_embedding_list).to(self.device)

        pooling_output_1=F.relu(self.pooling_layer1(all_local_embedding))
        pooling_output_2=self.pooling_layer2(pooling_output_1)
        weights = pooling_output_2.reshape(-1)
        # print(all_local_embedding.size(),len(weights))

        # cos_sim_matrix = self.nxn_cos_sim(all_local_embedding,all_local_embedding)
        exp_tensor = torch.exp(weights)
        sum_tensor = torch.sum(exp_tensor)
        weights = torch.div(exp_tensor,sum_tensor).tolist()


        aggregated_input = torch.zeros(self.input_size, requires_grad=True).to(self.device)
        for ind, local_embedding in enumerate(all_local_embedding):
            aggregated_input=aggregated_input.add(torch.mul(local_embedding, weights[ind]))
        
        # aggregated_input=torch.mul(aggregated_input, float(1/x_cum_freq))
        x = torch.cat((x_tensor,aggregated_input), 0).to(self.device)

        return x

    def forward(self, batch_data):
        global_embeddings_arr=[]
        for x_tup in batch_data:
            global_embeddings_arr.append(self.compute(x_tup))
        # global_embeddings_arr = np.array(global_embeddings_arr)
        global_embeddings_batch = torch.stack(global_embeddings_arr).to(self.device)
        # global_embeddings_batch.requires_grad = True
        # print(global_embeddings_batch.size())
        x = F.relu(self.linear1(global_embeddings_batch))
        x = self.batchnorm(x)
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        out = self.output_layer(x)
        return out

class EntityClassifierAlt():

    def __init__(self, to_train, device, candidateBase_train, candidateEmbeddingDict):


        self.candidateEmbeddingDict_train = candidateEmbeddingDict
        self.candidateBase_train = candidateBase_train
        self.combined_feature_list=['length']

        self.device=device

        self.input_size = 300 #len(['normalized_cf_'+str(i) for i in range(300)])

        self.entity_types = {0:'ne', 1:'org', 2:'misc', 3:'loc', 4:'per'}
        # self.confusion_matrices = {entity_type: {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0} for entity_type in self.entity_types.keys()}
        # self.scores = {entity_type: {'p': 0, 'r': 0, 'f1': 0} for entity_type in self.entity_types.keys()}

        # self.relevant_columns = ['candidate','normalized_length','cumulative']
        self.relevant_columns = ['candidate','normalized_length','cumulative','class']
        # self.relevant_columns = ['normalized_cf_'+str(i) for i in range(768)]
        
        #initialize the classifier model
        self.classifier = NN(self.input_size,device).to(device)
        #Loss and Optimizer
        self.ec_criterion = nn.CrossEntropyLoss()
        self.ec_optimizer = optim.Adam(self.classifier.parameters(), lr = 0.01, weight_decay=1e-8)
        self.ec_batch_size = 32
        self.ec_num_epochs = 200
        self.patience = 10


        if(to_train):

            # define checkpoint saved path
            ckp_path = "entityClassifierAlt/model_checkpoints_ner/classifierAlt_checkpoint_model300_wnut.pt" #300

            if(path.exists(ckp_path)):
                # load the saved checkpoint
                self.classifier, self.ec_optimizer, self.start_epoch = self.load_ckp(ckp_path, self.classifier, self.ec_optimizer)

            max_candidate_length = self.candidateBase_train['length'].max()+1
            # print(self.train['length'].tolist())
            self.candidateBase_train['normalized_length'] = self.candidateBase_train['length']/max_candidate_length

            print(self.candidateBase_train['normalized_length'].max())
            
            #Loading the data
            # candidate_array = self.train['candidate'].tolist()
            dataset_df = self.candidateBase_train[self.relevant_columns]

            dataset = CandidateDataset(dataset_df, self.candidateEmbeddingDict_train)

            print(dataset.__getitem__(0))

            train_len=int(math.ceil(len(dataset_df)*0.85))
            val_len=len(dataset_df)-train_len

            print('train_len',train_len)
            print('val_len',val_len)

            train_ds, val_ds = random_split(dataset, [train_len, val_len])

            self.train_loader = DataLoader(train_ds, self.ec_batch_size, shuffle=True)
            self.val_loader = DataLoader(val_ds, val_len, shuffle=True) #will execute in 1 batch

            #Training the model
            end_epoch = self.fit()

            checkpoint_dir = "entityClassifierAlt/model_checkpoints_ner"
            self.save_ckp(self.checkpoint, True, checkpoint_dir)

        else:
            # define checkpoint saved path
            ckp_path = "entityClassifierAlt/model_checkpoints_ner/classifierAlt_checkpoint_model300.pt" #300
            # ckp_path = "entityClassifierAlt/model_checkpoints_ner/classifierAlt_checkpoint_model300_wnut.pt" #300
            # load the saved checkpoint
            self.classifier, self.ec_optimizer, self.start_epoch = self.load_ckp(ckp_path, self.classifier, self.ec_optimizer)

    def load_ckp(self, checkpoint_fpath, model, optimizer):
        checkpoint = torch.load(checkpoint_fpath)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer, checkpoint['epoch']

    def save_ckp(self,state, is_best, checkpoint_dir):
        # f_path = checkpoint_dir + '/classifierAlt_checkpoint768.pt' #768
        f_path = checkpoint_dir + '/classifierAlt_checkpoint_model300.pt' #300
        torch.save(state, f_path)

    def custom_loss(self, out, targets, freq):
        print('verifying shapes:')
        print(out.shape, targets.shape)
        softmaxloss_arr=[]
        for i, out_i in enumerate(out):
            print(out_i.shape, targets[i].shape)
            loss_i = self.ec_criterion(out_i, targets[i])
            softmaxloss_arr.append(loss_i)
        # print(type(softmaxloss))
        softmaxloss = torch.stack(softmaxloss_arr).to(self.device)
        print(softmaxloss, freq)
        print(softmaxloss.shape, freq.shape)
        vec=softmaxloss*freq
        print(vec)
        final_loss = torch.sum(vec)/torch.sum(freq)
        print(vec.shape, loss.shape)
        return final_loss


    def fit(self):
        # Train Network
        history_validation = []
        history_training= []
        no_improvement_counter=0
        best_loss = np.float('inf')
        best_f1 = np.float('-inf')
        for epoch in range(self.ec_num_epochs):
            training_batch_loss=[]
            for batch_idx, (data_inter, targets) in enumerate(self.train_loader):
                # Get data to cuda if possible
                # data = data.to(device=device)

                # print(len(data_inter),len(targets),type(data_inter))
                # print(len(data_inter[0]),len(data_inter[1]),len(data_inter[2]))

                #(candidate,cluster_id,normalized_length,cumulative,normalized_cumulative)
                candidates = data_inter[0]
                cluster_ids = data_inter[1]
                length_arr = data_inter[2]
                cumulative_arr = data_inter[3]
                normalized_freq_arr = data_inter[4]

                data = [(length_arr[index], self.candidateEmbeddingDict_train[(candidate,cluster_ids[index])], cumulative_arr[index], normalized_freq_arr[index]) for index, candidate in enumerate(candidates)]
                # targets = targets.unsqueeze(1).to(device=self.device)
                targets = targets.type(torch.LongTensor)
                targets = targets.to(device=self.device)

                self.ec_optimizer.zero_grad()

                # forwards
                out = self.classifier(data)
                # out = out.to(torch.float32)
                # targets = targets.to(torch.float32)

                # #custom cross-entropy loss factoring in frequency
                # loss = self.custom_loss(out, targets, torch.FloatTensor(np.array(cumulative_arr)).unsqueeze(1).to(device=self.device))

                loss = self.ec_criterion(out, targets)

                training_batch_loss.append(loss.item())
                # print(loss.item())

                # backward
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.classifier.parameters(), 1.0)
                # gradient descent or adam step
                self.ec_optimizer.step()
            combined_training_loss = np.mean(training_batch_loss)
            print('combined_training_loss:',combined_training_loss)
            history_training.append(combined_training_loss)

            #Validation: DO NOT BACKPROPAGATE HERE
            validation_batch_loss = []
            labels = []
            all_candidates = []
            predictions = []
            self.confusion_matrices = {entity_type: {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0} for entity_type in self.entity_types.keys()}
            self.scores = {entity_type: {'p': 0, 'r': 0, 'f1': 0} for entity_type in self.entity_types.keys()}
            with torch.no_grad():
                self.classifier.eval()
                for batch_idx, (val_data_inter, val_targets) in enumerate(self.val_loader):
                    # val_data = val_data.to(device=device)
                    val_candidates = val_data_inter[0]
                    val_cluster_ids = val_data_inter[1]
                    val_length_arr = val_data_inter[2]
                    val_cumulative_arr = val_data_inter[3]
                    val_normalized_freq_arr = val_data_inter[4]

                    val_data = [(val_length_arr[index], self.candidateEmbeddingDict_train[(val_candidate,val_cluster_ids[index])], val_cumulative_arr[index], val_normalized_freq_arr[index]) for index, val_candidate in enumerate(val_candidates)]
                    # val_targets = val_targets.unsqueeze(1).to(device=device)
                    val_targets = val_targets.type(torch.LongTensor)
                    val_targets = val_targets.to(device=device)

                    val_out = self.classifier(val_data)

                    # y_pred = out.to(torch.float32)
                    # val_targets = val_targets.to(torch.float32)

                    # print('checking shapes:')
                    # print(out.shape)
                    # print(val_targets.shape)
                    
                    y_pred_softmax = torch.log_softmax(val_out, dim = 1)
                    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
                    batch_predictions = y_pred_tags.reshape(-1)
                    # print(y_pred_tags.shape)
                    # print(batch_predictions.shape)

                    predictions+=batch_predictions.tolist()

                    labels+=val_targets.tolist()
                    all_candidates+=candidates

                    # loss = F.mse_loss(out, val_targets) round
                    val_loss = self.ec_criterion(val_out, val_targets)
                    # loss = self.custom_loss(out, val_targets, torch.FloatTensor(np.array(cumulative_arr)).unsqueeze(1).to(device=self.device)) #custom l1 loss factoring in frequency
                    validation_batch_loss.append(val_loss.item())
                    # print(validation_batch_loss)
                combined_validation_loss= np.mean(validation_batch_loss)
                assert len(predictions)==len(labels)
                # print(len(predictions),len(labels),type(predictions[0]))
                # print(predictions)
                

                #EMD training objective
                for indx in range(len(predictions)):
                    predicted_class = predictions[indx]
                    true_class = labels[indx]

                    if(predicted_class==true_class):
                        self.confusion_matrices[true_class]['TP']+=1
                    else:
                        self.confusion_matrices[true_class]['FN']+=1
                        self.confusion_matrices[true_class]['FP']+=1

                macrof1=0
                for entity_type in self.entity_types:
                    # print(entity_type.upper())
                    try:
                        precision = self.confusion_matrices[entity_type]['TP']/(self.confusion_matrices[entity_type]['TP']+self.confusion_matrices[entity_type]['FP'])
                    except ZeroDivisionError:
                        precision = 0
                    try:
                        recall = self.confusion_matrices[entity_type]['TP']/(self.confusion_matrices[entity_type]['TP']+self.confusion_matrices[entity_type]['FN'])
                    except ZeroDivisionError:
                        recall = 0
                    try:
                        f_measure = 2*precision*recall/(precision+recall)
                    except ZeroDivisionError:
                        f_measure = 0

                    self.scores[entity_type]['p'] = precision
                    self.scores[entity_type]['r'] = recall
                    self.scores[entity_type]['f1'] = f_measure
                    macrof1+=f_measure
                    print(self.entity_types[entity_type], 'precision: ',precision, 'recall: ',recall, 'f_measure: ',f_measure)
                    
                macrof1 = macrof1/len(self.entity_types)
                print('macrof1:',str(macrof1))
                history_validation.append(combined_validation_loss)
                
                print('Epoch',str(epoch+1),':',combined_validation_loss)
                
                
                # if(combined_validation_loss<best_loss):
                if(macrof1>best_f1):
                    # best_loss = combined_validation_loss
                    best_f1 = macrof1
                    print('making this the checkpoint to save')
                    #Saving the model
                    self.checkpoint = {
                                'epoch': epoch + 1,
                                'state_dict': self.classifier.state_dict(),
                                'optimizer': self.ec_optimizer.state_dict()
                            }
                    no_improvement_counter=0
                else:
                    no_improvement_counter+=1
                    if(no_improvement_counter>self.patience):
                        break
                if(((epoch+1)%10==0)|(epoch == (self.ec_num_epochs-1))):
                    print('=========')
                print('------------------------')
                print('\n')

        return epoch

    def run(self,candidateBase,candidateEmbeddingDict):

        candidateBase['class']=-1
        max_length=candidateBase['length'].max()+1
        candidateBase['normalized_length']= candidateBase['length']/max_length

        test_dataset = CandidateDataset(candidateBase, candidateEmbeddingDict)
        test_loader = DataLoader(test_dataset, len(test_dataset)) #will execute in 1 batch

        #Testing
        predictions=[]
        with torch.no_grad():
            self.classifier.eval()
            for batch_idx, (data_inter, targets) in enumerate(test_loader):
                # data = data.to(device=device)
                candidates = data_inter[0]
                cluster_ids = data_inter[1]
                length_arr = data_inter[2]
                cumulative_arr = data_inter[3]
                normalized_freq_arr = data_inter[4]

                data = [(length_arr[index], candidateEmbeddingDict[(candidate,cluster_ids[index])], cumulative_arr[index], normalized_freq_arr[index]) for index, candidate in enumerate(candidates)]
                y_pred = self.classifier(data)
                # print(out.shape)
                y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
                _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
                predictions = y_pred_tags.reshape(-1)
        print(predictions.shape,len(candidateBase))

        candidateBase['class'] = predictions.tolist()
        # print(candidateBase['probability'].min(), candidateBase['probability'].max())
        return candidateBase

## **Extracting Embeddings to train the Phrase Embedder on Contrastive Loss and EntityClassifier**

In [ ]:
from emoji import demojize
from collections import defaultdict
from random import shuffle

In [ ]:
# Set device
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)
tweetTokenizer = TweetTokenizer()

cuda


In [ ]:
def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):
    tokens = tweetTokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) - ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return normTweet

In [ ]:
def collate_token_label_embedding(tweetWordList, token_dict, prediction_labels, entity_embeddings):
    counter=0
    collated_labels=[]
    collated_entity_embeddings=[]
    for word in tweetWordList:
        vals=token_dict[word]
        # print(word,vals)
        if(counter<len(prediction_labels)):
            labels=prediction_labels[counter:counter+len(vals)]
            token_entity_embeddings=entity_embeddings[counter:counter+len(vals)]
    #         print(token_entity_embeddings.shape)
            mean_tensor = torch.mean(token_entity_embeddings,dim=0)
            mean_tensor[torch.isnan(mean_tensor)] = 0
            collated_entity_embeddings.append(mean_tensor)
    #         print(collated_entity_embeddings)
            if('I-PER' in labels):
                collated_labels.append('I-PER')
            elif('I-LOC' in labels):
                collated_labels.append('I-LOC')
            elif('I-ORG' in labels):
                collated_labels.append('I-ORG')
            elif('I-MISC' in labels):
                collated_labels.append('I-MISC')
            elif('B-PER' in labels):
                collated_labels.append('B-PER')
            elif('B-LOC' in labels):
                collated_labels.append('B-LOC')
            elif('B-ORG' in labels):
                collated_labels.append('B-ORG')
            elif('B-MISC' in labels):
                collated_labels.append('B-MISC')
            else:
                collated_labels.append('O')
            counter+=len(vals)
        else:
            collated_labels.append('O')
            collated_entity_embeddings.append(torch.zeros(768).to(device))
    assert len(collated_labels)==len(collated_entity_embeddings)
    return collated_labels,collated_entity_embeddings

In [ ]:
# trainset, tokenizedtrainset, tweet_to_sentences_w_annotation = preprocess('wnut17test_ner.csv')
# trainset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('wnut17train_ner.csv')
trainset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('deduplicated_test_WTypes.csv')

candidateBaseHeaders_alt=['candidate', 'batch', 'length','cumulative','class']

In [ ]:
predictions=[]
tokenized_sentences=[]
count=0
with torch.no_grad():
    for train_record in trainset:
        
        sentence = normalizeTweet(train_record)
        tweetWordList = sentence.split()
        enumerated_tweetWordList=[(token,idx) for idx,token in enumerate(tweetWordList)]

        tokenized_input=tokenizer(sentence)
        initial_input_ids = torch.tensor([tokenizer.encode(sentence)])

        initial_input_ids = initial_input_ids[:,:128]
        token_dict = {x : tokenizer.encode(x, add_special_tokens=False) for x in sentence.split()}

        input_ids = initial_input_ids.to(device)
        tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

        output = alt_model(input_ids)
        
        token_embeddings=output.hidden_states[-1].squeeze()[1:-1] # we dont need embeddings for CLS and EOS

        assert torch.isnan(token_embeddings).any() == False

        prediction = (torch.argmax(output.logits, axis=2))
        prediction = prediction.cpu().numpy().reshape(-1)
        if(count<=5):
            print(prediction)
        # prediction_labels=[label_list[l].split('-')[0] for l in prediction]
        prediction_labels=[label_list[l] for l in prediction]
        # prediction_labels=collate_token_labels(token_dict, prediction_labels[1:-1])
        prediction_labels, entity_aware_embeddings = collate_token_label_embedding(tweetWordList, token_dict, prediction_labels[1:-1], token_embeddings)

        # print(len(enumerated_tweetWordList),len(token_dict.keys()),len(entity_aware_embeddings),len(prediction_labels))

        assert len(enumerated_tweetWordList)==len(entity_aware_embeddings)
        assert len(prediction_labels)==len(enumerated_tweetWordList)

        predictions.append(prediction_labels)
        tokenized_sentences.append((tweetWordList,entity_aware_embeddings))

        if(count<=5):
            print(len(output.hidden_states))
            print(token_dict.keys())
            print(initial_input_ids)
            print(input_ids)
            print(prediction_labels)
            print('===============')
        count+=1

print(len(predictions),len(tokenized_sentences))

[0 0 0 3 0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 3 4 0 0]
13
dict_keys(['REPORT', ':', 'FBI', 'Obtained', 'FISA', 'Warrant', 'For', 'Trump', 'Aide', ',', 'Believed', 'He', 'Was', "'", 'Agent', 'Of', 'The', 'Russian', 'Government'])
tensor([[    0, 24537,    22,  5029,  6152, 33859, 47367,  4001,  7850,   287,
           394, 51322,     7, 15415, 28288,   162,  1050,    69, 12284,   490,
            47,  2768,  3521,    69,     2]])
tensor([[    0, 24537,    22,  5029,  6152, 33859, 47367,  4001,  7850,   287,
           394, 51322,     7, 15415, 28288,   162,  1050,    69, 12284,   490,
            47,  2768,  3521,    69,     2]], device='cuda:0')
['O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O']
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
13
dict_keys(['Well', ',', 'this', 'made', 'My', 'Day', 'Thank', 'you', 'Apple', ':', ')', '#', 'RelationshipSongs', 'NoStupidHeadsAllowed', 'InMyFreeTime'])
tensor([[  

In [ ]:
local_ner_arrays=[]
for n, sentence_tup in enumerate(tokenized_sentences):  
    sentence = sentence_tup[0]
    assert (len(sentence)==len(predictions[n]))
    word_tag_tuples=zip(sentence,predictions[n])
    entities_from_sentence=get_entities(word_tag_tuples)
    local_ner_arrays.append(entities_from_sentence)
print('tally:',len(tokenized_sentences),len(local_ner_arrays))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
/usr

tally: 5622 5622


In [ ]:
canonical_candidate_df=pd.read_csv("data/candidate_train_records_altClassifier_new.csv",sep =',', keep_default_na=False)
canonical_ne_list = canonical_candidate_df[(canonical_candidate_df['class']==0)&(canonical_candidate_df['cumulative']>=5)].candidate.tolist()


canonical_ne_list = set(canonical_ne_list)
canonical_ne_list.remove('ex-trump')
canonical_ne_list.remove('ex-trump adviser')
canonical_ne_list.remove('jr')
canonical_ne_list.remove('dt')
canonical_ne_list.remove('ic')
canonical_ne_list.remove('al')
canonical_ne_list.remove('report')
canonical_ne_list.remove('impeachment')
canonical_ne_list.remove('missiles')
canonical_ne_list.remove('america')
canonical_ne_list.remove('military')
canonical_ne_list.remove('congressional')
canonical_ne_list.remove('secretary of state')
canonical_ne_list.remove('america first')
canonical_ne_list.remove('easter bunny')
canonical_ne_list.remove('west')
canonical_ne_list.remove('east')
canonical_ne_list.remove('states')
canonical_ne_list.remove('white')
canonical_ne_list.remove('science')

print(canonical_ne_list)
print(len(canonical_ne_list))

entity_types_dict = {'ne':0, 'org':1, 'misc':2, 'loc':3, 'per':4}
type_candidate_dict = {entity_type:{} for entity_type in entity_types_dict}

{'tired', 'law', 'investigations', 'north', 'certainly', 'evil', 'blame', 'terror', 'neo', 'reputation', 'gone', 'getting', 'might want', 'spy', 'daughters', 'god', 'missile', 'public', 'mainstream', 'mouthpiece', 'called', 'ignorance', 'fms', 'ongoing', 'bet', 'day', 'aide carter', 'meets', 'mad', 'jab', 'find', 'inauguration', 'overwatch', 'beautiful', 'live', 'seeks', 'satan', 'launches', 'beyond', 'cable', 'knows', 'products', 'played', 'basket', 'cut', 're-accommodated', 'subject', 'staffers', 'platform', 'cabal', 'airstrikesγç', 'dad', 'admits', 'supporters', 'peace', 'win', 'repeated', 'pop', 'hired', 'falling', 'advisors', 'daughter', 'took', 'worse', 'mouth', 'morning', 'big', 'tweet', 'population', 'capital', 'comments', 'skittles', 'rise', 'clarify', 'back', 'pig', 'surveillance', 'gay', 'probe', 'lil', 'gif', 'club', 'butt', 'trump said', 'guy', 'ladies', 'sudden', 'pay', 'dummies', 'wanted', 'sean spicerγçös hitler', 'court jester', 'theory', 'lacks', 'daily', 'makes', 'wo

In [ ]:
# #comment this out if collecting samples for training the entity phrase embedder
# entityPhraseEmbedder.eval()

candidate_embedding_dict = defaultdict(list)
candidateBase_dict_alt = {} #<batch, length, cumulative, class>
non_entity_count = 0
entity_count = 0
for tweetID in tweet_to_sentences_w_annotation.keys():
    annotated_token_list = []
    annotated_mention_list=tweet_to_sentences_w_annotation[tweetID][1]
    tweet_token_list=[]
    token_embedding_list=[]
    local_mentions_list=[]
    idRange=tweet_to_sentences_w_annotation[tweetID][0]

    for sentID in range(idRange[0],idRange[1]):
        tweet_token_list+=tokenized_sentences[sentID][0]
        token_embedding_list+=tokenized_sentences[sentID][1]
        local_mentions_list+=local_ner_arrays[sentID]
    
    assert len(tweet_token_list)==len(token_embedding_list)
    print(tweetID, tweet_token_list)
    print('annotated_mention_list:',annotated_mention_list)

    
    annotated_candidates = set([mention_tup[0].lower().strip() for mention_tup in annotated_mention_list])
    for mention_tup in annotated_mention_list:
        annotated_token_list+= mention_tup[0].lower().strip().split()
    annotated_token_list = set(annotated_token_list)

    #------------------------ negative candidate mining ------------------------
    non_entity_list = list(set([elem.strip().lower() for elem in tweet_token_list]).intersection(canonical_ne_list)-annotated_candidates)

    
    if(non_entity_list):
        print('non_entity_list:',non_entity_list)
    
    tweet_token_ind = 0
    while((len(non_entity_list)>0)&(tweet_token_ind < len(tweet_token_list))):
        #must pop from the left
        non_entity_candidate = non_entity_list.pop(0)
        non_entity_candidate, nonentity_type = non_entity_candidate.lower(),'ne'
        non_entity_candidate_tokens = non_entity_candidate.split()
        while(tweet_token_ind < len(tweet_token_list)):
            if((' '.join(tweet_token_list[tweet_token_ind:tweet_token_ind+len(non_entity_candidate_tokens)])).strip().lower()!=non_entity_candidate):
                # print(tweet_token_ind)
                tweet_token_ind+=1
            else:
                ne_candidate=(' '.join(tweet_token_list[tweet_token_ind:tweet_token_ind+len(non_entity_candidate_tokens)])).strip().lower()
                ne_candidate_token_embeddings = torch.stack(token_embedding_list[tweet_token_ind:tweet_token_ind+len(non_entity_candidate_tokens)]) #2d tensor of ne candidate token embeddings
                
                tensor_inter = torch.mean(ne_candidate_token_embeddings,dim=0)
                tensor_inter_norm = torch.norm(tensor_inter, p=2,dim=0)
                ne_candidate_tokens_avg_embedding = tensor_inter/tensor_inter_norm

                assert torch.isnan(ne_candidate_tokens_avg_embedding).any() == False

                # print(ne_candidate, ne_candidate_tokens_avg_embedding.shape)
                if((non_entity_candidate, nonentity_type) not in candidate_embedding_dict):
                    non_entity_count+=1
                    candidateBase_dict_alt[non_entity_candidate+'||'+nonentity_type] = [0,len(non_entity_candidate_tokens),1,entity_types_dict[nonentity_type]]
                else:
                    candidateBase_dict_alt[non_entity_candidate+'||'+nonentity_type][2] += 1
                
                #---------------------- When training the entity phrase embedder ----------------------
                # !!no need to unsqueeze here since not training on STS padded data
                candidate_embedding_dict[(non_entity_candidate, nonentity_type)].append(ne_candidate_tokens_avg_embedding)
                type_candidate_dict[nonentity_type][non_entity_candidate]=True

                # #---------------------- When collecting mention phrase embeddings to train the entity classifier ----------------------
                # # !! must unsqueeze at test time with embedder else batchnorm throws error
                # ne_candidate_tokens_avg_embedding = ne_candidate_tokens_avg_embedding.unsqueeze(0)
                # ne_candidate_phrase_embedding = (entityPhraseEmbedder.getEmbedding(ne_candidate_tokens_avg_embedding)).squeeze(0)
                # # print(ne_candidate_tokens_avg_embedding.shape, ne_candidate_phrase_embedding.shape)
                # candidate_embedding_dict[(non_entity_candidate, nonentity_type)].append(ne_candidate_phrase_embedding)

                tweet_token_ind+=len(non_entity_candidate_tokens)
                break
    #------------------------ negative candidate mining ------------------------

    #------------------------ positive candidate mining ------------------------
    tweet_token_ind = 0
    while((len(annotated_mention_list)>0)&(tweet_token_ind < len(tweet_token_list))):
        #must pop from the left
        annotated_candidate_tuple = annotated_mention_list.pop(0)
        annotated_candidate, entity_type = annotated_candidate_tuple[0].lower(),annotated_candidate_tuple[1].lower()
        candidate_tokens = annotated_candidate.split()

        print(annotated_candidate)
        while((' '.join(tweet_token_list[tweet_token_ind:tweet_token_ind+len(candidate_tokens)])).strip().lower()!=annotated_candidate):
            tweet_token_ind+=1
        candidate=(' '.join(tweet_token_list[tweet_token_ind:tweet_token_ind+len(candidate_tokens)])).strip().lower()
        candidate_token_embeddings = torch.stack(token_embedding_list[tweet_token_ind:tweet_token_ind+len(candidate_tokens)]) #2d tensor of candidate token embeddings
        tensor_inter = torch.mean(candidate_token_embeddings,dim=0)
        tensor_inter_norm = torch.norm(tensor_inter, p=2,dim=0)
        candidate_tokens_avg_embedding = tensor_inter/tensor_inter_norm

        assert torch.isnan(candidate_tokens_avg_embedding).any() == False
        # print(candidate,annotated_candidate,candidate_token_embeddings.shape)

        if((annotated_candidate, entity_type) not in candidate_embedding_dict):
            entity_count+=1
            candidateBase_dict_alt[annotated_candidate+'||'+entity_type] = [0,len(candidate_tokens),1,entity_types_dict[entity_type]]
        else:
            candidateBase_dict_alt[annotated_candidate+'||'+entity_type][2] += 1

        #---------------------- When training the entity phrase embedder ----------------------
        # !!no need to unsqueeze here since not training on STS padded data
        candidate_embedding_dict[(annotated_candidate, entity_type)].append(candidate_tokens_avg_embedding)
        type_candidate_dict[entity_type][annotated_candidate]=True

        # #---------------------- When collecting mention phrase embeddings to train the entity classifier ----------------------
        # # !! must unsqueeze at test time with embedder else batchnorm throws error
        # candidate_tokens_avg_embedding = candidate_tokens_avg_embedding.unsqueeze(0)
        # candidate_phrase_embedding = (entityPhraseEmbedder.getEmbedding(candidate_tokens_avg_embedding)).squeeze(0)
        # # print(candidate_tokens_avg_embedding.shape, candidate_phrase_embedding.shape)
        # candidate_embedding_dict[(annotated_candidate, entity_type)].append(candidate_phrase_embedding)

        tweet_token_ind+=len(candidate_tokens)
    #------------------------ positive candidate mining ------------------------


Streaming output truncated to the last 5000 lines.
spotify
2184 ['WHO', 'KNOWS', 'WHAT', 'HOTEL', 'Pinkfloyd', 'were', 'STAYING', 'AT', 'ON', 'SUNDAY', 'IN', 'Madrid', '?']
annotated_mention_list: [('pinkfloyd', 'misc'), ('madrid', 'loc')]
non_entity_list: ['knows']
pinkfloyd
madrid
2185 ['Carter', 'Page', 'said', 'that', 'the', 'Steele', 'dossier', 'was', '"', 'little', 'more', 'than', 'a', 'comedic', 'distraction', 'for', 'months', 'on', 'end', '"']
annotated_mention_list: [('carter page', 'per'), ('steele dossier', 'misc')]
non_entity_list: ['distraction', 'said', 'little', 'end']
carter page
steele dossier
2186 ['@USER', 'Carter', 'Page', 'tuning', 'up', 'for', 'his', 'big', 'number', '.']
annotated_mention_list: [('carter page', 'per')]
non_entity_list: ['big']
carter page
2187 ['A', 'FISA', 'warrant', 'for', 'the', 'guy', 'who', 'was', 'one', 'of', 'Trump', "'s", 'few', 'named', 'foreign', 'policy', 'advisers', '.', 'So', 'chew', 'on', 'that', '.']
annotated_mention_list: [('fisa

In [ ]:
print('Found '+str(len(candidate_embedding_dict))+' candidates in training set')
print('Found '+str(entity_count)+' entities in training set')
print('Found '+str(non_entity_count)+' non entities in training set')
print('========')

Found 1760 candidates in training set
Found 1133 entities in training set
Found 627 non entities in training set


## **Training the Entity Phrase Embedder**



In [ ]:
print(candidateBase_dict_alt)

{'agent||ne': [0, 1, 45, 0], 'government||ne': [0, 1, 16, 0], 'fbi||org': [0, 1, 560, 1], 'fisa warrant||misc': [0, 2, 385, 2], 'trump||per': [0, 1, 819, 4], 'russian government||org': [0, 2, 5, 1], 'day||ne': [0, 1, 17, 0], 'apple||org': [0, 1, 6, 1], 'team||ne': [0, 1, 33, 0], 'carter page||per': [0, 2, 1202, 4], 'time||ne': [0, 1, 38, 0], 'tomahawks||misc': [0, 1, 3, 2], 'sean spicer||per': [0, 2, 62, 4], 'foreign||ne': [0, 1, 37, 0], 'session||per': [0, 1, 1, 4], 'gave||ne': [0, 1, 11, 0], 'flynn||per': [0, 1, 26, 4], 'russians||misc': [0, 1, 37, 2], 'social||ne': [0, 1, 8, 0], 'us||loc': [0, 1, 60, 3], 'see||ne': [0, 1, 41, 0], 'go||ne': [0, 1, 31, 0], 'problem||ne': [0, 1, 1, 0], 'susan rice||per': [0, 2, 9, 4], 'tower||ne': [0, 1, 7, 0], 'trump tower||loc': [0, 2, 7, 3], 'going||ne': [0, 1, 56, 0], 'hack||ne': [0, 1, 2, 0], 'steele||per': [0, 1, 1, 4], 'told||ne': [0, 1, 9, 0], 'hire||ne': [0, 1, 2, 0], 'aide||ne': [0, 1, 15, 0], 'last||ne': [0, 1, 42, 0], 'adviser||ne': [0, 1, 

In [ ]:
# #softknn TUPLET MINING
# printonce=0
# candidate_index_dict = {}
# index_candidate_dict = {}

# for ind, elem in enumerate(list(candidate_embedding_dict.keys())):
#     candidate_index_dict[elem] = ind
#     index_candidate_dict[ind] = elem

# datalist=[] #schema: anchor, positive, n-negatives
# for candidate_tup in candidate_embedding_dict.keys():
#     if(len(candidate_embedding_dict[candidate_tup])>1):
#         candidate, candidate_type = candidate_tup[0], candidate_tup[1]
#         mention_embeddings_list = candidate_embedding_dict[candidate_tup]
#         if(len(mention_embeddings_list)>300):
#             mention_index_random = random.sample(list(range(len(mention_embeddings_list))), 300)
#         else:
#             mention_index_random = list(range(len(mention_embeddings_list)))
        
#         for ind in range(len(mention_embeddings_list)):
#             negative_candidates_list=[]
#             for entity_type in entity_types_dict:
#                 if(candidate_type!=entity_type):
#                     if(candidate in type_candidate_dict[entity_type]):
#                         print('=>',candidate,candidate_type,entity_type)
#                         neg_candidate_tup = (candidate,entity_type)
#                     else:
#                         negative_candidate = random.choice(list(type_candidate_dict[entity_type]))
#                         neg_candidate_tup = (negative_candidate,entity_type)
#                     negative_candidates_list.append(candidate_index_dict[neg_candidate_tup])
#             tuplet = tuple([ind,candidate_index_dict[candidate_tup]]+negative_candidates_list)
#             if(printonce<1):
#                 print(tuplet)
#                 printonce+=1
#             datalist.append(tuplet)
# print('dataset length is ',len(datalist))
# shuffle(datalist)
# print('dataset length is ',len(datalist))

In [ ]:
# #N+1 TUPLET MINING
# datalist=[] #schema: anchor, positive, n-negatives
# for candidate_tup in candidate_embedding_dict.keys():
#     if(len(candidate_embedding_dict[candidate_tup])>1):
#         candidate, candidate_type = candidate_tup[0], candidate_tup[1]
#         mention_embeddings_list = candidate_embedding_dict[candidate_tup]
#         if(len(mention_embeddings_list)>300):
#             mention_embeddings_list_random = random.sample(mention_embeddings_list, 300)
#         else:
#             mention_embeddings_list_random = mention_embeddings_list

#         tuplets_size = 1
#         negatives_arr = []
#         for entity_type in entity_types_dict:
#             if(candidate_type!=entity_type):
#                 if(candidate in type_candidate_dict[entity_type]):
#                     print('=>',candidate,candidate_type,entity_type)
#                     neg_candidate_tup = (candidate,entity_type)
#                 else:
#                     negative_candidate = random.choice(list(type_candidate_dict[entity_type]))
#                     neg_candidate_tup = (negative_candidate,entity_type)
#                 negative_embedding_list = candidate_embedding_dict[neg_candidate_tup]

#                 if(len(negative_embedding_list)>350):
#                     negative_embedding_list_random = random.sample(negative_embedding_list, 350)
#                 else:
#                     negative_embedding_list_random = negative_embedding_list
#                 tuplets_size *= len(negative_embedding_list_random)
#                 negatives_arr.append(negative_embedding_list_random)
        
#         limit = min(500,tuplets_size)
#         negatives_list = []
#         while(limit>0):
#             inner=[]
#             for i in range(4):
#                 inner.append(random.choice(negatives_arr[i]))
#             negatives_list.append(inner)
#             limit-=1

#         for ind1, elem in enumerate(mention_embeddings_list_random):
#             for ind2, elem in enumerate(mention_embeddings_list_random):
#                 if(ind1!=ind2):
#                     for neg_tuplet in negatives_list:
#                         tuplet = tuple([mention_embeddings_list_random[ind1],mention_embeddings_list_random[ind2]]+ neg_tuplet)
#                         datalist.append(tuplet)

# print('dataset length is ',len(datalist))
# shuffle(datalist)
# print('dataset length is ',len(datalist))

In [ ]:
#TRIPLET MINING

datalist=[] #schema: anchor, positive, negative
for candidate_tup in candidate_embedding_dict.keys():
    if(len(candidate_embedding_dict[candidate_tup])>1):
        candidate, candidate_type = candidate_tup[0], candidate_tup[1]
        mention_embeddings_list = candidate_embedding_dict[candidate_tup]
        if(len(mention_embeddings_list)>350):
            mention_embeddings_list_random = random.sample(mention_embeddings_list, 350)
        else:
            mention_embeddings_list_random = mention_embeddings_list
        
        for ind1, elem in enumerate(mention_embeddings_list_random):
            for ind2, elem in enumerate(mention_embeddings_list_random):
                if(ind1!=ind2):
                    for entity_type in entity_types_dict:
                        if(candidate_type!=entity_type):
                            if(candidate in type_candidate_dict[entity_type]):
                                print('=>',candidate,candidate_type,entity_type)
                                neg_candidate_tup = (candidate,entity_type)
                            else:
                                negative_candidate = random.choice(list(type_candidate_dict[entity_type]))
                                neg_candidate_tup = (negative_candidate,entity_type)
                            negative_embedding_list = candidate_embedding_dict[neg_candidate_tup]

                            if(len(negative_embedding_list)>350):
                                # negative_embedding_list_random = random.shuffle(negative_embedding_list)
                                negative_embedding_list_random = random.sample(negative_embedding_list, 350)
                            else:
                                negative_embedding_list_random = negative_embedding_list
                            
                            #For triplets
                            interlist = [(mention_embeddings_list_random[ind1],mention_embeddings_list_random[ind2],negative_elem) for negative_elem in negative_embedding_list_random]
                            datalist.extend(interlist)
print('dataset length is ',len(datalist))
shuffle(datalist)
print('dataset length is ',len(datalist))

=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne
=> us loc ne

In [ ]:
#Hyperparameters
learning_rate = 0.001 #0.001
num_epochs = 50
patience = 8
batch_size = 32

In [ ]:
class PhraseEmbeddingDataset(torch.utils.data.Dataset):

    def __init__(self, input):

        self.input = input
        #input schema is a triplet of anchor|positive|negative examples
        print(type(self.input))

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        X = self.input[idx]
        # y = self.output[idx]
        # return X,y
        return X

In [ ]:
full_dataset = PhraseEmbeddingDataset(datalist)
train_size = int(0.75 * len(full_dataset))
validation_size = len(full_dataset) - train_size
print('Training set:', train_size)
print('Validation set:', validation_size)

# print(type(training_set))
val_batch=int(validation_size/11)

<class 'list'>
Training set: 10974279
Validation set: 3658094


In [ ]:
training_set, validation_set = torch.utils.data.random_split(full_dataset, [train_size, validation_size])

training_generator = torch.utils.data.DataLoader(training_set, batch_size=2048, shuffle=False)# 512 for soft-knn and 2048 for triplet
validation_generator = torch.utils.data.DataLoader(validation_set, batch_size=val_batch, shuffle=True)

In [ ]:
def save_ckp(state, is_best, checkpoint_dir):
    # f_path = checkpoint_dir + '/checkpoint.pt' 
    f_path = checkpoint_dir + '/checkpoint_model300.pt'
    # f_path = checkpoint_dir + '/checkpoint_model300_softknn.pt'
    torch.save(state, f_path)

In [ ]:
# # soft knn
# candidate_tup = index_candidate_dict[datalist[1430][1]]
# embeddingSize = list(candidate_embedding_dict[candidate_tup][datalist[1430][0]].shape)[0]

# triplet
embeddingSize = datalist[1430][0].shape[0]
output_embedding_size = 300
print(embeddingSize,output_embedding_size)

768 300


In [ ]:
def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_uniform_(m.weight.data,nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Initialize network
phraseEmbeddingModel = PhraseEmbedding(embeddingSize, output_embedding_size, device).to(device)
# phraseEmbeddingModel.apply(initialize_weights)

#Loss and Optimizer
optimizer = optim.Adam(phraseEmbeddingModel.parameters(), lr=learning_rate, weight_decay=0.0001)
criterion = TripletLoss()

In [ ]:
# phraseEmbeddingModel = PhraseEmbeddingI(embeddingSize, output_embedding_size, device,candidate_embedding_dict,index_candidate_dict).to(device)

# #Loss and Optimizer
# optimizer = optim.Adam(phraseEmbeddingModel.parameters(), lr=learning_rate, weight_decay=0.0005)
# criterion = SoftKNNLoss()

In [ ]:
checkpoint_dir='entityEmbedding/model_checkpoints_ner'
ckp_path = "entityEmbedding/model_checkpoints_ner/checkpoint_model300.pt" #300
# ckp_path = "entityEmbedding/model_checkpoints_ner/checkpoint_model300_softknn.pt" #300

# if(path.exists(ckp_path)):
#     # load the saved checkpoint
#     phraseEmbeddingModel, optimizer, start_epoch = load_ckp(ckp_path, phraseEmbeddingModel, optimizer)

#     print("starting with model at epoch:", start_epoch)

In [ ]:
# Train Network
history_training= []
history_validation = []
best_loss = np.float('inf')
best_checkpoint = {}

for epoch in range(num_epochs):
    training_batch_loss=[]
    mbcount=0
    for batch_idx, batch_data in enumerate(training_generator):
        # print(type(data),len(data),len(data[0]),len(data[1]),len(data[2]))
        optimizer.zero_grad()
        out = phraseEmbeddingModel(batch_data)
        # print('network output type:',type(out))
        loss = criterion(out)
        # print('mini batch loss:',loss.item())
        training_batch_loss.append(loss.item())
         
         

        # backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(phraseEmbeddingModel.parameters(), max_norm=2.0, norm_type=2)
        # gradient descent or adam step
        optimizer.step()
        mbcount+=1
        if(mbcount%1000==0):
            print(mbcount//1000)
    combined_training_loss = np.mean(training_batch_loss)
    print('\nEpoch',str(epoch+1),' Training Loss:', combined_training_loss)
    history_training.append(combined_training_loss)

    #Validation: DO NOT BACKPROPAGATE HERE
    validation_batch_loss = []
    print_only_one=True
    with torch.no_grad():
        for batch_idx, data in enumerate(validation_generator):
            # if(print_only_one):
                # print(len(data[0]),len(data[1]))
                # print(type(data))
                # print_only_one=False
            out = phraseEmbeddingModel(data)
            loss = criterion(out)
            validation_batch_loss.append(loss.item())
            # print(validation_batch_loss)
    combined_validation_loss= np.mean(validation_batch_loss)
    history_validation.append(combined_validation_loss)
    # if(((epoch+1)%10==0)|(epoch == (num_epochs-1))):
    print('\nEpoch',str(epoch+1),' Validation Loss:',combined_validation_loss)
    print('=====================================================================\n')
    
    if(combined_validation_loss<best_loss):
        best_loss = combined_validation_loss
        print('saving this checkpoint')
        best_checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': phraseEmbeddingModel.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        no_improvement_counter=0
    else:
        no_improvement_counter+=1
        if(no_improvement_counter>patience):
            break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


1
2
3
4
5

Epoch 1  Training Loss: 0.014150862475149132

Epoch 1  Validation Loss: 0.006315389351749962

saving this checkpoint
1
2
3
4
5

Epoch 2  Training Loss: 0.005428125148264225

Epoch 2  Validation Loss: 0.0031699858690527353

saving this checkpoint
1
2
3
4
5

Epoch 3  Training Loss: 0.0038540469251013842

Epoch 3  Validation Loss: 0.0030689049427482214

saving this checkpoint
1
2
3
4
5

Epoch 4  Training Loss: 0.003507194656971338

Epoch 4  Validation Loss: 0.002983760977671905

saving this checkpoint
1
2
3
4
5

Epoch 5  Training Loss: 0.0033275638045506756

Epoch 5  Validation Loss: 0.002321392572908239

saving this checkpoint
1
2
3
4
5

Epoch 6  Training Loss: 0.0032001936546549267

Epoch 6  Validation Loss: 0.002383294684643095

1
2
3
4
5

Epoch 7  Training Loss: 0.003093651891434032

Epoch 7  Validation Loss: 0.005676613231612878

1
2
3
4
5

Epoch 8  Training Loss: 0.003092365155336615

Epoch 8  Validation Loss: 0.00228909849697216

saving this checkpoint
1
2
3
4
5

Epoch 9

In [ ]:
save_ckp(best_checkpoint, True, checkpoint_dir) 

In [ ]:
# # tensor = -1*torch.Tensor([3,2,1])
# # sum = torch.sum(tensor) 
# # sum_int = int(sum.item())
# # print(tensor, type(sum), sum_int)

# def nxn_cos_sim(A, B, dim=1, eps=1e-8):
#       numerator = A @ B.T
#       A_l2 = torch.mul(A, A).sum(axis=dim)
#       B_l2 = torch.mul(B, B).sum(axis=dim)
#       denominator = torch.max(torch.sqrt(torch.outer(A_l2, B_l2)), torch.tensor(eps))
#       return torch.div(numerator, denominator)

# example_2D_list1 = [[0.2423,-0.6679,0.8277],[0.2423,-0.6679,0.8277],[0.2423,-0.6679,0.8277]]
# example_2D_Tensor1 = torch.tensor(example_2D_list1)

# example_2D_list2 = [[0.2245,-2.2268,0.4577],[0.2245,-2.2268,0.3295],[0.2245,-2.2268,0.4577]]
# example_2D_Tensor2 = torch.tensor(example_2D_list2)


# sim_matrix = nxn_cos_sim(example_2D_Tensor1,example_2D_Tensor2)
# print(sim_matrix)

# for i in range(sim_matrix.size()[0]):
#     print(i,sim_matrix[i])
#     exp_tensor = torch.exp(sim_matrix[i])
#     print(exp_tensor)
#     sum_tensor = torch.sum(exp_tensor)
#     print(sum_tensor)
#     weights = torch.div(exp_tensor,sum_tensor)
#     print('weights:',weights)
#     print(weights[:,None]*example_2D_Tensor2)

## **Training the Entity Classifier**

In [ ]:
trainset_ec, tokenizedtestset_ec, tweet_to_sentences_w_annotation_ec = preprocess('deduplicated_test_WTypes.csv')
candidateBaseHeaders_alt=['candidate', 'batch', 'length','cumulative','class']

predictions=[]
tokenized_sentences=[]
count=0
with torch.no_grad():
    for train_record in trainset_ec:
        
        sentence = normalizeTweet(train_record)
        tweetWordList = sentence.split()
        enumerated_tweetWordList=[(token,idx) for idx,token in enumerate(tweetWordList)]

        tokenized_input=tokenizer(sentence)
        initial_input_ids = torch.tensor([tokenizer.encode(sentence)])

        initial_input_ids = initial_input_ids[:,:128]
        token_dict = {x : tokenizer.encode(x, add_special_tokens=False) for x in sentence.split()}

        input_ids = initial_input_ids.to(device)
        tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

        output = alt_model(input_ids)
        
        token_embeddings=output.hidden_states[-1].squeeze()[1:-1] # we dont need embeddings for CLS and EOS

        prediction = (torch.argmax(output.logits, axis=2))
        prediction = prediction.cpu().numpy().reshape(-1)
        # prediction_labels=[label_list[l].split('-')[0] for l in prediction]
        prediction_labels=[label_list[l] for l in prediction]
        # prediction_labels=collate_token_labels(token_dict, prediction_labels[1:-1])
        prediction_labels, entity_aware_embeddings = collate_token_label_embedding(tweetWordList, token_dict, prediction_labels[1:-1], token_embeddings)

        # print(len(enumerated_tweetWordList),len(token_dict.keys()),len(entity_aware_embeddings),len(prediction_labels))

        assert len(enumerated_tweetWordList)==len(entity_aware_embeddings)
        assert len(prediction_labels)==len(enumerated_tweetWordList)

        predictions.append(prediction_labels)
        tokenized_sentences.append((tweetWordList,entity_aware_embeddings))
        count+=1

print(len(predictions),len(tokenized_sentences))

local_ner_arrays=[]
for n, sentence_tup in enumerate(tokenized_sentences):  
    sentence = sentence_tup[0]
    assert (len(sentence)==len(predictions[n]))
    word_tag_tuples=zip(sentence,predictions[n])
    entities_from_sentence=get_entities(word_tag_tuples)
    local_ner_arrays.append(entities_from_sentence)
print('tally:',len(tokenized_sentences),len(local_ner_arrays))

In [ ]:
phraseEmbeddingModel = PhraseEmbedding(768, output_embedding_size, device).to(device) #triplet
# # phraseEmbeddingModel = PhraseEmbeddingI(768, output_embedding_size, device).to(device) #soft-knn
# phraseEmbeddingModel.apply(initialize_weights)

# #Loss and Optimizer
optimizer = optim.Adam(phraseEmbeddingModel.parameters(), lr=learning_rate, weight_decay=0.0001)

# # define checkpoint saved path for entity phrase embedder
ckp_path = "entityEmbedding/model_checkpoints_ner/checkpoint_model300.pt" #300 triplet
# # ckp_path = "entityEmbedding/model_checkpoints_ner/checkpoint_model300_softknn.pt" #300 tuplet

if(path.exists(ckp_path)):
    # load the saved checkpoint
    entityPhraseEmbedder, optimizer, start_epoch = load_ckp(ckp_path, phraseEmbeddingModel, optimizer)
    print("starting with model at epoch:", start_epoch)

#comment this out if collecting samples for training the entity phrase embedder
entityPhraseEmbedder.eval()

candidate_embedding_dict = defaultdict(list)
candidateBase_dict_alt_ec = {} #<batch, length, cumulative, class>
non_entity_count = 0
entity_count = 0
for tweetID in tweet_to_sentences_w_annotation_ec.keys():
    annotated_token_list = []
    annotated_mention_list=tweet_to_sentences_w_annotation_ec[tweetID][1]
    tweet_token_list=[]
    token_embedding_list=[]
    local_mentions_list=[]
    idRange=tweet_to_sentences_w_annotation_ec[tweetID][0]

    for sentID in range(idRange[0],idRange[1]):
        tweet_token_list+=tokenized_sentences[sentID][0]
        token_embedding_list+=tokenized_sentences[sentID][1]
        local_mentions_list+=local_ner_arrays[sentID]
    
    assert len(tweet_token_list)==len(token_embedding_list)
    print(tweetID, tweet_token_list)
    print('annotated_mention_list:',annotated_mention_list)

    
    annotated_candidates = set([mention_tup[0].lower().strip() for mention_tup in annotated_mention_list])
    for mention_tup in annotated_mention_list:
        annotated_token_list+= mention_tup[0].lower().strip().split()
    annotated_token_list = set(annotated_token_list)

    #------------------------ negative candidate mining ------------------------
    non_entity_list = list(set([elem.strip().lower() for elem in tweet_token_list]).intersection(canonical_ne_list)-annotated_candidates)

    
    if(non_entity_list):
        print('non_entity_list:',non_entity_list)
    
    tweet_token_ind = 0
    while((len(non_entity_list)>0)&(tweet_token_ind < len(tweet_token_list))):
        #must pop from the left
        non_entity_candidate = non_entity_list.pop(0)
        non_entity_candidate, nonentity_type = non_entity_candidate.lower(),'ne'
        non_entity_candidate_tokens = non_entity_candidate.split()
        while(tweet_token_ind < len(tweet_token_list)):
            if((' '.join(tweet_token_list[tweet_token_ind:tweet_token_ind+len(non_entity_candidate_tokens)])).strip().lower()!=non_entity_candidate):
                # print(tweet_token_ind)
                tweet_token_ind+=1
            else:
                ne_candidate=(' '.join(tweet_token_list[tweet_token_ind:tweet_token_ind+len(non_entity_candidate_tokens)])).strip().lower()
                ne_candidate_token_embeddings = torch.stack(token_embedding_list[tweet_token_ind:tweet_token_ind+len(non_entity_candidate_tokens)]) #2d tensor of ne candidate token embeddings
                
                tensor_inter = torch.mean(ne_candidate_token_embeddings,dim=0)
                tensor_inter_norm = torch.norm(tensor_inter, p=2,dim=0)
                ne_candidate_tokens_avg_embedding = tensor_inter/tensor_inter_norm

                

                # print(ne_candidate, ne_candidate_tokens_avg_embedding.shape)
                if((non_entity_candidate, nonentity_type) not in candidate_embedding_dict):
                    non_entity_count+=1
                    candidateBase_dict_alt_ec[non_entity_candidate+'||'+nonentity_type] = [0,len(non_entity_candidate_tokens),1,entity_types_dict[nonentity_type]]
                else:
                    candidateBase_dict_alt_ec[non_entity_candidate+'||'+nonentity_type][2] += 1
                

                #---------------------- When collecting mention phrase embeddings to train the entity classifier ----------------------
                # !! must unsqueeze at test time with embedder else batchnorm throws error
                ne_candidate_tokens_avg_embedding = ne_candidate_tokens_avg_embedding.unsqueeze(0)
                assert torch.isnan(ne_candidate_tokens_avg_embedding).any() == False
                ne_candidate_phrase_embedding = (entityPhraseEmbedder.getEmbedding(ne_candidate_tokens_avg_embedding)).squeeze(0)
                assert torch.isnan(ne_candidate_phrase_embedding).any() == False
                # print(ne_candidate_tokens_avg_embedding.shape, ne_candidate_phrase_embedding.shape)
                candidate_embedding_dict[(non_entity_candidate, nonentity_type)].append(ne_candidate_phrase_embedding)

                tweet_token_ind+=len(non_entity_candidate_tokens)
                break
    #------------------------ negative candidate mining ------------------------

    #------------------------ positive candidate mining ------------------------
    tweet_token_ind = 0
    while((len(annotated_mention_list)>0)&(tweet_token_ind < len(tweet_token_list))):
        #must pop from the left
        annotated_candidate_tuple = annotated_mention_list.pop(0)
        annotated_candidate, entity_type = annotated_candidate_tuple[0].lower(),annotated_candidate_tuple[1].lower()
        candidate_tokens = annotated_candidate.split()

        print(annotated_candidate)
        while((' '.join(tweet_token_list[tweet_token_ind:tweet_token_ind+len(candidate_tokens)])).strip().lower()!=annotated_candidate):
            tweet_token_ind+=1
        candidate=(' '.join(tweet_token_list[tweet_token_ind:tweet_token_ind+len(candidate_tokens)])).strip().lower()
        candidate_token_embeddings = torch.stack(token_embedding_list[tweet_token_ind:tweet_token_ind+len(candidate_tokens)]) #2d tensor of candidate token embeddings
        tensor_inter = torch.mean(candidate_token_embeddings,dim=0)
        tensor_inter_norm = torch.norm(tensor_inter, p=2,dim=0)
        candidate_tokens_avg_embedding = tensor_inter/tensor_inter_norm
        # print(candidate,annotated_candidate,candidate_token_embeddings.shape)

        if((annotated_candidate, entity_type) not in candidate_embedding_dict):
            entity_count+=1
            candidateBase_dict_alt_ec[annotated_candidate+'||'+entity_type] = [0,len(candidate_tokens),1,entity_types_dict[entity_type]]
        else:
            candidateBase_dict_alt_ec[annotated_candidate+'||'+entity_type][2] += 1

        #---------------------- When collecting mention phrase embeddings to train the entity classifier ----------------------
        # !! must unsqueeze at test time with embedder else batchnorm throws error
        candidate_tokens_avg_embedding = candidate_tokens_avg_embedding.unsqueeze(0)
        assert torch.isnan(candidate_tokens_avg_embedding).any() == False
        candidate_phrase_embedding = (entityPhraseEmbedder.getEmbedding(candidate_tokens_avg_embedding)).squeeze(0)
        assert torch.isnan(candidate_phrase_embedding).any() == False
        # print(candidate_tokens_avg_embedding.shape, candidate_phrase_embedding.shape)
        candidate_embedding_dict[(annotated_candidate, entity_type)].append(candidate_phrase_embedding)

        tweet_token_ind+=len(candidate_tokens)
    #------------------------ positive candidate mining ------------------------


In [ ]:
print(len(candidateBase_dict_alt_ec))
print(candidateBase_dict_alt_ec.keys())
# candidateBaseHeaders_alt
candidate_featureBase_DF_alt = pd.DataFrame.from_dict(candidateBase_dict_alt_ec, orient='index')
candidate_featureBase_DF_alt.columns = candidateBaseHeaders_alt[1:]
candidate_featureBase_DF_alt.index.name = candidateBaseHeaders_alt[0]
candidate_featureBase_DF_alt  = candidate_featureBase_DF_alt.reset_index(drop=False)
print(candidate_featureBase_DF_alt.columns.tolist())
print(len(candidateBase_dict_alt_ec),len(candidate_embedding_dict))
print(candidate_featureBase_DF_alt.head())

entity_types = set(candidate_featureBase_DF_alt['class'].tolist())
for entity_type in entity_types:
    print(len(candidate_featureBase_DF_alt[candidate_featureBase_DF_alt['class']==entity_type]))

print('=============')


# entity_types = set(candidate_featureBase_DF_alt_filtered['class'].tolist())
# for entity_type in entity_types:
#     print(len(candidate_featureBase_DF_alt_filtered[candidate_featureBase_DF_alt_filtered['class']==entity_type]))
# print(len(candidate_featureBase_DF_alt_filtered))

In [ ]:
entity_classifier_alt = EntityClassifierAlt(True, device, candidate_featureBase_DF_alt, candidate_embedding_dict)

In [ ]:
# p= 0.8297872340425532
# r= 0.6933962264150944

# f=2*p*r/(p+r)
# print(f)

In [ ]:
# import torch.nn.functional as F
# data1=np.array([0.0,0.0,0.0])
# x_data1 = torch.tensor(data1)

# tensor_inter_norm = torch.norm(x_data1, p=2,dim=0)
# print(tensor_inter_norm)
# updated_norm = torch.add(tensor_inter_norm, 1e-6)
# print(x_data1/tensor_inter_norm)
# print(x_data1/updated_norm)

# data2=np.array([2.0,3.0,4.0])
# x_data2 = torch.tensor(data2)

# diff = x_data1-x_data2
# print(x_data1.shape,x_data2.shape,diff.shape)
# print(diff)
# losses=torch.relu(diff)
# print(losses)
# print(losses.mean())
# print('=====')
# # print(x_data.shape)
# # stacked=torch.stack([x_data1,x_data2])
# # print(stacked.shape)
# tensorlist = [x_data1,x_data2]
# # mean = torch.mean(torch.stack(tensorlist), axis=1)
# # sd = torch.std(torch.stack(tensorlist), axis=1)
# # print('mean:', mean)
# # print('sd:' , sd)
# for tensor in tensorlist:
#     print(tensor)
#     tensor_norm = torch.norm(tensor, p=2,dim=0)
#     print(tensor_norm)
#     print(tensor/tensor_norm)
#     print('----')


## **Phase I: Local NER to collect entity candidates and Token Contextual Embeddings**

In [ ]:
import nltk
import re
import emoji
from emoji import demojize
import string

import numpy as np
import pandas  as pd
from itertools import groupby
from operator import itemgetter
from collections import Iterable, OrderedDict
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from nltk.tokenize import TweetTokenizer
#from datasketch import MinHash, MinHashLSH
# import NE_candidate_module as ne
# import Mention

import time
import datetime
import copy
import trie as trie
import ast

from datasets import load_dataset, load_metric, ClassLabel, Sequence
import random
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, set_seed
import copy

# import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
# import matplotlib.pyplot as plt
from transformers import DataCollatorForTokenClassification

import math
import shutil

torch.autograd.set_detect_anomaly(True)

cachedStopWords = stopwords.words("english")
tempList=["i","and","or","other","since","hence","onto","another","across","unlike","anytime","were","you","then","still","till","nor","perhaps","probably","otherwise","until","sometimes","sometime","seem","cannot","seems","because","can","like","into","able","unable","either","neither","if","we","it","else","elsewhere","how","not","what","who","when","where","who's","who’s","let","today","tomorrow","tonight","let's","let’s","lets","know","make","oh","via","i","yet","must","mustnt","mustn't","mustn’t","i'll","i’ll","you'll","you’ll","we'll","we’ll","done","doesnt","doesn't","doesn’t","dont","don't","don’t","did","didnt","didn't","didn’t","much","without","could","couldn't","couldn’t","would","wouldn't","wouldn’t","should","shouldn't","souldn’t","shall","isn't","isn’t","hasn't","hasn’t","wasn't","wasn’t","also","let's","let’s","let","well","just","everyone","anyone","noone","none","someone","theres","there's","there’s","everybody","nobody","somebody","anything","else","elsewhere","something","nothing","everything","i'd","i’d","i’m","won't","won’t","i’ve","i've","they're","they’re","we’re","we're","we'll","we’ll","we’ve","we've","they’ve","they've","they’d","they'd","they’ll","they'll","again","you're","you’re","you've","you’ve","thats","that's",'that’s','here’s',"here's","what's","what’s","i’m","i'm","a","so","except","arn't","aren't","arent","this","when","it","it’s","it's","he's","she's","she'd","he'd","he'll","she'll","she’ll","many","can't","cant","can’t","even","yes","no","these","here","there","to","maybe","<hashtag>","<hashtag>.","ever","every","never","there's","there’s","whenever","wherever","however","whatever","always","although"]
for item in tempList:
    if item not in cachedStopWords:
        cachedStopWords.append(item)
cachedStopWords.remove("don")
# cachedStopWords.remove("your")
# cachedStopWords.remove("up")

cachedTitles = ["mr.","mr","mrs.","mrs","miss","ms","sen.","dr","dr.","prof.","president","congressman"]
prep_list=["of","v."]  #includes common conjunction as well
# prep_list=[]
# article_list=[]
article_list=["a","an","the"]
conjoiner=["de"]
day_list=["sunday","monday","tuesday","wednesday","thursday","friday","saturday","mon","tues","wed","thurs","fri","sat","sun"]
month_list=["january","february","march","april","may","june","july","august","september","october","november","december","jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
chat_word_list=["nope","gee","hmm","httpurl","n't","pls","bye","€","vs","ouch","am","pm","omw","cbb","tmi","http","https","tv","tmw","og","psst","b.s","thanku","em","ip","qft","ima","icymi","bdsm","ah","ive","qt","dj","dm","pts","pt","yrs","congrat","haueheuaeh","ahushaush","jr","please","retweet","2mrw","2moro","4get","ooh","reppin","idk","oops","yup","stfu","uhh","2b","dear","yay","btw","ahhh","b4","ugh","ty","cuz","coz","sorry","yea","asap","ur","bs","rt","lmfao","lfmao","slfmao","u","r","nah","umm","ummm","thank","thanks","congrats","whoa","rofl","ha","ok","okay","hey","hi","huh","ya","yep","yeah","fyi","duh","damn","lol","omg","congratulations","fucking","fuck","f*ck","wtf","wth","aka","wtaf","xoxo","rofl","imo","wow","fck","haha","hehe","hoho"]
string.punctuation=string.punctuation+'…‘’'


class LocalNERModule():
    def __init__(self, sentenceTokenizer, nerTokenizer, nerEngine, device):
        self.counter=0

        if(sentenceTokenizer):
            self.my_sentence_tokenizer = sentenceTokenizer
        else:
            nltk.download('gutenberg')
            gutenberg_text = ""
            for file_id in gutenberg.fileids():
                gutenberg_text += gutenberg.raw(file_id)
            tokenizer_trainer = PunktTrainer()
            tokenizer_trainer.INCLUDE_ALL_COLLOCS = True
            tokenizer_trainer.train(gutenberg_text)

            self.my_sentence_tokenizer = PunktSentenceTokenizer(tokenizer_trainer.get_params())
            self.my_sentence_tokenizer._params.abbrev_types.add('dr')
            self.my_sentence_tokenizer._params.abbrev_types.add('c.j')
            self.my_sentence_tokenizer._params.abbrev_types.add('u.s')
            self.my_sentence_tokenizer._params.abbrev_types.add('u.s.a')
            self.my_sentence_tokenizer._params.abbrev_types.add('ret')
            self.my_sentence_tokenizer._params.abbrev_types.add('rep')
            self.my_sentence_tokenizer._params.abbrev_types.add('mr')
            self.my_sentence_tokenizer._params.abbrev_types.add('ms')
            self.my_sentence_tokenizer._params.abbrev_types.add('mrs')
            self.my_sentence_tokenizer._params.abbrev_types.add('v')
            self.my_sentence_tokenizer._params.abbrev_types.add('vs')

        self.quickRegex=re.compile("[a-z]+")

        self.tweet_to_sentences_w_annotation = {}
        self.device = device
        self.apostrophe_list =["'s",'’s','s']
        # self.label_list = ['O','B','I']
        self.label_list = ['O','B-ORG','I-ORG','B-MISC','I-MISC','B-LOC','I-LOC','B-PER','I-PER']

        self.tweetTokenizer = TweetTokenizer()

        self.contextual_embeddings = {}

        print('Starting Local NER Engine!')
        self.expanded_label_dict={0:'O', 1:'B-corporation', 2:'I-corporation', 3:'B-creative-work', 4:'I-creative-work', 5:'B-group', 6:'I-group', 7:'B-location', 8:'I-location', 9:'B-person', 10:'I-person', 11:'B-product', 12:'I-product'}
        self.BIO_dict={'O':0,'B':1,'I':2}
        self.BIO_type_dict={'O':0, 'B-ORG':1, 'I-ORG':2, 'B-MISC':3, 'I-MISC':4, 'B-LOC':5, 'I-LOC':6, 'B-PER':7, 'I-PER':8}
        self.label_map_dict={'O':'O', 'B-corporation':'B-ORG', 'I-corporation':'I-ORG', 'B-creative-work':'B-MISC', 'I-creative-work':'I-MISC', 'B-group':'B-MISC', 'I-group':'I-MISC', 'B-location':'B-LOC', 'I-location':'I-LOC', 'B-person':'B-PER', 'I-person':'I-PER', 'B-product':'B-MISC', 'I-product':'I-MISC'}


        if((nerTokenizer is not None)&(nerEngine is not None)):
            self.nerTokenizer = nerTokenizer
            self.localNEREngine = nerEngine
        else:
            self.train_engine()

    def train_engine(self):
        task = "ner" # Should be one of "ner", "pos" or "chunk"
        model_checkpoint = "vinai/bertweet-base"
        batch_size = 16
        # set_seed(42)
        datasets = load_dataset("wnut_17")
        self.nerTokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
        label_all_tokens = True
        tokenized_datasets = datasets.map(self.tokenize_and_align_labels)
        data_collator = DataCollatorForTokenClassification(self.nerTokenizer)
        self.metric = load_metric("seqeval")
        self.localNEREngine = AutoModelForTokenClassification.from_pretrained("vinai/bertweet-base", output_hidden_states=True, num_labels=len(self.label_list))
        alt_training_args = TrainingArguments(
            f"test-{task}",
            evaluation_strategy = "epoch",
            learning_rate=1e-5,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=3,
            weight_decay=0.01,
        )
        alt_trainer = Trainer(
        self.localNEREngine,
        alt_training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        tokenizer=self.nerTokenizer,
        compute_metrics=self.compute_metrics
        )
        alt_trainer.train()

        # tokenizer.save_pretrained('test-ner/')
        # alt_model.save_pretrained('test-ner/')

    def tokenize_and_align_labels(self,example):
        
        tokenized_ds_input = self.nerTokenizer(example["tokens"], is_split_into_words=True)
        inputId_to_token_dict={}
        for index, token in enumerate(example["tokens"]):
            values=self.nerTokenizer.encode(token, add_special_tokens=False, truncation=True)
            for value in values:
                try:
                    inputId_to_token_dict[value].append(index)
                except KeyError:
                    inputId_to_token_dict[value]=[index]
        labels=[]
        for inputID in tokenized_ds_input['input_ids']:
            try:
                index_list=copy.deepcopy(inputId_to_token_dict[inputID])
                index_to_address=index_list.pop(0)

                label = self.BIO_type_dict[self.label_map_dict[self.expanded_label_dict[example['ner_tags'][index_to_address]]]]
                # label = example['ner_tags'][index_to_address]

                labels.append(label)
                inputId_to_token_dict[inputID]=index_list
            except KeyError:
                labels.append(-100)

        assert (len(tokenized_ds_input['input_ids']) == len(labels))
        tokenized_ds_input['labels']=labels
        
        return tokenized_ds_input

    def compute_metrics(self, p):
        # print(p.shape)
        output, labels = p

        # print(len(predictions))
        # print(predictions[0].shape)
        # for elem in predictions[1]:
        #   print(elem.shape)

        predictions, _ = output
        
        predictions = np.argmax(predictions, axis=2)

        # Remove ignored index (special tokens)
        true_predictions = [
            [self.label_list[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [self.label_list[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]

        results = self.metric.compute(predictions=true_predictions, references=true_labels)
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

    def is_float(self,string):
        try:
            f=float(string)
            if(f==0.0):
              return True
            else:
              return ((f) and (string.count(".")==1))
      #return True# True if string is a number with a dot
        except ValueError:  # if string is not a number
          return False

    def collate_token_label_embedding(self, tweetWordList, token_dict, prediction_labels, entity_embeddings):
        counter=0
        collated_labels=[]
        collated_entity_embeddings=[]
        for word in tweetWordList:
            vals=token_dict[word]
            # print(word,vals)
            if(counter<len(prediction_labels)):
                labels=prediction_labels[counter:counter+len(vals)]
                token_entity_embeddings=entity_embeddings[counter:counter+len(vals)]
        #         print(token_entity_embeddings.shape)
                mean_tensor = torch.mean(token_entity_embeddings,dim=0)
                mean_tensor[torch.isnan(mean_tensor)] = 0
                collated_entity_embeddings.append(mean_tensor)
        #         print(collated_entity_embeddings)
                if('I-PER' in labels):
                    collated_labels.append('I-PER')
                elif('I-LOC' in labels):
                    collated_labels.append('I-LOC')
                elif('I-ORG' in labels):
                    collated_labels.append('I-ORG')
                elif('I-MISC' in labels):
                    collated_labels.append('I-MISC')
                elif('B-PER' in labels):
                    collated_labels.append('B-PER')
                elif('B-LOC' in labels):
                    collated_labels.append('B-LOC')
                elif('B-ORG' in labels):
                    collated_labels.append('B-ORG')
                elif('B-MISC' in labels):
                    collated_labels.append('B-MISC')
                else:
                    collated_labels.append('O')
                counter+=len(vals)
            else:
                collated_labels.append('O')
                collated_entity_embeddings.append(torch.zeros(768).to(self.device))
        assert len(collated_labels)==len(collated_entity_embeddings)
        return collated_labels,collated_entity_embeddings


    def get_entities(self, word_tag_tuples):
        mentions=[]
        candidateMention=''
        positions=[]
        type_tag=''
        
        #emoji.get_emoji_regexp().sub(u'', candidateMention)
        for index, tup in enumerate(word_tag_tuples):
            candidate=tup[0]
            tag=tup[1]
            if(tag=='O'):
                if(candidateMention):
                    if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
                        mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
                        if mention_to_add.endswith("'s"):
                            li = mention_to_add.rsplit("'s", 1)
                            mention_to_add=''.join(li)
                        elif mention_to_add.endswith("’s"):
                            li = mention_to_add.rsplit("’s", 1)
                            mention_to_add=''.join(li)
                        else:
                            mention_to_add=mention_to_add
                        if(mention_to_add!=''):
                            try:
                                assert len(mention_to_add.split()) == len(positions)
                                mentions.append((mention_to_add,type_tag,positions))
                            except AssertionError:
                                print(word_tag_tuples)
                                print(mention_to_add,type_tag,positions)
                                return
                candidateMention=''
                positions=[]
                type_tag=''
            else:
                boundary_tag = tag.split('-')[0]
                type_tag = tag.split('-')[1]
                if (boundary_tag=='B'):
                    if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
                        mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
                        if mention_to_add.endswith("'s"):
                            li = mention_to_add.rsplit("'s", 1)
                            mention_to_add=''.join(li)
                        elif mention_to_add.endswith("’s"):
                            li = mention_to_add.rsplit("’s", 1)
                            mention_to_add=''.join(li)
                        else:
                            mention_to_add=mention_to_add
                        if(mention_to_add!=''):
                            try:
                                assert len(mention_to_add.split()) == len(positions)
                                mentions.append((mention_to_add,type_tag,positions))
                            except AssertionError:
                                print(word_tag_tuples)
                                print(mention_to_add,type_tag,positions)
                                return
                    if((candidate.strip() not in string.punctuation)&(emoji.get_emoji_regexp().sub(u'', candidate).strip(string.punctuation).lower().strip()!='')&(candidate.strip().strip(string.punctuation) not in self.apostrophe_list)):
                        candidateMention=candidate
                        positions=[index]
                else:
                    if((candidate.strip() not in string.punctuation)&(emoji.get_emoji_regexp().sub(u'', candidate).strip(string.punctuation).lower().strip()!='')&(candidate.strip().strip(string.punctuation) not in self.apostrophe_list)):
                        candidateMention+=" "+candidate
                        positions.append(index)
            # if (tag=='B'):
            #     if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))):
            #         mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
            #         if(mention_to_add):
            #             mentions.append(mention_to_add)
            #     candidateMention=candidate
            # else:
            #     candidateMention+=" "+candidate
        if(emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).strip()):
            if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))&(candidate.strip().strip(string.punctuation) not in self.apostrophe_list)):
                mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
                if(mention_to_add!=''):
                    try:
                        assert len(mention_to_add.split()) == len(positions)
                        mentions.append((mention_to_add,type_tag,positions))
                    except AssertionError:
                        print(word_tag_tuples)
                        print(mention_to_add,type_tag,positions)
                        return
            # mentions.append(emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip())
        # print('extracted mentions:', mentions)
        return mentions

    def normalize_to_sentences(self, text):
        tweetSentences=list(filter (lambda sentence: len(sentence)>1, text.split('\n')))
        tweetSentenceList_inter=self.custom_flatten(list(map(lambda sentText: self.my_sentence_tokenizer.tokenize(sentText.lstrip().rstrip()),tweetSentences)),[])
        tweetSentenceList=list(filter (lambda sentence: len(sentence)>1, tweetSentenceList_inter))
        return tweetSentenceList

    def custom_flatten(self, mylist, outlist, ignore_types=(str, bytes, int)):
        
        if (mylist !=[]):
            for item in mylist:
                #print not isinstance(item, ne.NE_candidate)
                if isinstance(item, list) and not isinstance(item, ignore_types):
                    self.custom_flatten(item, outlist)
                else:
                    item=item.strip(' \t\n\r')
                    outlist.append(item)
        return outlist

    def getWords(self, sentence):
        tempList=[]
        tempWordList=sentence.split()
        p_dots= re.compile(r'[.]{2,}')
        #print(tempWordList)
        for word in tempWordList:
            temp=[]

            if "(" in word:
                temp=list(filter(lambda elem: elem!='',word.split("(")))
                if(temp):
                    temp=list(map(lambda elem: '('+elem, temp))
            elif ")" in word:
                temp=list(filter(lambda elem: elem!='',word.split(")")))
                if(temp):
                    temp=list(map(lambda elem: elem+')', temp))
                # temp.append(temp1[-1])
    #         elif (("-" in word)&(not word.endswith("-"))):
    #             temp1=list(filter(lambda elem: elem!='',word.split("-")))
    #             if(temp1):
    #                 temp=list(map(lambda elem: elem+'-', temp1[:-1]))
    #             temp.append(temp1[-1])
            elif (("?" in word)&(not word.endswith("?"))):
                temp1=list(filter(lambda elem: elem!='',word.split("?")))
                if(temp1):
                    temp=list(map(lambda elem: elem+'?', temp1[:-1]))
                temp.append(temp1[-1])
            elif ((":" in word)&(not word.endswith(":"))):
                temp1=list(filter(lambda elem: elem!='',word.split(":")))
                if(temp1):
                    temp=list(map(lambda elem: elem+':', temp1[:-1]))
                temp.append(temp1[-1])
            elif (("," in word)&(not word.endswith(","))):
                #temp=list(filter(lambda elem: elem!='',word.split(",")))
                temp1=list(filter(lambda elem: elem!='',word.split(",")))
                if(temp1):
                    temp=list(map(lambda elem: elem+',', temp1[:-1]))
                temp.append(temp1[-1])
            elif (("/" in word)&(not word.endswith("/"))):
                temp1=list(filter(lambda elem: elem!='',word.split("/")))
                if(temp1):
                    temp=list(map(lambda elem: elem+'/', temp1[:-1]))
                temp.append(temp1[-1])
            elif (list(p_dots.finditer(word))):
                matched_spans= list(p_dots.finditer(word)) 
                temp=[]
                next_string_start=0
                for matched_span in matched_spans:
                    matched_start=matched_span.span()[0]
                    this_excerpt=word[next_string_start:matched_start]
                    if(this_excerpt):
                        temp.append(this_excerpt)
                    next_string_start=matched_span.span()[1]
                if(next_string_start<len(word)):
                    last_excerpt=word[next_string_start:]
                    if(last_excerpt):
                        temp.append(last_excerpt)
            elif "…" in word:
                temp=list(filter(lambda elem: elem!='',word.split("…")))
                if(temp):
                    if(word.endswith("…")):
                        temp=list(map(lambda elem: elem+'…', temp))
                    else:
                        temp=list(map(lambda elem: elem+'…', temp[:-1]))+[temp[-1]]
            else:
                #if word not in string.punctuation:
                temp=[word]
            if(temp):
                tempList.append(temp)
        tweetWordList=self.custom_flatten(tempList,[])
        return tweetWordList

    def rreplace(self,s, old, new, occurrence):
        if s.endswith(old):
            li = s.rsplit(old, occurrence)
            return new.join(li)
        else:
            return s

    def remAmpersand(self,candidateStr):
        candidateStr=candidateStr.replace('&amp;','')
        return candidateStr

    def normalizeToken(self,token):
        lowercased_token = token.lower()
        if token.startswith("@"):
            return "@USER"
        elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
            return "HTTPURL"
        elif len(token) == 1:
            return demojize(token)
        else:
            if token == "’":
                return "'"
            elif token == "…":
                return "..."
            else:
                return token

    def normalizeTweet(self, tweet):
        tokens = self.tweetTokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
        normTweet = " ".join([self.normalizeToken(token) for token in tokens])

        normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
        normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
        normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

        normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
        normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
        normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
        
        return normTweet

    #removing commonly used expletives, enunciated chat words and other common words (like days of the week, common expressions)
    def slang_remove(self,ne_phrase):
        phrase=ne_phrase.strip().strip(string.punctuation).lower()
        p1= re.compile(r'([A-Za-z]+)\1\1{1,}')
        match_lst = p1.findall(phrase)
        if phrase in article_list:
            return True
        elif phrase in day_list:
            return True
        elif phrase in month_list:
            return True
        elif match_lst:
            return True
        else:
            return False

    def all_slang(self, ne_phrase):
        ne_words=ne_phrase.split()
        combined=[]+cachedStopWords+cachedTitles+chat_word_list+day_list
        is_invalid=0

        for word in ne_words:
            if(word.strip().strip(string.punctuation).lower() in combined):
                is_invalid+=1
        if(is_invalid==len(ne_words)):
            return True
        else:
            return False

    def start_end_token_check(self, ne_candidate):
        positions = ne_candidate[2]
        entity_type = ne_candidate[1]
        ne_words=ne_candidate[0].split()
        combined=[]+cachedStopWords+cachedTitles+chat_word_list+day_list+prep_list+article_list
        # print(combined)
        # print(ne_words)
        start_word = ne_words[0].strip().strip(string.punctuation).lower()
        end_word = ne_words[-1].strip().strip(string.punctuation).lower()
        while((len(ne_words)>1)&((start_word in combined)|(end_word in combined))):
            if(start_word in combined):
                ne_words.pop(0)
                positions.pop(0)
            if(end_word in combined):
                ne_words.pop()
                positions.pop()
            if(len(ne_words)>1):
                start_word = ne_words[0].strip().strip(string.punctuation).lower()
                end_word = ne_words[-1].strip().strip(string.punctuation).lower()
        ne_phrase = (' '.join(ne_words)).strip()
        # print(ne_phrase)
        return (ne_phrase,entity_type,positions)

    def set_stopword_exceptions(self,words):
        combined=cachedStopWords+prep_list+article_list+day_list
        for word in words:
            if word in combined:
                self.swSet.add(word)

    def extract(self, batch, batch_number):
        print("Running Local NER now")
        time_in=time.time()
        self.batch=batch
        self.df_out= pd.DataFrame(columns=('tweetID', 'sentID', 'TweetSentence','tweetwordList', 'phase1Candidates','start_time','entry_batch'))
        df_holder=[]
        self.swSet= set()

        if(self.counter==0):
            #self.df_out= pd.DataFrame(columns=('tweetID', 'sentID', 'hashtags', 'user', 'TweetSentence', 'phase1Candidates','correct_candidates_tweet'))
            #dict1 = {'tweetID':0, 'sentID':0, 'hashtags':'first', 'user':'user', 'TweetSentence':'sentence', 'phase1Candidates':'phase1Out','start_time':'now','entry_batch':'batch_number'}
            self.CTrie=trie.Trie("ROOT")
            self.phase2stopWordList=[]
            self.sentenceID = 0
            self.f=0
            self.phaseIpredictions=[]

        for row in self.batch.itertuples():

            now = datetime.datetime.now()
            tweetID=str(row.Index)
            text=str(row.TweetText)
            row_sentences = self.normalize_to_sentences(text)

            annnotated_mentions=[]

            #Comment out when no annotations available
            annnotated_mentions=[]
            annotations = str(row.mentions_other).strip()
            print(tweetID, annotations)
            if(annotations):
                for sentence_level in annotations.split(';'):
                    sentence_level=sentence_level.strip()
                    if(sentence_level):
                        for elem in sentence_level.split(','):
                            elem = elem.strip()
                            if(elem):
                                mention_record = elem.split('|')
                                if(mention_record):
                                    mention, entity_type =  mention_record[0], mention_record[1].lower()
                                    if(mention):
                                        if((mention !='')&(mention !='nan')):
                                            mention = mention.lower().strip(string.punctuation).strip()
                                            if(mention.startswith('the ')):
                                                mention = mention[4:]
                                                mention = mention.lower().strip(string.punctuation).strip()
                                            annnotated_mentions.append((mention, entity_type))

            self.tweet_to_sentences_w_annotation[tweetID]=((self.sentenceID,self.sentenceID+len(row_sentences)),annnotated_mentions)
            self.sentenceID+=len(row_sentences)


            with torch.no_grad():
                for sen_index, sentence in enumerate(row_sentences):

                    # print('tuple index:',tweetID,sen_index)
                    phase1Out=""
                    sentence = self.normalizeTweet(sentence)
                    # tweetWordList=self.getWords(sentence)
                    tweetWordList = sentence.split()
                    tweetWordList = list(filter(lambda element: not element.startswith('http://'), tweetWordList))
                    sentence=(' '.join(tweetWordList)).strip()
                    enumerated_tweetWordList=[(token,idx) for idx,token in enumerate(tweetWordList)]

                    entities_from_sentence=[] #with positions
                    entity_predictions=[]
                    entity_aware_embeddings=[]

                    if(len(tweetWordList)>0):

                        # print(test_record)
                        # tokenized_input=tokenizer(test_record)
                        # initial_input_ids = torch.tensor([tokenizer.encode(test_record)])
                        # token_dict = {x : tokenizer.encode(x, add_special_tokens=False) for x in test_record.split()}
                        # input_ids = initial_input_ids.to(device)
                        # tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

                        tokenized_input= self.nerTokenizer(sentence)
                        initial_input_ids = torch.tensor([self.nerTokenizer.encode(sentence)])
                        # num_tokens = initial_input_ids.shape[1]

                        initial_input_ids = initial_input_ids[:,:128]
                        token_dict = {x : self.nerTokenizer.encode(x, add_special_tokens=False) for x in sentence.split()} #token, add_special_tokens=False, truncation=True
                        input_ids = initial_input_ids.to(self.device)
                        tokens = self.nerTokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

                        output = self.localNEREngine(input_ids)
                        token_embeddings=output.hidden_states[-1].squeeze()[1:-1] # we dont need embeddings for CLS and EOS

                        assert torch.isnan(token_embeddings).any() == False

                        prediction = (torch.argmax(output.logits, axis=2))
                        prediction = prediction.cpu().numpy().reshape(-1)

                        # prediction_labels=[self.label_list[l].split('-')[0] for l in prediction]
                        prediction_labels=[self.label_list[l] for l in prediction]

                        prediction_labels, entity_aware_embeddings=self.collate_token_label_embedding(tweetWordList, token_dict, prediction_labels[1:-1],token_embeddings)

                        assert len(enumerated_tweetWordList)==len(entity_aware_embeddings)
                        assert len(prediction_labels)==len(enumerated_tweetWordList)

                        word_tag_tuples=list(zip(tweetWordList,prediction_labels))
                        print(list(word_tag_tuples))
                        entities_from_sentence=self.get_entities(word_tag_tuples)
                        print('entities_from_sentence:',entities_from_sentence)

                        if(self.f<5):
                            print(len(tweetWordList),initial_input_ids.shape,len(prediction[1:-1]))
                            print(tweetWordList)
                            print(token_dict)
                            print(initial_input_ids)
                            print(input_ids)
                            print(prediction_labels)
                            print('entities_from_sentence:',entities_from_sentence)
                            print('======')
                            self.f+=1

                    just_candidates=[]

                    # place some necessary filters
                    entities_from_sentence= list(filter(lambda element: not self.slang_remove(element[0]), entities_from_sentence))
                    entities_from_sentence= list(map(lambda element: self.start_end_token_check(element), entities_from_sentence))
                    entities_from_sentence= list(filter(lambda element: not self.all_slang(element[0]), entities_from_sentence))
                    entities_from_sentence= list(filter(lambda element: len(element[0])>1, entities_from_sentence))
                    entities_from_sentence= list(filter(lambda element: element[0]!='', entities_from_sentence))

                    

                    for candidateTuple in entities_from_sentence:
                        #self.insert_dict (candidate,self.NE_container,candidateBase,index,candidate.sen_index,batch_number)
                        candidateText, entity_type, positions = candidateTuple

                        entity_predictions.append((candidateText, entity_type))

                        candidateText=(((candidateText.lstrip(string.punctuation)).rstrip(string.punctuation)).strip(' \t\n\r')).lower()
                        candidateText=(self.remAmpersand(candidateText).lstrip('“‘’”')).rstrip('“‘’”')
                        candidateText= self.rreplace(self.rreplace(self.rreplace(candidateText,"'s","",1),"’s","",1),"’s","",1)
                        candidateText= candidateText.strip()
                        self.set_stopword_exceptions(candidateText.split())
                        just_candidates.append(candidateText)
                        # if(index==9423):
                        #     print(candidateText)
                        position = '*'+'*'.join(str(v) for v in positions)
                        position=position+'*'

                        phase1Out+=(((candidateText).lstrip(string.punctuation)).strip())+'::'+str(position)+"::"+entity_type+"||"

                        combined=[]+cachedStopWords+cachedTitles+prep_list+chat_word_list+article_list+day_list
                        if not ((candidateText in combined)|(len(candidateText)<=1)|(candidateText.isdigit())|(self.is_float(candidateText))):
                            if(self.quickRegex.match(candidateText)):
                                self.CTrie.__setitem__(candidateText.split(),len(candidateText.split()),[],batch_number)
                    
                    print('candidate_strings_from_sentence:',just_candidates)
                    self.phaseIpredictions.append(entity_predictions)
                    #storing the outputs and token embeddings in dataframe tweetID,sen_index
                    dict1 = {'tweetID':str(tweetID), 'sentID':str(sen_index), 'TweetSentence':sentence, 'tweetwordList': enumerated_tweetWordList,'phase1Candidates': just_candidates ,'phase1CandidatesWPositions':phase1Out,
                             'contextual_embeddings':entity_aware_embeddings,
                             'start_time':now,'entry_batch':batch_number}
                    df_holder.append(dict1)
                    # self.contextual_embeddings[(tweetID,sen_index)] = entity_aware_embeddings

        time_out=time.time()
        self.append_rows(df_holder)
        self.phase2stopWordList=list(set(self.phase2stopWordList)|self.swSet)
        self.counter=self.counter+1
        
        #return (copy.deepcopy(self.df_out),copy.deepcopy(freqs),time_in,time_out)
        return (self.df_out,self.contextual_embeddings,self.CTrie,time_in,time_out,self.phase2stopWordList,self.tweet_to_sentences_w_annotation)

    def append_rows(self,df_holder):
    
        df = pd.DataFrame(df_holder)
        self.df_out=self.df_out.append(df)

## **Phase II: Global NER with Phrase Embedder to collect the Entity Embeddings**

In [ ]:
# coding: utf-8
from nltk.corpus import stopwords
import pandas  as pd
# import NE_candidate_module as ne
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import copy
import numpy
import math
from itertools import groupby
from operator import itemgetter
from collections import Iterable, OrderedDict
from scipy import stats
import emoji
import statistics
import pandas as pd
import time
import datetime
import trie as trie
import re
import ast
import pickle
import itertools
from scipy import spatial

# from sklearn.preprocessing import PolynomialFeatures
# from sklearn import linear_model
# from sklearn.cluster import KMeans, MeanShift
# from sklearn.metrics import silhouette_samples, silhouette_score

# Clustering imports
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

import torch

#CLASSIFIER CLASS
# import SVM as svm
# import entityClassifier as entityClassifier

cachedStopWords = stopwords.words("english")
tempList=["i","and","or","since","hence","onto","other","another","across","unlike","anytime","were","you","then","still","till","nor","perhaps","probably","otherwise","until","sometimes","sometime","seem","cannot","seems","because","can","like","into","able","unable","either","neither","if","we","it","else","elsewhere","how","not","what","who","when","where","who's","who’s","let","today","tomorrow","tonight","let's","let’s","lets","know","make","oh","via","i","yet","must","mustnt","mustn't","mustn’t","i'll","i’ll","you'll","you’ll","we'll","we’ll","done","doesnt","doesn't","doesn’t","dont","don't","don’t","did","didnt","didn't","didn’t","much","without","could","couldn't","couldn’t","would","wouldn't","wouldn’t","should","shouldn't","souldn’t","shall","isn't","isn’t","hasn't","hasn’t","wasn't","wasn’t","also","let's","let’s","let","well","just","everyone","anyone","noone","none","someone","theres","there's","there’s","everybody","nobody","somebody","anything","else","elsewhere","something","nothing","everything","i'd","i’d","i’m","won't","won’t","i’ve","i've","they're","they’re","we’re","we're","we'll","we’ll","we’ve","we've","they’ve","they've","they’d","they'd","they’ll","they'll","again","you're","you’re","you've","you’ve","thats","that's",'that’s','here’s',"here's","what's","what’s","i’m","i'm","a","so","except","arn't","aren't","arent","this","when","it","it’s","it's","he's","she's","she'd","he'd","he'll","she'll","she’ll","many","can't","cant","can’t","even","yes","no","these","here","there","to","maybe","<hashtag>","<hashtag>.","ever","every","never","there's","there’s","whenever","wherever","however","whatever","always","although"]
for item in tempList:
    if item not in cachedStopWords:
        cachedStopWords.append(item)
cachedStopWords.remove("don")
# cachedStopWords.remove("your")
# cachedStopWords.remove("us")
cachedTitles = ["mr.","mr","mrs.","mrs","miss","ms","sen.","dr","dr.","prof.","president","congressman"]
prep_list=["of","&;","v."] #includes common conjunction as well
# prep_list=[]
# article_list=[]
article_list=["a","an","the"]
conjoiner=["de"]
day_list=["sunday","monday","tuesday","wednesday","thursday","friday","saturday","mon","tues","wed","thurs","fri","sat","sun"]
month_list=["january","february","march","april","may","june","july","august","september","october","november","december","jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
chat_word_list=["nope","httpurl","n't","&amp;","gee","€","hmm","bye","pls","please","yrs","4get","cbb","tmi","ooh","ouch","am","tv","ima","tmw","og","psst","b.s","thanku","em","qft","ip","icymi","bdsm","ah","http","https","pm","omw","pts","pt","ive","reppin","idk","oops","yup","stfu","uhh","2b","dear","yay","btw","ahhh","b4","ugh","ty","cuz","coz","sorry","yea","asap","ur","bs","rt","lmfao","lfmao","slfmao","u","r","nah","umm","ummm","thank","thanks","congrats","whoa","rofl","ha","ok","okay","hey","hi","huh","ya","yep","yeah","fyi","duh","damn","lol","omg","congratulations","fucking","fuck","f*ck","wtf","wth","aka","wtaf","xoxo","rofl","imo","wow","fck","haha","hehe","hoho"]
string.punctuation=string.punctuation+'…‘’'
punct = string.punctuation
punct=punct.replace(".","",1)
# print(punct)



class GlobalNERModule():


    def executor(self,max_batch_value,TweetBase,CTrie,phase2stopwordList,z_score_threshold,reintroduction_threshold,raw_tweets_for_others):


        # SET CB
        # print(phase2stopwordList)
        candidate_featureBase_DF,data_frame_holder,phase2_candidates_holder,phase2_unnormalized_candidates_holder = self.set_cb(TweetBase,CTrie,phase2stopwordList,z_score_threshold,reintroduction_threshold)
        candidate_df = candidate_featureBase_DF[['candidate','class']]
        for index, row in candidate_df.iterrows():
            print(row['candidate']+' : '+str(row['class']))
        candidate_df.to_csv("candidate_records.csv", sep=',', encoding='utf-8',index=False)


        # # SET TF 
        ner_arrays = self.set_tf(data_frame_holder, candidate_featureBase_DF,phase2_candidates_holder,phase2_unnormalized_candidates_holder)

        # self.calculate_tp_fp_f1(z_score_threshold,untrashed_tweets,raw_tweets_for_others)

        # # return candidate_featureBase_DF, self.complete_tweet_dataframe_grouped_df_sorted,time_out 
        return ner_arrays


    def __init__(self,entity_phrase_embedder,device,filename):
        self.counter=0
        self.decay_factor=2**(-1/2)
        self.decay_base_staggering=2
        self.true_positive_count=0
        self.false_positive_count=0
        self.false_negative_count=0
        self.device=device
        self.save_file= filename

        self.entity_phrase_embedder = entity_phrase_embedder

        # context_feature_list=['cf_'+str(i) for i in range(768)]
        context_feature_list=['cf_'+str(i) for i in range(300)]
        self.candidateBaseHeaders=['candidate', 'batch', 'length','cap','substring-cap','s-o-sCap','all-cap','non-cap','non-discriminative']+context_feature_list+['cumulative']
        self.candidateBaseHeaders_alt=['candidate', 'batch', 'length','cumulative','class']
        self.complete_tweet_dataframe_grouped_df_sorted=pd.DataFrame([], columns=['tweetID', 'TweetSentence', 'ambiguous_candidates', 'annotation', 'candidates_with_label', 'completeness', 'current_minus_entry', 'entry_batch', 'hashtags', 'index', 'only_good_candidates', 'output_mentions', 'phase1Candidates', 'sentID', 'stanford_candidates', 'user'])


    def classify_candidate_base(self,z_score_threshold,candidate_featureBase_DF):

        # zscore_array1=stats.zscore(candidate_featureBase_DF['cumulative'])

        # candidate_featureBase_DF['Z_ScoreUnweighted']=zscore_array1
        # cumulative_threshold=1.0 #set threshold here
        # z_score_threshold=candidate_featureBase_DF[candidate_featureBase_DF['cumulative']==cumulative_threshold].Z_ScoreUnweighted.tolist()[0]
        # print(cumulative_threshold,z_score_threshold)
        # #candidate_featureBase_DF.to_csv("cf_new_with_z_score.csv", sep=',', encoding='utf-8')

        # #multi-word infrequent candidates ---> to be used for recall correction
        # infrequent_candidates=candidate_featureBase_DF[(candidate_featureBase_DF['Z_ScoreUnweighted'] < z_score_threshold) & (candidate_featureBase_DF.length>1)].candidate.tolist()
        # candidate_featureBase_DF = candidate_featureBase_DF[candidate_featureBase_DF['Z_ScoreUnweighted'] >= z_score_threshold]
        return self.entity_classifier_alt.run(candidate_featureBase_DF,self.candidateEmbeddingDict)

    # recall_correction
    def set_partition_dict(self,candidate_featureBase_DF,infrequent_candidates):

        #print(list(self.partition_dict.keys()))
        ambiguous_bad_candidates=candidate_featureBase_DF[(((candidate_featureBase_DF.status=="a")|(candidate_featureBase_DF.status=="b"))&(candidate_featureBase_DF.length.astype(int)>1))]
        good_candidates=candidate_featureBase_DF[(candidate_featureBase_DF.status=="g")].candidate.tolist()
        flag1=False
        flag2=False
        if(len(ambiguous_bad_candidates)>0):
            # ambiguous_bad_candidates['max_column'] =ambiguous_bad_candidates[['cap','substring-cap','s-o-sCap','all-cap','non-cap','non-discriminative']].idxmax(axis=1) 
            # ambiguous_bad_candidates_wFilter= ambiguous_bad_candidates[ambiguous_bad_candidates.max_column=='substring-cap']

            #good_candidates=candidate_featureBase_DF[(candidate_featureBase_DF.status=="g")].candidate.tolist()
            #print(ambiguous_bad_candidates_wFilter.candidate.tolist())

            # for candidate in ambiguous_bad_candidates_wFilter.candidate.tolist():
            for candidate in ambiguous_bad_candidates.candidate.tolist():
                
                #print(candidate)
                if candidate not in self.partition_dict.keys():

                    substring_candidates=self.get_substring_candidates(candidate.split(),good_candidates)
                    if(len(substring_candidates)>0):
                        self.partition_dict[candidate]=substring_candidates

            flag1= True
        if(len(infrequent_candidates)>0):
            #print(len(ambiguous_bad_candidates_wFilter.candidate.tolist()))

            for candidate in infrequent_candidates:
                #print(candidate)
                if candidate not in self.partition_dict.keys():
                    substring_candidates=self.get_substring_candidates(candidate.split(),good_candidates)
                    if(len(substring_candidates)>0):
                        self.partition_dict[candidate]=substring_candidates
            flag2= True
        print(list(self.partition_dict.keys()))
        return (flag1|flag2)

    def update_Candidatedict(self,candidate,cluster_id,contextual_embedding_vector):
        
        alt_feature_list=[] # this is for the reduced CandidateBase for the Alt_Classifier
        candidate_name = candidate+'||'+cluster_id

        if(candidate_name in self.CandidateBase_dict_alt.keys()):
            # feature_list=self.CandidateBase_dict[normalized_candidate]
            alt_feature_list=self.CandidateBase_dict_alt[candidate_name]
        else:
            alt_feature_list=[0]*4
            alt_feature_list[0]=self.counter
            alt_feature_list[1]=len(candidate.split())
            alt_feature_list[2]=0
            alt_feature_list[3]=-1

        alt_feature_list[2]+=1 #increase cumulative frequency
        self.CandidateBase_dict_alt[candidate_name]=alt_feature_list

        if((candidate,cluster_id) in self.candidateEmbeddingDict.keys()):
            self.candidateEmbeddingDict[(candidate,cluster_id)].append(contextual_embedding_vector)
        else:
            self.candidateEmbeddingDict[(candidate,cluster_id)]=[contextual_embedding_vector]
        return

    def get_candidate_clusters(self, candidate, input_arr):
        if(len(input_arr)>1):
            print(candidate)
            # X = np.nan_to_num(np.array(input_arr))
            X = np.array(input_arr)
            # print(X)
            # clusterer = AgglomerativeClustering(n_clusters=5, affinity="cosine", linkage="single") #total 5 entity type clusters can be formed
            # cluster_labels = clusterer.fit_predict(X)

            # clusterer = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='complete', compute_full_tree=True, distance_threshold=1.0)
            clusterer = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', compute_full_tree=True, distance_threshold=0.999)
            # clusterer = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='single', compute_full_tree=True, distance_threshold=1.0)

            # clusterer = AgglomerativeClustering(n_clusters=None, affinity='euclidean', linkage='ward', compute_full_tree=True, distance_threshold=1.0)

            cluster_labels = clusterer.fit_predict(X)
            # print("Number of clusters for "+candidate+" : "+str(1+np.amax(cluster_labels)))

            # range_n_clusters = [1, 2, 3, 4, 5]
            # max_silhouette = float('-inf')
            # for n_clusters in range_n_clusters:
            #     if(n_clusters <= len(X)):
            #         clusterer = KMeans(n_clusters=n_clusters, random_state=42)
            #         labels = clusterer.fit_predict(X)
            #         silhouette = silhouette_score(X, cluster_labels)
            #         if(silhouette>max_silhouette):
            #             max_silhouette = silhouette
            #             cluster_labels = labels
        else:
            cluster_labels = np.array([0])

        # print(cluster_labels)
        return cluster_labels
    
    def set_cb(self,TweetBase,CTrie,phase2stopwordList,z_score_threshold,reintroduction_threshold):

        #input new_tweets, z_score, Updated candidatebase of phase1
        #output candidate_featureBase_DF, Incomplete_tweets
        data_frame_holder=pd.DataFrame([], columns=['index','entry_batch','tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
        phase2_candidates_holder=[]
        phase2_unnormalized_candidates_holder=[]
        df_holder=[]

        df_holder_extracted,phase2_candidates_holder_extracted,phase2_unnormalized_candidates_holder_extracted= self.extract(TweetBase,CTrie,phase2stopwordList,0)
        phase2_candidates_holder.extend(phase2_candidates_holder_extracted)
        phase2_unnormalized_candidates_holder.extend(phase2_unnormalized_candidates_holder_extracted)
        df_holder.extend(df_holder_extracted)

        data_frame_holder = pd.DataFrame(df_holder)

        #perform clustering over mentions of each candidate in self.candidateEmbeddingPool
        for candidate in self.candidateEmbeddingPool.keys():
            labels = self.get_candidate_clusters(candidate, self.candidateEmbeddingPool[candidate])
            for ind,int_label in enumerate(labels):
                # print(self.candidateEmbedding_records[candidate][ind])
                label = str(int_label)
                mention_embedding_vector = self.candidateEmbedding_records[candidate][ind][0]
                assert self.candidateEmbeddingPool[candidate][ind] == mention_embedding_vector.tolist()
                self.candidateEmbedding_records[candidate][ind] += (label,)
                # print(self.candidateEmbedding_records[candidate][ind])
                self.update_Candidatedict(candidate,label,mention_embedding_vector)

        # #self.CandidateBase_dict_alt and self.candidateEmbeddingDict are formed after candidate clusters have been separated
        candidate_featureBase_DF_alt = pd.DataFrame.from_dict(self.CandidateBase_dict_alt, orient='index')

        candidate_featureBase_DF_alt.columns = self.candidateBaseHeaders_alt[1:]
        candidate_featureBase_DF_alt.index.name = self.candidateBaseHeaders_alt[0]
        candidate_featureBase_DF_alt  = candidate_featureBase_DF_alt.reset_index(drop=False)

        # print(candidate_featureBase_DF_alt.head(5))
        # for elem in list(self.candidateEmbeddingDict.keys())[:5]:
        #     print(elem, len(self.candidateEmbeddingDict[elem]))

        #initialize and classify
        self.entity_classifier_alt = EntityClassifierAlt(False,self.device,None,None)
        candidate_featureBase_DF= self.classify_candidate_base(z_score_threshold,candidate_featureBase_DF_alt)

        return candidate_featureBase_DF,data_frame_holder,phase2_candidates_holder,phase2_unnormalized_candidates_holder

    def set_tf(self,data_frame_holder, candidate_featureBase_DF, phase2_candidates_holder,phase2_unnormalized_candidates_holder):
        entity_mentions_holder = []
        for phase2_mentions in phase2_candidates_holder:
            row_entity_mentions=[]
            for mention in phase2_mentions:
                # print(mention)
                mention_record = mention.split('||')
                candidate, mention_id = mention_record[0], int(mention_record[1])
                # print(self.candidateEmbedding_records[candidate][mention_id])
                cluster_id = self.candidateEmbedding_records[candidate][mention_id][1]
                class_value = candidate_featureBase_DF[candidate_featureBase_DF['candidate'] == candidate+'||'+cluster_id]['class'].tolist()[0]
                class_label = self.entity_classifier_alt.entity_types[class_value]
                if(class_label!='ne'):
                    row_entity_mentions.append((candidate,class_label))
            entity_mentions_holder.append(row_entity_mentions)
        data_frame_holder["only_good_candidates"]=entity_mentions_holder
        return entity_mentions_holder

    def get_incomplete_tf(self,untrashed_tweets):
        return untrashed_tweets[untrashed_tweets.completeness==False]

    def get_complete_tf(self,untrashed_tweets):
        return untrashed_tweets[untrashed_tweets.completeness==True]

    def compute_seen_tweets_so_far(self,start_batch,end_batch):
        if(start_batch==end_batch):
            sliced_seen_tweets=self.number_of_seen_tweets_per_batch[start_batch]


        sliced_seen_tweets=self.number_of_seen_tweets_per_batch[start_batch:]


        counter=0
        for elem in sliced_seen_tweets:
            counter=counter+elem

        return counter


    def rreplace(self,s, old, new, occurrence):
        if s.endswith(old):
            li = s.rsplit(old, occurrence)
            return new.join(li)
        else:
            return s
    #ME_EXTR=Mention.Mention_Extraction()



    def set_column_for_candidates_in_incomplete_tweets(self,candidate_featureBase_DF,input_to_eval):

        incomplete_candidates= input_to_eval['2nd Iteration Candidates'].tolist()

        candidate_featureBase_DF= candidate_featureBase_DF.set_index('candidate')

        candidate_with_label_holder=[]
        one_level=[]
        

        for sentence_level_candidates in incomplete_candidates:

            one_level.clear()

            for candidate in sentence_level_candidates:
                if candidate.lower() in candidate_featureBase_DF.index:
                    # label=candidate_featureBase_DF.get_value(candidate.lower(),'status')
                    label=candidate_featureBase_DF.at[candidate.lower(),'status']
                    one_level.append((candidate,label))
                else:
                    one_level.append((candidate,"na"))

            candidate_with_label_holder.append(copy.deepcopy(one_level))


        input_to_eval["candidates_with_label"]=candidate_with_label_holder
        debug_candidates_label_list= input_to_eval['candidates_with_label'].tolist()
        candidates_filtered_g_labeled=[]
        row_level_candidates=[]
        index_outer=0

        candidates_filtered_a_labeled=[]
        row_level_a_candidates=[]

        for sentence_level in debug_candidates_label_list:

            # sentence_level_candidates_unnormalized= incomplete_candidates_unnormalized[index_outer]
            row_level_candidates.clear()
            row_level_a_candidates.clear()
            for candidate in sentence_level:
                if(candidate[1]=="g"):
                    candidate_str = self.erode_article(candidate[0])
                    row_level_candidates.append(candidate_str)
                if(((candidate[1]=="b")|(candidate[1]=="a"))&(candidate[0]=="US")):
                    # print('here')
                    row_level_candidates.append(candidate[0])
                if(candidate[1]=="a"):
                    row_level_a_candidates.append(candidate[0])

            candidates_filtered_g_labeled.append(copy.deepcopy(row_level_candidates))
            candidates_filtered_a_labeled.append(copy.deepcopy(row_level_a_candidates))
            index_outer+=1


        input_to_eval["only_good_candidates"]=candidates_filtered_g_labeled
        input_to_eval["ambiguous_candidates"]=candidates_filtered_a_labeled



    def calculate_tp_fp_f1(self,z_score_threshold,input_to_eval,raw_tweets_for_others):

        input_to_eval_grouped_df= (input_to_eval.groupby('tweetID', as_index=False).aggregate(lambda x: x.tolist()))
        input_to_eval_grouped_df['tweetID']=input_to_eval_grouped_df['tweetID'].astype(int)
        input_to_eval_df_sorted=(input_to_eval_grouped_df.sort_values(by='tweetID', ascending=True)).reset_index(drop=True)

        print(len(input_to_eval_df_sorted),len(raw_tweets_for_others))
        
        print(set(input_to_eval_df_sorted['tweetID'].values.tolist())-set(raw_tweets_for_others['ID'].values.tolist()))

        input_to_eval_df_sorted['annotation']=input_to_eval_df_sorted['tweetID'].apply(lambda x: raw_tweets_for_others[raw_tweets_for_others['ID']==x]['mentions_other'].iloc[0])
        # input_to_eval_df_sorted['annotation']=input_to_eval_df_sorted['tweetID'].apply(lambda x: raw_tweets_for_others[raw_tweets_for_others['ID']==x]['mentions_limited_types'].iloc[0])
        # input_to_eval_df_sorted['annotation']=input_to_eval_df_sorted['tweetID'].apply(lambda x: raw_tweets_for_others[raw_tweets_for_others['ID']==x]['annotation_limited types'].iloc[0])

        column_candidates_holder = input_to_eval_df_sorted['only_good_candidates'].tolist()
        # column_candidates_holder = input_to_eval_df_sorted['phase1Candidates'].tolist()

        file1 = open(self.save_file+".txt", "w")
        

        column_annot_holder= input_to_eval_df_sorted['annotation'].tolist()
        
        output_str=''
        # print(column_candidates_holder)

        true_positive_count=0
        false_positive_count=0
        false_negative_count=0
        ambigious_not_in_annotation=0
        total_mentions=0
        total_annotation=0

        all_annotations=[]
        all_mentions=[]

        true_positive_holder = []
        false_negative_holder=[]
        false_positive_holder=[]
        total_mention_holder=[]
        ambigious_not_in_annotation_holder=[]
        f_measure_holder=[]

        quickRegex=re.compile("[a-z]+")

        print('=========================BERTNER_candidates')

        for idx in range(len(column_annot_holder)):
            unrecovered_annotated_mention_list=[]
            tp_counter_inner=0
            fp_counter_inner=0
            fn_counter_inner=0

            annotated_mention_list=[]
            output_mentions_list=[]
            tweet_level_candidate_list=str(column_annot_holder[idx]).split(';')
            for tweet_level_candidates in tweet_level_candidate_list:
                sentence_level_cand_list= tweet_level_candidates.split(',')
                annotated_mention_list.extend(sentence_level_cand_list)
            annotated_mention_list=list(map(lambda element: element.lower().strip(),annotated_mention_list))
            annotated_mention_list=list(filter(lambda element: quickRegex.match(element), annotated_mention_list))
            annotated_mention_list=list(filter(lambda element: ((element !='')&(element !='nan')), annotated_mention_list))

            for lst in column_candidates_holder[idx]:
                output_mentions_list.extend(lst)

            output_mentions_list=list(filter(lambda element: (element !=''), output_mentions_list))
            output_mentions_list=list(map(lambda element: element.lower(), output_mentions_list))
            total_annotation+=len(annotated_mention_list)

            print(idx, annotated_mention_list,output_mentions_list)
            output_str+=','.join(output_mentions_list)+'\n'

            all_annotations.extend(annotated_mention_list)
            all_mentions.extend(output_mentions_list)

            total_mentions+=len(output_mentions_list)
            all_postitive_counter_inner=len(output_mentions_list)

            while(annotated_mention_list):
                if(len(output_mentions_list)):
                    annotated_candidate= self.normalize(annotated_mention_list.pop())
                    if(annotated_candidate in output_mentions_list):
                        output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                        tp_counter_inner+=1
                    else:
                        unrecovered_annotated_mention_list.append(annotated_candidate)
                else:
                    unrecovered_annotated_mention_list.extend(annotated_mention_list)
                    break

            # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
            fn_counter_inner=len(unrecovered_annotated_mention_list)
            fp_counter_inner=all_postitive_counter_inner- tp_counter_inner

            # print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

            self.true_positive_count+=tp_counter_inner
            self.false_positive_count+=fp_counter_inner
            self.false_negative_count+=fn_counter_inner

        print('TP||||FP||||FN')
        print(self.true_positive_count,self.false_positive_count,self.false_negative_count,total_mentions,total_annotation)

        precision=(self.true_positive_count)/(self.true_positive_count+self.false_positive_count)
        recall=(self.true_positive_count)/(self.true_positive_count+self.false_negative_count)
        f_measure=2*(precision*recall)/(precision+recall)

        # all_annotations=set(all_annotations)
        # all_mentions=set(all_mentions)
        
        # true_positive_count= len(all_annotations.intersection(all_mentions))
        # false_positive_count=len(all_mentions-all_annotations)
        # false_negative_count=len(all_annotations-all_mentions)
        # total_mentions=len(all_mentions)
        # total_annotation=len(all_annotations)


        # print(true_positive_count,false_positive_count,false_negative_count,total_mentions,total_annotation)

        # precision=(true_positive_count)/(true_positive_count+false_positive_count)
        # recall=(true_positive_count)/(true_positive_count+false_negative_count)
        # f_measure=2*(precision*recall)/(precision+recall)

        file1.write(output_str)
        file1.close()



        self.accuracy_vals=(z_score_threshold,f_measure,precision,recall)

        print('Precision:',precision)
        print('Recall:',recall)
        print('F1:',f_measure)

        # print('z_score:', z_score_threshold , 'precision: ',precision,'recall: ',recall,'f measure: ',f_measure)
        # print('trupe positive: ',tp_count, 'false positive: ',fp_count,'false negative: ', fn_count,'total mentions: ', tm_count)

        return input_to_eval


    def recall_correction(self,candidate_featureBase_DF,phase2_candidates_holder,phase2_unnormalized_candidates_holder,data_frame_holder):

        corrected_phase2_candidates_holder=[]
        index_outer=0
        for candidates in phase2_candidates_holder:
            unnormalized_candidates=phase2_unnormalized_candidates_holder[index_outer]
            corrected_phase2_candidates=[]
            for idx, candidate in enumerate(candidates):
                unnormalized_candidate=unnormalized_candidates[idx]
                # if((candidate in self.partition_dict.keys())&((candidate in self.infrequent_candidates)|(candidate in self.bad_candidates))):
                if((candidate in self.partition_dict.keys())&((candidate in self.infrequent_candidates)|(candidate in self.bad_candidates)|(candidate in self.ambiguous_candidates))):   #do this only for 3K tweets
                    #print(candidate, self.partition_dict[candidate])
                    corrected_phase2_candidates.extend(self.partition_dict[candidate])
                else:
                    if(((candidate in self.bad_candidates)|(candidate in self.ambiguous_candidates))&(candidate=='us')&(unnormalized_candidate=='US')):
                        # print(index_outer)
                        candidate=unnormalized_candidate
                    # if((len(candidate.strip().strip(string.punctuation).split())>1)&(candidate.strip().strip(string.punctuation).split()[0].lower() in ['a','an','the'])):
                    #     candidate = (' '.join(candidate.strip().strip(string.punctuation).split()[1:])).strip()
                    corrected_phase2_candidates.append(candidate)
            corrected_phase2_candidates_holder.append(copy.deepcopy(corrected_phase2_candidates))
            index_outer+=1

        
        #print(corrected_phase2_candidates_holder)
        data_frame_holder['2nd Iteration Candidates']=corrected_phase2_candidates_holder

        return corrected_phase2_candidates_holder,data_frame_holder                  


    def erode_article(self, entity_string):
        if((len(entity_string.strip().strip(string.punctuation).split())>1)&(entity_string.lower().strip().strip(string.punctuation).split()[0] in ['a','an','the'])):
            # print(entity_string)
            entity_string = ' '.join(entity_string.strip().strip(string.punctuation).split()[1:])
            # print(entity_string)
        return entity_string.strip()


    def set_completeness_in_tweet_frame(self,data_frame_holder,candidate_featureBase_DF,phase2_candidates_holder,phase2_unnormalized_candidates_holder,correction_flag):
        #print(candidate_featureBase_DF.head())
        good_candidates=candidate_featureBase_DF[candidate_featureBase_DF.status=="g"].candidate.tolist()
        bad_candidates=candidate_featureBase_DF[candidate_featureBase_DF.status=="b"].candidate.tolist()

        merged_g_b= bad_candidates+good_candidates

        #candidate_featureBase_DF.to_csv("cf_before_labeling_comp.csv", sep=',', encoding='utf-8')
        ambiguous_candidates=candidate_featureBase_DF[candidate_featureBase_DF.status=="a"].candidate.tolist()

        if(correction_flag):
            phase2_candidates_holder,data_frame_holder=self.recall_correction(candidate_featureBase_DF,phase2_candidates_holder,phase2_unnormalized_candidates_holder,data_frame_holder)

         

        
        truth_vals=[False if any(x not in merged_g_b for x in list1) else True for list1 in phase2_candidates_holder]

        intermediate_output_mentions=[list(filter(lambda candidate: ((candidate in good_candidates))|(candidate=='US'), list1)) for list1 in phase2_candidates_holder]

        output_mentions=[list(map(lambda candidate: self.erode_article(candidate), list1)) for list1 in intermediate_output_mentions]

        # truth_vals=[False if any(x in ambiguous_candidates for x in list1) else True for list1 in phase2_candidates_holder]
        completeness_series = pd.Series( (v for v in truth_vals) )
        output_mentions_series = pd.Series( (v for v in output_mentions) )


        data_frame_holder['output_mentions']=output_mentions_series
        data_frame_holder['completeness']=completeness_series
        data_frame_holder["current_minus_entry"]=self.counter-data_frame_holder['entry_batch']

        return data_frame_holder



    def normalize(self,word):
        strip_op=word
        strip_op=(((strip_op.lstrip(string.punctuation)).rstrip(string.punctuation)).strip()).lower()
        strip_op=(strip_op.lstrip('“‘’”')).rstrip('“‘’”')
        strip_op=(((strip_op.lstrip(string.punctuation)).rstrip(string.punctuation)).strip()).lower()
        #strip_op= self.rreplace(self.rreplace(self.rreplace(strip_op,"'s","",1),"’s","",1),"’s","",1)
        if strip_op.endswith("'s"):
            li = strip_op.rsplit("'s", 1)
            return ''.join(li)
        elif strip_op.endswith("’s"):
            li = strip_op.rsplit("’s", 1)
            return ''.join(li)
        else:
            return strip_op
        #return strip_op

    
    def isSubstring(self,to_increase_element,id_to_incr,comparison_holder,phase1_holder_holder_copy):
        combined_list=comparison_holder[id_to_incr]+phase1_holder_holder_copy[id_to_incr]

        for idx,val in enumerate(comparison_holder[id_to_incr]):
            if((to_increase_element[0] in val[0]) and to_increase_element[0] != val[0]):
                if((to_increase_element[5] in val[5]) and to_increase_element[5] != val[5]):
                    return True
        for idx,val in enumerate(phase1_holder_holder_copy[id_to_incr]):
            if((to_increase_element[0] in val[0]) and to_increase_element[0] != val[0]):
                if((to_increase_element[5] in val[2]) and to_increase_element[5] != val[2]):
                    return True   
                
        return False


    def calculate_pmi(self,big,x1,x2,total):
        big__= float(big/total)
        x1__=float(x1/total)
        x2__=float(x2/total)
        pmi= math.log(big__/(x1__*x2__),2.71828182845)
        pklv=big__*pmi
        #return (1/(1+math.exp(-1*pmi)))
        npmi= pmi/(-1.0*(math.log(big__,2.71828182845)))
        return npmi,pklv
        #return pklv

    def multiSlice(self,s,cutpoints,good_candidates):
        k = len(cutpoints)
        multislices=[]
        if k == 0:
            curr_candidate=self.normalize(' '.join(s))

            if(curr_candidate in good_candidates):
                multislices = [curr_candidate]        
        else:
            
            curr_candidate=self.normalize(' '.join(s[:cutpoints[0]]))
            alt_list=[curr_candidate]
            
            if(curr_candidate in good_candidates):
                multislices = [curr_candidate]

            alt_list.extend(self.normalize(' '.join(s[cutpoints[i]:cutpoints[i+1]])) for i in range(k-1))
            multislices.extend(self.normalize(' '.join(s[cutpoints[i]:cutpoints[i+1]])) for i in range(k-1) if self.normalize(' '.join(s[cutpoints[i]:cutpoints[i+1]])) in good_candidates)

            curr_candidate=self.normalize(' '.join(s[cutpoints[k-1]:]))
            alt_list.append(curr_candidate)
            
            if(curr_candidate in good_candidates):
                multislices.append(curr_candidate)
            # print('::',alt_list)
        return multislices



    def get_substring_candidates(self,candidate_words,good_candidates):
        n = len(candidate_words)
        all_partitions=[]
        all_partitions_length=[]
        cuts = list(range(1,n))
        for k in range(n):
            # all_partitions_inner=[]
            partition_list=[]
            partition_length_list=[]
            for cutpoints in itertools.combinations(cuts,k):
                ret_list=self.multiSlice(candidate_words,cutpoints,good_candidates)
                if(ret_list):
                    partition_length=sum([len(elem.split()) for elem in ret_list])
                    # print('==',ret_list,partition_length)
                    if(partition_length==len(candidate_words)):
                        return ret_list
                    partition_list.append(ret_list)
                    partition_length_list.append(partition_length)
                    # yield ret_list
            # print('------')
            if(partition_length_list):
                max_index=partition_length_list.index(max(partition_length_list))
                all_partitions.append(partition_list[max_index])
                all_partitions_length.append(partition_length_list[max_index])
        # print(all_partitions)
        if(all_partitions_length):
            max_index=all_partitions_length.index(max(all_partitions_length))
            # print(all_partitions[max_index])
            return all_partitions[max_index]
        else:
            return []
    

    def verify(self, subsequence, CTrie):
        return CTrie.__contains__(subsequence)



    def check_sequence(self, sequence, l, CTrie):
        result=[]
        subsequence_length=l
        while(subsequence_length>0):
            shift=len(sequence)-subsequence_length
            verified_subsequence=[]
            verified=False
            for i in range(0,shift+1):
                list1=sequence[i:(i+subsequence_length)]
                text=' '.join(str(e[0]) for e in list1)
                subsequence=(self.normalize(text)).split()
                #print("search for", subsequence)
                if self.verify(subsequence, CTrie):
                    verified_subsequence.append(i)
                    verified_subsequence.append(i+subsequence_length)
                    #print(subsequence)
                    #print(subsequence,[(verified_subsequence[0]-0),(int(sequence[-1][1])-verified_subsequence[1])])
                    verified=True
                    break
            if(verified):
                result.append(sequence[verified_subsequence[0]:verified_subsequence[1]])
                if(verified_subsequence[0]-0)>0:
                    subequence_to_check=sequence[0:verified_subsequence[0]]
                    #since tokens before the starting position of the verified subsequence have already been checked for subsequences of this length
                    partition_length=min(len(subequence_to_check),(subsequence_length-1))
                    #print(subequence_to_check)
                    lst=self.check_sequence(subequence_to_check,partition_length, CTrie)
                    if(lst):
                        result.extend(lst)
                if(int(sequence[-1][1])-verified_subsequence[1])>0:
                    subequence_to_check=sequence[(verified_subsequence[1]):]
                    #since tokens following the end position of the verified subsequence have not been checked for subsequences of this length
                    partition_length=min(len(subequence_to_check),(subsequence_length))
                    #print(subequence_to_check)
                    lst=self.check_sequence(subequence_to_check,partition_length, CTrie)
                    if(lst):
                        result.extend(lst)
                return result
            else:
                subsequence_length-=1
        return result

    # def flatten(self,mylist, outlist,ignore_types=(str, bytes, int, ne.NE_candidate)):
    def flatten(self,mylist, outlist,ignore_types=(str, bytes, int)):
    
        if mylist !=[]:
            for item in mylist:
                #print not isinstance(item, ne.NE_candidate)
                if isinstance(item, list) and not isinstance(item, ignore_types):
                    self.flatten(item, outlist)
                else:
                    # if isinstance(item,ne.NE_candidate):
                    #     item.phraseText=item.phraseText.strip(' \t\n\r')
                    #     item.reset_length()
                    # else:
                    if type(item)!= int:
                        item=item.strip(' \t\n\r')
                    outlist.append(item)
        return outlist


    def getWords(self, sentence):
        tempList=[]
        tempWordList=sentence.split()
        p_dots= re.compile(r'[.]{2,}')
        #print(tempWordList)
        for word in tempWordList:
            temp=[]
            
            if "(" in word:
                temp=list(filter(lambda elem: elem!='',word.split("(")))
                if(temp):
                    temp=list(map(lambda elem: '('+elem, temp))
            elif ")" in word:
                temp=list(filter(lambda elem: elem!='',word.split(")")))
                if(temp):
                    temp=list(map(lambda elem: elem+')', temp))
                # temp.append(temp1[-1])
            # elif (("-" in word)&(not word.endswith("-"))):
            #     temp1=list(filter(lambda elem: elem!='',word.split("-")))
            #     if(temp1):
            #         temp=list(map(lambda elem: elem+'-', temp1[:-1]))
            #     temp.append(temp1[-1])
            elif (("?" in word)&(not word.endswith("?"))):
                temp1=list(filter(lambda elem: elem!='',word.split("?")))
                if(temp1):
                    temp=list(map(lambda elem: elem+'?', temp1[:-1]))
                temp.append(temp1[-1])
            elif ((":" in word)&(not word.endswith(":"))):
                temp1=list(filter(lambda elem: elem!='',word.split(":")))
                if(temp1):
                    temp=list(map(lambda elem: elem+':', temp1[:-1]))
                temp.append(temp1[-1])
            elif (("," in word)&(not word.endswith(","))):
                #temp=list(filter(lambda elem: elem!='',word.split(",")))
                temp1=list(filter(lambda elem: elem!='',word.split(",")))
                if(temp1):
                    temp=list(map(lambda elem: elem+',', temp1[:-1]))
                temp.append(temp1[-1])
            elif (("/" in word)&(not word.endswith("/"))):
                temp1=list(filter(lambda elem: elem!='',word.split("/")))
                if(temp1):
                    temp=list(map(lambda elem: elem+'/', temp1[:-1]))
                temp.append(temp1[-1])
                #print(index, temp)
            # elif "..." in word:
            #     #print("here")
            #     temp=list(filter(lambda elem: elem!='',word.split("...")))
            #     if(temp):
            #         if(word.endswith("...")):
            #             temp=list(map(lambda elem: elem+'...', temp))
            #         else:
            #            temp=list(map(lambda elem: elem+'...', temp[:-1]))+[temp[-1]]
            #     # temp.append(temp1[-1])
            # elif ".." in word:
            #     temp=list(filter(lambda elem: elem!='',word.split("..")))
            #     if(temp):
            #         if(word.endswith("..")):
            #             temp=list(map(lambda elem: elem+'..', temp))
            #         else:
            #             temp=list(map(lambda elem: elem+'..', temp[:-1]))+[temp[-1]]
            #     #temp.append(temp1[-1])
            elif (list(p_dots.finditer(word))):
                matched_spans= list(p_dots.finditer(word)) 
                temp=[]
                next_string_start=0
                for matched_span in matched_spans:
                    matched_start=matched_span.span()[0]
                    this_excerpt=word[next_string_start:matched_start]
                    if(this_excerpt):
                        temp.append(this_excerpt)
                    next_string_start=matched_span.span()[1]
                if(next_string_start<len(word)):
                    last_excerpt=word[next_string_start:]
                    if(last_excerpt):
                        temp.append(last_excerpt)
            elif "…" in word:
                temp=list(filter(lambda elem: elem!='',word.split("…")))
                if(temp):
                    if(word.endswith("…")):
                        temp=list(map(lambda elem: elem+'…', temp))
                    else:
                        temp=list(map(lambda elem: elem+'…', temp[:-1]))+[temp[-1]]
            else:
                #if word not in string.punctuation:
                temp=[word]
            if(temp):
                tempList.append(temp)
        tweetWordList=self.flatten(tempList,[])
        return tweetWordList

    def get_Candidates(self, sequence, CTrie,flag):
        #flag: debug_flag
        candidateList=[]
        left=0
        start_node=CTrie
        last_cand="NAN"
        last_cand_substr=""
        reset=False
        right=0
        while (right < len(sequence)):
            # if(flag):
            #     print(right)
            if(reset):
                start_node=CTrie
                last_cand_substr=""
                left=right
            curr_text=sequence[right][0]
            curr_pos=[sequence[right][1]]
            #normalized curr_text
            curr=self.normalize(sequence[right][0])
            cand_str=self.normalize(last_cand_substr+" "+curr)
            cand_str_wPunct=(last_cand_substr+" "+curr_text).lower()
            last_cand_sequence=sequence[left:(right+1)]
            last_cand_text=' '.join(str(e[0]) for e in last_cand_sequence)
            last_cand_text_norm=self.normalize(' '.join(str(e[0]) for e in last_cand_sequence))
            if(flag):
                print("==>",cand_str,last_cand_text_norm)
            if((cand_str==last_cand_text_norm)&((curr in start_node.path.keys())|(curr_text.lower() in start_node.path.keys()))):
            #if (((curr in start_node.path.keys())&(cand_str==last_cand_text_norm))|(curr_text.lower() in start_node.path.keys())):
                if flag:
                    print("=>",cand_str,last_cand_text)
                reset=False
                if (curr_text.lower() in start_node.path.keys()):
                    if (start_node.path[curr_text.lower()].value_valid):
                        last_cand_pos=[e[1] for e in last_cand_sequence]
                        last_cand_batch=start_node.path[curr_text.lower()].feature_list[-1]
                        last_cand=last_cand_text
                    elif(curr in start_node.path.keys()):
                        if ((start_node.path[curr].value_valid)):
                            last_cand_pos=[e[1] for e in last_cand_sequence]
                            last_cand=last_cand_text
                            last_cand_batch=start_node.path[curr].feature_list[-1]
                        else:
                            if((right==(len(sequence)-1))&(last_cand=="NAN")&(left<right)):
                                #print("hehe",cand_str)
                                right=left
                                reset=True
                    else:
                        if((right==(len(sequence)-1))&(last_cand=="NAN")&(left<right)):
                            #print("hehe",cand_str)
                            right=left
                            reset=True
                elif ((start_node.path[curr].value_valid)&(cand_str==last_cand_text_norm)):
                    # if flag:
                    #     print("==",last_cand_text)
                    last_cand_pos=[e[1] for e in last_cand_sequence]
                    last_cand=last_cand_text
                    last_cand_batch=start_node.path[curr].feature_list[-1]
                else:
                    if((right==(len(sequence)-1))&(last_cand=="NAN")&(left<right)):
                        #print("hehe",cand_str)
                        right=left
                        reset=True
                if((curr_text.lower() in start_node.path.keys())&(cand_str==last_cand_text_norm)):
                    start_node=start_node.path[curr_text.lower()]
                    last_cand_substr=cand_str_wPunct
                else:
                    start_node=start_node.path[curr]
                    last_cand_substr=cand_str
            else:
                #print("=>",cand_str,last_cand_text)
                if(last_cand!="NAN"):
                    candidateList.append((last_cand,last_cand_pos,last_cand_batch))
                    last_cand="NAN"
                    if(start_node!=CTrie):
                        start_node=CTrie
                        last_cand_substr=""
                        if curr in start_node.path.keys():
                            # if(flag):
                            #     print("here",curr)
                            reset=False
                            if start_node.path[curr].value_valid:
                                last_cand_text=curr_text
                                last_cand_pos=curr_pos
                                last_cand=last_cand_text
                                last_cand_batch=start_node.path[curr].feature_list[-1]
                            left=right
                            start_node=start_node.path[curr]
                            last_cand_substr=curr
                        else:
                            reset=True
                    else:
                        reset=True
                else:
                    if(left<right):
                        # if(flag):
                        #     print(sequence[(left+1):(right+1)])
                        #candidateList.extend(self.get_Candidates(sequence[(left+1):(right+1)], CTrie, flag))
                        right=left
                        # if(flag):
                        #     print("++",right)
                    reset=True
            right+=1
        # if(flag):
        #     print(last_cand)
        if(last_cand!="NAN"):
            candidateList.append((last_cand,last_cand_pos,last_cand_batch))
        return candidateList


    def append_rows(self,df_holder):
    
        df = pd.DataFrame(df_holder)
        #self.data_frame_holder=self.data_frame_holder.append(df,ignore_index=True)
        #self.data_frame_holder=self.data_frame_holder.reset_index(drop=True)
        return df




    def join_token_tuples(self,list_of_tuples):
        #print(string.punctuation)
        combined_str=(' '.join(tuple[0] for tuple in list_of_tuples)).lstrip(string.punctuation).rstrip(string.punctuation).strip()
        combined_pos='*'.join(str(tuple[1]) for tuple in list_of_tuples)
        combined_tuple=(combined_str,combined_pos,list_of_tuples[0][2],list_of_tuples[0][3],list_of_tuples[0][4],list_of_tuples[0][5],list_of_tuples[0][6])
        return combined_tuple




    def all_capitalized(self,candidate):
        strip_op=candidate
        strip_op=(((strip_op.lstrip(string.punctuation)).rstrip(string.punctuation)).strip())
        strip_op=(strip_op.lstrip('“‘’”')).rstrip('“‘’”')
        strip_op= self.rreplace(self.rreplace(self.rreplace(strip_op,"'s","",1),"’s","",1),"’s","",1)
        prep_article_list=prep_list+article_list+self.phase2stopwordList
        word_list=strip_op.split()
        for i in range(len(word_list)):
            word=word_list[i]
            if((word[0].isupper())|(word[0].isdigit())):
                continue
            else:
                if(word in prep_article_list):
                    if (i!=0):
                        continue
                    else:
                        return False
                elif(word in conjoiner):
                    continue
                else:
                    return False
        return True


    def check_feature_update(self, candidate_tuple,non_discriminative_flag):
        #print(candidate_tuple)
        if(non_discriminative_flag):
            return 7
        candidateText=candidate_tuple[0]
        position=candidate_tuple[1]
        word_list=candidateText.split()
        if candidateText.islower():
            return 6
        elif candidateText.isupper():
            return 5
        elif (len(word_list)==1):
            #start-of-sentence-check
            if self.all_capitalized(candidateText):
                if(int(position[0])==0):
                    return 4
                else:
                    return 2
            else:
                return 3
        else:
            if(self.all_capitalized(candidateText)):
                return 2
            else:
                return 3

    def addEmbedding(self, candidate_tuple,contextual_embedding_vector):

        candidateText=candidate_tuple[0]
        normalized_candidate=self.normalize(candidateText)

        if(normalized_candidate in self.candidateEmbedding_records.keys()):
            curr_id = self.candidate_string_freq_dict[normalized_candidate]
            self.candidateEmbedding_records[normalized_candidate][curr_id]=(contextual_embedding_vector,)
            self.candidateEmbeddingPool[normalized_candidate].append(contextual_embedding_vector.tolist())
        else:
            curr_id = 0
            self.candidateEmbedding_records[normalized_candidate]={curr_id:(contextual_embedding_vector,)}
            self.candidateEmbeddingPool[normalized_candidate]=[contextual_embedding_vector.tolist()]
        self.candidate_string_freq_dict[normalized_candidate]=curr_id+1
        
        return curr_id

    def extract(self,tweetBaseInput,CTrie,phase2stopwordList,new_or_old):


        if(self.counter==0):
            #output_queue
            self.candidate_string_freq_dict= {} #not mindful of clusters

            self.CandidateBase_dict_alt={}
            self.candidateEmbeddingDict={}

            #new data structures for clustering
            self.candidateEmbeddingPool={}
            self.candidateEmbedding_records={} # {candidate:{mention_id: <contextual embedding,cluster_id>...},....} multi level dictionary

            self.ambiguous_candidate_distanceDict_prev={}
            self.partition_dict={}
            self.good_candidates=[]
            self.bad_candidates=[]
            self.ambiguous_candidates=[]

            self.aggregator_incomplete_tweets=pd.DataFrame([], columns=['index', 'entry_batch', 'tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
            self.just_converted_tweets=pd.DataFrame([], columns=['index', 'entry_batch', 'tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
            #self.data_frame_holder=pd.DataFrame([], columns=['index','entry_batch','tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates'])
            self.raw_tweets_for_others=pd.DataFrame([], columns=['index','entry_batch','tweetID', 'sentID', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])

            self.accuracy_tuples_prev_batch=[]
            self.accuracy_vals=[]
            
            #frequency_w_decay related information
            self.ambiguous_candidates_reintroduction_dict={}

            #### other systems
            self.accuracy_vals_stanford=[]
            self.accuracy_vals_opencalai=[]
            self.accuracy_vals_ritter=[]
            self.accuracy_vals_neuroner=[]

            self.number_of_seen_tweets_per_batch=[]
        self.phase2stopwordList=phase2stopwordList
        self.number_of_seen_tweets_per_batch.append(len(tweetBaseInput))


        #data_frame_holder=pd.DataFrame([], columns=['index','entry_batch','tweetID', 'sentID', 'hashtags', 'user', 'TweetSentence','phase1Candidates', '2nd Iteration Candidates', '2nd Iteration Candidates Unnormalized'])
        phase1_holder_holder=[]
        phase2_candidates_holder=[]
        phase2_unnormalized_candidates_holder=[]
        df_holder=[]


        combined_list_here=([]+list(cachedStopWords)+chat_word_list+day_list+month_list+article_list+prep_list)
        combined_list_filtered=list(filter(lambda word: word not in (prep_list+article_list+month_list+self.phase2stopwordList), combined_list_here))

        self.entity_phrase_embedder.eval()
        #--------------------------------------PHASE II---------------------------------------------------
        for index, row in tweetBaseInput.iterrows():

            #phase 1 candidates for one sentence
            phase1_holder=[]

            tweetText=str(row['TweetSentence'])
            tweetWordList = row['tweetwordList']
            sentID=str(row['sentID'])
            tweetID=str(row['tweetID'])
            phase1Candidates=str(row['phase1CandidatesWPositions'])
            batch=int(row['entry_batch'])
            contextual_embeddings_dict= {ind: embedding for ind, embedding in enumerate(row['contextual_embeddings'])}
            
            # print('====',tweetID,sentID)
            # print('tweetWordList:',tweetWordList)
            
            non_discriminative_flag=False
            phase1CandidatesList=[]

            # print('phase1Candidates:',phase1Candidates)

            if (phase1Candidates !='nan'):
                phase1Raw=phase1Candidates.split("||")
                phase1Raw = list(filter(None, phase1Raw))


                for entities_with_loc in phase1Raw:
                    candidateData=entities_with_loc.split("::")
                    entity_to_store=candidateData[0]
                    #print(entity_to_store)
                    position=candidateData[1]
                    #print(position)
                    entityType=candidateData[2]
                    phase1_holder.append((entity_to_store,entityType,position))
                    phase1_holder.clear()
                    phase1CandidatesList.append(entity_to_store.lower())

                phase1_holder_holder.append(copy.deepcopy(phase1_holder))
                

            else:
                non_discriminative_flag=True
                phase1_holder_holder.append([])

            
            # tweetWordList=self.getWords(tweetText)
            # tweetWordList= [(token,idx) for idx,token in enumerate(tweetWordList)]
            
            tweetWordList_stopWords=list(filter (lambda word: ((((word[0].strip()).strip(punct)).lower() in combined_list_filtered)|(word[0].strip() in punct)|(word[0].startswith('@'))|(word[0].startswith('#'))), tweetWordList))

            # phase 2 candidate tuples without stopwords for a sentence
            c=[(y[0],str(y[1]),tweetID,sentID,'ne',batch,time) for y  in tweetWordList if y not in tweetWordList_stopWords ]
            #c=[(y[0],str(y[1])) for y  in tweetWordList if y not in tweetWordList_stopWords ]

            
            sequences=[]
            for k, g in groupby(enumerate(c), lambda element: element[0]-int(element[1][1])):
                sequences.append(list(map(itemgetter(1), g)))

            # print('phaseII candidates:')
            phase2_candidates=[]
            phase2_candidates_unnormalized=[]
            for sequence in sequences:
                seq_candidate_list=self.get_Candidates(sequence, CTrie,False)
                if(seq_candidate_list):
                    for candidate_tuple in seq_candidate_list:

                        # extract candidate token embeddings
                        candidate_token_embeddings = torch.stack([contextual_embeddings_dict[int(position)] for position in candidate_tuple[1]])
                        assert torch.isnan(candidate_token_embeddings).any() == False
                        tensor_inter = torch.mean(candidate_token_embeddings,dim=0)
                        assert torch.isnan(tensor_inter).any() == False
                        tensor_inter_norm = torch.norm(tensor_inter, p=2,dim=0)
                        
                        if (not torch.is_nonzero(tensor_inter_norm)):
                            print(candidate_tuple, 'adjustement needed')
                            tensor_inter = torch.add(tensor_inter,1e-7)
                            tensor_inter_norm = torch.norm(tensor_inter, p=2,dim=0)
                        

                        candidate_tokens_avg_embedding = tensor_inter/tensor_inter_norm
                        assert torch.isnan(candidate_tokens_avg_embedding).any() == False

                        # !! must unsqueeze at test time with embedder else batchnorm throws error
                        candidate_tokens_avg_embedding = candidate_tokens_avg_embedding.unsqueeze(0)
                        
                        candidate_embedding = (self.entity_phrase_embedder.getEmbedding(candidate_tokens_avg_embedding)).squeeze(0)
                        
                        assert torch.isnan(candidate_embedding).any() == False

                        # print(candidate_tuple,candidate_token_embeddings.shape)
                        # print('candidate_embedding:',candidate_embedding.shape)

                        #inserts into CandidatePool: pre-clustering
                        # if not ((float(batch)<self.counter)&(candidate_tuple[-1]<self.counter)):
                        mention_id = self.addEmbedding(candidate_tuple,candidate_embedding)
                        # self.update_Candidatedict(candidate_tuple,non_discriminative_flag,candidate_embedding.tolist())

                        phase2_candidates.append(self.normalize(candidate_tuple[0])+'||'+str(mention_id))
                        phase2_candidates_unnormalized.append(candidate_tuple[0]+'||'+str(mention_id))

            phase2_candidates_holder.append(phase2_candidates)
            phase2_unnormalized_candidates_holder.append(phase2_candidates_unnormalized)

            dict1 = {'entry_batch':batch, 'tweetID':tweetID, 'sentID':sentID, 'TweetSentence':tweetText, 'phase1Candidates':phase1CandidatesList,'2nd Iteration Candidates':phase2_candidates,'2nd Iteration Candidates Unnormalized':phase2_candidates_unnormalized}

            df_holder.append(dict1)

        # sanity check
        for candidate in self.candidate_string_freq_dict.keys():
            assert self.candidate_string_freq_dict[candidate] == len(self.candidateEmbedding_records[candidate])
            assert self.candidate_string_freq_dict[candidate] == len(self.candidateEmbeddingPool[candidate])

        return df_holder,phase2_candidates_holder,phase2_unnormalized_candidates_holder

## **Running the NER Globalizer engine**

In [ ]:
def calculate_f1_ner_engine(tweet_to_sentences_w_annotation, ner_arrays):
    
    entity_types = ['org','misc', 'loc', 'per']
    confusion_matrices = {entity_type: {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0} for entity_type in entity_types}
    scores = {entity_type: {'p': 0, 'r': 0, 'f1': 0} for entity_type in entity_types}

    out_str = ''
    mistyping_count = 0
    total_mention_count = 0

    annotation_dict={}
    output_dict={}

    for tweetID in tweet_to_sentences_w_annotation.keys():
        unrecovered_annotated_mention_list=[]
        annotated_mention_list=tweet_to_sentences_w_annotation[tweetID][1]
        output_mentions_list=[]
        true_positive_list=[]
        idRange=tweet_to_sentences_w_annotation[tweetID][0]

        for sentID in range(idRange[0],idRange[1]):
            output_mentions_list+=ner_arrays[sentID]
        output_mentions_list=[(elem[0],elem[1].lower()) for elem in output_mentions_list]
        print(tweetID,annotated_mention_list,output_mentions_list)
        out_str+=str(tweetID)+str(annotated_mention_list)+str(output_mentions_list)+'\n'

        total_mention_count+=len(annotated_mention_list)

        for annotated_entity in annotated_mention_list:
            try:
                annotation_dict[annotated_entity]+=1
            except KeyError:
                annotation_dict[annotated_entity]=1

        while(annotated_mention_list):
            if(len(output_mentions_list)):
                annotated_candidate= annotated_mention_list.pop()
                if(annotated_candidate in output_mentions_list):
                    output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                    # tp_counter_inner+=1 # for emd
                    true_positive_list.append(annotated_candidate)
                    # #for ner
                    # entity_type=annotated_candidate[1]
                    # confusion_matrices[entity_type]['TP']+=1
                else:
                    unrecovered_annotated_mention_list.append(annotated_candidate)
            else:
                unrecovered_annotated_mention_list.extend(annotated_mention_list)
                break
        # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)

        # fn_counter_inner=len(unrecovered_annotated_mention_list)
        # fp_counter_inner=all_postitive_counter_inner - tp_counter_inner
        # print(tp_counter_inner,fp_counter_inner,fn_counter_inner)

        print('true positives:',true_positive_list)
        print('false positives:',output_mentions_list)
        print('false negatives:',unrecovered_annotated_mention_list)

        out_str+= 'true positives:'+str(true_positive_list)+'\n'
        out_str+= 'false positives:'+str(output_mentions_list)+'\n'
        out_str+= 'false negatives:'+str(unrecovered_annotated_mention_list)+'\n'

        mistyped_candidates = []
        false_positive_candidate_strings = [elem[0] for elem in output_mentions_list]

        for mention_tup in true_positive_list:
            output_dict[mention_tup] = True
            entity_type=mention_tup[1]
            if(mention_tup[0]):
                confusion_matrices[entity_type]['TP']+=1

        for mention_tup in unrecovered_annotated_mention_list:
            entity_type=mention_tup[1]
            if(mention_tup[0]):
                confusion_matrices[entity_type]['FN']+=1

        for mention_tup in output_mentions_list:
            entity_type=mention_tup[1]
            if(mention_tup[0]):
                confusion_matrices[entity_type]['FP']+=1

        while(unrecovered_annotated_mention_list):
            if(false_positive_candidate_strings):
                fn_candidate = unrecovered_annotated_mention_list.pop()
                if(fn_candidate[0] in false_positive_candidate_strings):
                    false_positive_candidate_strings.pop(false_positive_candidate_strings.index(fn_candidate[0]))
                    mistyped_candidates.append(fn_candidate)
            else:
                break
        print('mistypings:',mistyped_candidates)
        out_str+= 'mistypings:'+str(mistyped_candidates)+'\n'
        mistyping_count+= len(mistyped_candidates)
    
    # file1 = open("btc.txt", "w")
    file1 = open("tweets_3K.txt", "w")
    file1.write(out_str)
    file1.close()

    freq_bucket = {} 
    for candidate in annotation_dict.keys():
        candidate_freq = annotation_dict[candidate]
        flag = False
        if(candidate in output_dict):
            flag = True
        try:
            old_tup = freq_bucket[candidate_freq]
            if flag:
                freq_bucket[candidate_freq] = (old_tup[0]+1,old_tup[1])
            else:
                freq_bucket[candidate_freq] = (old_tup[0],old_tup[1]+1)
        except KeyError:
            if flag:
                freq_bucket[candidate_freq] = (1,0)
            else:
                freq_bucket[candidate_freq] = (0,1)
    
    x_axis=[]
    y_axis =[]
    cumulative_tp=0
    cumulative_annotated=0
    freq_bucket_sorted = {k : freq_bucket[k] for k in sorted(freq_bucket)}

    maxKey = list(freq_bucket_sorted.keys())[-1]
    print(freq_bucket_sorted.keys(),maxKey)

    step=0
    for key in range(maxKey):
        try:
            tup = freq_bucket_sorted[key]
        except KeyError:
            tup = (0,0)

        # freq_recall= tup[0]/(tup[0]+tup[1])

        if(step==5):
            if((cumulative_tp==0)&(cumulative_annotated==0)):
                freq_recall= y_axis[-1]
            else:
                freq_recall= cumulative_tp/cumulative_annotated
            # print(key, freq_recall)
            x_axis.append(key)
            y_axis.append(freq_recall)
            step=0
            cumulative_tp=0
            cumulative_annotated=0


        cumulative_tp+=tup[0]
        cumulative_annotated+=(tup[0]+tup[1])
        # freq_recall= cumulative_tp/cumulative_annotated
        step+=1

    print(x_axis)
    print(y_axis)

    # print20=20
    for ind, elem in enumerate(x_axis):
        # if(print20>0):
        print(x_axis[ind],':',y_axis[ind])

    print('========Entity Mention Detection========')
    print('Total mentions:',str(total_mention_count))
    print('Mistyped mentions:',str(mistyping_count))
    for entity_type in entity_types:
        print(entity_type.upper())
        try:
            precision = confusion_matrices[entity_type]['TP']/(confusion_matrices[entity_type]['TP']+confusion_matrices[entity_type]['FP'])
        except ZeroDivisionError:
            precision = 0
        try:
            recall = confusion_matrices[entity_type]['TP']/(confusion_matrices[entity_type]['TP']+confusion_matrices[entity_type]['FN'])
        except ZeroDivisionError:
            recall = 0
        try:
            f_measure = 2*precision*recall/(precision+recall)
        except ZeroDivisionError:
            f_measure = 0

        scores[entity_type]['p'] = precision
        scores[entity_type]['r'] = recall
        scores[entity_type]['f1'] = f_measure

        print('true positives: ',confusion_matrices[entity_type]['TP'])
        print('false positives: ',confusion_matrices[entity_type]['FP'])
        print('false negatives: ',confusion_matrices[entity_type]['FN'])

        print('precision: ',precision)
        print('recall: ',recall)
        print('f_measure: ',f_measure)

        print('------------------------')

In [ ]:
# tweets_unpartitoned=pd.read_csv('data/tweets_3k_annotated.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/tweets_6k_annotated.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/venezuela.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/covid_2K.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/broad_twitter_corpus.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/billdeblasio.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/roevwade.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/pikapika.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/ripcity.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/billnye.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/wnut17test_ner.csv',sep =',',keep_default_na=False)
tweets_unpartitoned=pd.read_csv('data/btctest_ner.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/venezuela_ner.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/covid_2K_ner.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/tweets_3k_annotated_ner.csv',sep =',',keep_default_na=False)

# tweets_unpartitoned=pd.read_csv('data/roevwade_ner.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/billnye_ner.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/pikapika_ner.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/billdeblasio_ner.csv',sep =',',keep_default_na=False)
# tweets_unpartitoned=pd.read_csv('data/ripcity_ner.csv',sep =',',keep_default_na=False)

In [ ]:
#tokenizer here is the BERT model's tokenizer
local_NER_Module= LocalNERModule(sentence_tokenizer, tokenizer, alt_model, device)

In [ ]:
# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'wnut17')

global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'btc')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'tweets_3k_annotated')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'tweets_6k_annotated')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'venezuela')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'covid_2K')

# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'billdeblasio')
# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'ripcity')
# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'pikapika')
# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'roevwade')
# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'billnye')


# global_NER_Module = GlobalNERModule(entityPhraseEmbedder, device,'classifier-train')

In [ ]:
print('Tweets are in memory...')

length=len(tweets_unpartitoned)
batch_size=length
print(length, batch_size)
val=math.ceil(length/batch_size)-1

In [ ]:
tweet_batch = tweets_unpartitoned
print(len(tweet_batch))

In [ ]:
tweet_to_sentences_w_annotation={}
# df_out_holder_Phase1=[]
total_time=0
reintroduction_threshold_dummy=0
max_batch_value=112
z_score=1

# for g, tweet_batch in tweets_unpartitoned.groupby(np.arange(length) //batch_size):
phaseI_timein=time.time()
g=0
tuple_of= local_NER_Module.extract(tweet_batch,g)
phaseI_timeout=time.time()

print('local emd time',(phaseI_timeout-phaseI_timein))

tweet_base=tuple_of[0]
contextual_embeddings=tuple_of[1]
candidate_base=tuple_of[2]
elapsedTime= tuple_of[4] - tuple_of[3]
phase2stopwordList=tuple_of[5]
# print('len of tweet_base = '  len(tweet_base))
tweet_to_sentences_w_annotation=tuple_of[6]
total_time+=elapsedTime
print(elapsedTime,total_time)

# df_out_holder_Phase1.append(tweet_base)

print ('Produced', g)
print("**********************************************************")


In [ ]:
# # #PHASE I CHECKING:
# calculate_f1_ner_engine(tweet_to_sentences_w_annotation, local_NER_Module.phaseIpredictions)

In [ ]:
additional_list = ['dannysimpson 12','new zealand','latin america','east coast','west coast','chatteris- manea','lasa national congress', 'aust special forces',
                   'adammacdougall 5','south korean','irish news','mv augusta brutale','myung ga','central asia','general motors','premier league','new zealand','west midlands','china town','celestine lake','maida vale','west africa','boeing 777','liztilley 84','liberal democrats','fort hood',
                   'white house','waltengoo nar','new york','north america','south america','united states','olive garden','crystal palace' ,'abortion','gmail','fox news','physics','trump administration' ,'fisa warrant','global warming',
                   'rogue one','labour party','fred hollows foundation','biddulph moor','lady mary downer','ac / dc','ac/dc','northern ontario','cape breton','atlantic canada','air canada','ontario tories','vancouver canucks','statistics canada','air canada','cloverhill prison','south sudan','coryjane 1080','hrjohn 44','dancohen 17','mt cook','president obama','southeast ukraine',
                   'supreme court','south korea','north korea','costa rica','new orleans', 'badlands national park','covid 19','harry _ styles', 'punta cana', 'los angeles','united kingdom','coronavirus',
                   'bbc news','bbc 1','bbc world','kuala lumpur','ffc','brfc','honey nutloops','australian open','blackberry messenger','hongkong','wakefield council','sir alex ferguson','russian bank','bank of ireland','kilkenny bridge','north galway','galway bay','northern ireland','tel aviv','sierra leone','evening standard']
for candidateText in additional_list:
    candidate_base.__setitem__(candidateText.split(),len(candidateText.split()),[],0)

In [ ]:
phaseI_candidates = candidate_base.displayTrie("",[])
print(str(len(phaseI_candidates))+' candidates from phaseI')
print(phaseI_candidates)

5699 candidates from phaseI
['tosca', 'gateshead', 'shaw', 'blink 182', 'blink technologies inc', 'dorothy gale', 'return to oz', 'chargers', 'sherlock', 'coe', 'stamford', 'facebook', 'facebook timeline', 'wadey', 'salmon valley', 'j edgar', 'j leno sito', 'j lo', 'j whaley music', 'j j abrams', 'j cole', 'leo', 'leo messi', 'leo johnson', 'pride and prejudice', 'fulham', 'wenger', 'hobo', 'rob', 'rob ogleby', 'rob mcclelland', 'rob ford', 'otis', 'green', 'green goddess', 'kane', 'disneyworld', 'florida', 'daniel', 'daniel james evans', 'daniel jones', 'daniel cormier', 'daniel sturridge', 'daniel ellsberg', 'daniel morcombe', 'daniel radcliffe', 'daniel finkelstein', 'woolwich', 'oracle', 'mysql', 'perl', 'java', 'city', 'city centre', 'city garden apts', 'london', 'london college', 'london king cross', 'yorkshire', 'yorkshire police', 'yorkshire dales', 'doncaster', 'doncaster rovers', 'leeds', 'lamb of god', 'mancity', 'liverpool', 'liverpool city council', 'kuyt', 'aguero', 'loui

In [ ]:
phaseII_timein=time.time()
global_ner_predictions = global_NER_Module.executor(max_batch_value,tweet_base,candidate_base,phase2stopwordList,z_score,reintroduction_threshold_dummy,tweet_batch)
phaseII_timeout=time.time()

print('global emd time',(phaseII_timeout-phaseII_timein))

In [ ]:
calculate_f1_ner_engine(tweet_to_sentences_w_annotation, global_ner_predictions)

## **Other stuff**

In [ ]:
from scipy.spatial.distance import cdist
import numpy as np
import torch
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.metrics import normalized_mutual_info_score as nmi
from pathlib import Path
import torchvision
import urllib.request
import os
import ssl
import zipfile
import platform
import ctypes


"""
Dip related stuff
"""

C_DIP_FILE = None


def dip_test(X, is_data_sorted=False, debug=False):
    n_points = X.shape[0]
    data_dip = dip(X, just_dip=True, is_data_sorted=is_data_sorted, debug=debug)
    pval = dip_pval(data_dip, n_points)
    return data_dip, pval


def dip(X, just_dip=False, is_data_sorted=False, debug=False):
    assert X.ndim == 1, "Data must be 1-dimensional for the dip-test. Your shape:{0}".format(X.shape)
    N = len(X)
    if not is_data_sorted:
        X = np.sort(X)
    if N < 4 or X[0] == X[-1]:
        d = 0.0
        return d if just_dip else (d, None, None)
    # Prepare data to match C data types
    if C_DIP_FILE is None:
        load_c_dip_file()
    X = np.asarray(X, dtype=np.float64)
    X_c = X.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    N_c = np.array([N]).ctypes.data_as(ctypes.POINTER(ctypes.c_int))
    dip_value = np.zeros(1, dtype=np.float).ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    low_high = np.zeros(4).ctypes.data_as(ctypes.POINTER(ctypes.c_int))
    modal_triangle = np.zeros(3).ctypes.data_as(ctypes.POINTER(ctypes.c_int))
    gcm = np.zeros(N).ctypes.data_as(ctypes.POINTER(ctypes.c_int))
    lcm = np.zeros(N).ctypes.data_as(ctypes.POINTER(ctypes.c_int))
    mn = np.zeros(N).ctypes.data_as(ctypes.POINTER(ctypes.c_int))
    mj = np.zeros(N).ctypes.data_as(ctypes.POINTER(ctypes.c_int))
    debug_c = np.array([1 if debug else 0]).ctypes.data_as(ctypes.POINTER(ctypes.c_int))
    # Execute C dip test
    _ = C_DIP_FILE.diptst(X_c, N_c, dip_value, low_high, modal_triangle, gcm, lcm, mn, mj, debug_c)
    dip_value = dip_value[0]
    if just_dip:
        return dip_value
    else:
        low_high = (low_high[0], low_high[1], low_high[2], low_high[3])
        modal_triangle = (modal_triangle[0], modal_triangle[1], modal_triangle[2])
        return dip_value, low_high, modal_triangle


def load_c_dip_file():
    global C_DIP_FILE
    files_path = os.path.dirname(__file__)
    # if platform.system() == "Windows":
    #     dip_compiled = files_path + "/dip.dll"
    # else:
    dip_compiled = files_path + "/dip.so"
    if os.path.isfile(dip_compiled):
        # load c file
        try:
            C_DIP_FILE = ctypes.CDLL(dip_compiled)
            C_DIP_FILE.diptst.restype = None
            C_DIP_FILE.diptst.argtypes = [ctypes.POINTER(ctypes.c_double),
                                          ctypes.POINTER(ctypes.c_int),
                                          ctypes.POINTER(ctypes.c_double),
                                          ctypes.POINTER(ctypes.c_int),
                                          ctypes.POINTER(ctypes.c_int),
                                          ctypes.POINTER(ctypes.c_int),
                                          ctypes.POINTER(ctypes.c_int),
                                          ctypes.POINTER(ctypes.c_int),
                                          ctypes.POINTER(ctypes.c_int),
                                          ctypes.POINTER(ctypes.c_int)]
        except Exception as e:
            print("[WARNING] Error while loading the C compiled dip file.")
            raise e
    else:
        raise Exception("C compiled dip file can not be found.\n"
                        "On Windows execute: gcc -fPIC -shared -std=c99 -o dip.dll dip.c\n"
                        "On Linux execute: gcc -fPIC -shared -o dip.so dip.c")


def dip_pval(data_dip, n_points):
    N, SIG, CV = _dip_table_values()
    i1 = N.searchsorted(n_points, side='left')
    i0 = i1 - 1
    # if n falls outside the range of tabulated sample sizes, use the
    # critical values for the nearest tabulated n (i.e. treat them as
    # 'asymptotic')
    i0 = max(0, i0)
    i1 = min(N.shape[0] - 1, i1)
    # interpolate on sqrt(n)
    n0, n1 = N[[i0, i1]]
    fn = float(n_points - n0) / (n1 - n0)
    y0 = np.sqrt(n0) * CV[i0]
    y1 = np.sqrt(n1) * CV[i1]
    sD = np.sqrt(n_points) * data_dip
    pval = 1. - np.interp(sD, y0 + fn * (y1 - y0), SIG)
    return pval


def _dip_table_values():
    N = np.array([4, 5, 6, 7, 8, 9, 10, 15, 20,
                  30, 50, 100, 200, 500, 1000, 2000, 5000, 10000,
                  20000, 40000, 72000])
    SIG = np.array([0., 0.01, 0.02, 0.05, 0.1, 0.2,
                    0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                    0.9, 0.95, 0.98, 0.99, 0.995, 0.998,
                    0.999, 0.9995, 0.9998, 0.9999, 0.99995, 0.99998,
                    0.99999, 1.])
    # [len(N), len(SIG)] table of critical values
    CV = np.array([[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.132559548782689,
                    0.157497369040235, 0.187401878807559, 0.20726978858736, 0.223755804629222, 0.231796258864192,
                    0.237263743826779, 0.241992892688593, 0.244369839049632, 0.245966625504691, 0.247439597233262,
                    0.248230659656638, 0.248754269146416, 0.249302039974259, 0.249459652323225, 0.24974836247845],
                   [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.108720593576329, 0.121563798026414, 0.134318918697053,
                    0.147298798976252, 0.161085025702604, 0.176811998476076, 0.186391796027944, 0.19361253363045,
                    0.196509139798845, 0.198159967287576, 0.199244279362433, 0.199617527406166, 0.199800941499028,
                    0.199917081834271, 0.199959029093075, 0.199978395376082, 0.199993151405815, 0.199995525025673,
                    0.199999835639211],
                   [0.0833333333333333, 0.0833333333333333, 0.0833333333333333, 0.0833333333333333,
                    0.0833333333333333, 0.0924514470941933, 0.103913431059949, 0.113885220640212, 0.123071187137781,
                    0.13186973390253, 0.140564796497941, 0.14941924112913, 0.159137064572627, 0.164769608513302,
                    0.179176547392782, 0.191862827995563, 0.202101971042968, 0.213015781111186, 0.219518627282415,
                    0.224339047394446, 0.229449332154241, 0.232714530449602, 0.236548128358969, 0.2390887911995,
                    0.240103566436295, 0.244672883617768],
                   [0.0714285714285714, 0.0714285714285714, 0.0714285714285714, 0.0725717816250742,
                    0.0817315478539489, 0.09405901819225269, 0.103244490800871, 0.110964599995697,
                    0.117807846504335, 0.124216086833531, 0.130409013968317, 0.136639642123068, 0.144240669035124,
                    0.159903395678336, 0.175196553271223, 0.184118659121501, 0.191014396174306, 0.198216795232182,
                    0.202341010748261, 0.205377566346832, 0.208306562526874, 0.209866047852379, 0.210967576933451,
                    0.212233348558702, 0.212661038312506, 0.21353618608817],
                   [0.0625, 0.0625, 0.06569119945032829, 0.07386511360717619, 0.0820045917762512,
                    0.0922700601131892, 0.09967371895993631, 0.105733531802737, 0.111035129847705,
                    0.115920055749988, 0.120561479262465, 0.125558759034845, 0.141841067033899, 0.153978303998561,
                    0.16597856724751, 0.172988528276759, 0.179010413496374, 0.186504388711178, 0.19448404115794,
                    0.200864297005026, 0.208849997050229, 0.212556040406219, 0.217149174137299, 0.221700076404503,
                    0.225000835357532, 0.233772919687683],
                   [0.0555555555555556, 0.0613018090298924, 0.0658615858179315, 0.0732651142535317,
                    0.0803941629593475, 0.0890432420913848, 0.0950811420297928, 0.09993808978110461,
                    0.104153560075868, 0.108007802361932, 0.112512617124951, 0.122915033480817, 0.136412639387084,
                    0.146603784954019, 0.157084065653166, 0.164164643657217, 0.172821674582338, 0.182555283567818,
                    0.188658833121906, 0.194089120768246, 0.19915700809389, 0.202881598436558, 0.205979795735129,
                    0.21054115498898, 0.21180033095039, 0.215379914317625],
                   [0.05, 0.0610132555623269, 0.0651627333214016, 0.0718321619656165, 0.077966212182459,
                    0.08528353598345639, 0.09032041737070989, 0.0943334983745117, 0.0977817630384725,
                    0.102180866696628, 0.109960948142951, 0.118844767211587, 0.130462149644819, 0.139611395137099,
                    0.150961728882481, 0.159684158858235, 0.16719524735674, 0.175419540856082, 0.180611195797351,
                    0.185286416050396, 0.191203083905044, 0.195805159339184, 0.20029398089673, 0.205651089646219,
                    0.209682048785853, 0.221530282182963],
                   [0.0341378172277919, 0.0546284208048975, 0.0572191260231815, 0.0610087367689692,
                    0.06426571373304441, 0.06922341079895911, 0.0745462114365167, 0.07920308789817621,
                    0.083621033469191, 0.08811984822029049, 0.093124666680253, 0.0996694393390689,
                    0.110087496900906, 0.118760769203664, 0.128890475210055, 0.13598356863636, 0.142452483681277,
                    0.150172816530742, 0.155456133696328, 0.160896499106958, 0.166979407946248, 0.17111793515551,
                    0.175900505704432, 0.181856676013166, 0.185743454151004, 0.192240563330562],
                   [0.033718563622065, 0.0474333740698401, 0.0490891387627092, 0.052719998201553,
                    0.0567795509056742, 0.0620134674468181, 0.06601638720690479, 0.06965060750664009,
                    0.07334377405927139, 0.07764606628802539, 0.0824558407118372, 0.08834462700173699,
                    0.09723460181229029, 0.105130218270636, 0.114309704281253, 0.120624043335821, 0.126552378036739,
                    0.13360135382395, 0.138569903791767, 0.14336916123968, 0.148940116394883, 0.152832538183622,
                    0.156010163618971, 0.161319225839345, 0.165568255916749, 0.175834459522789],
                   [0.0262674485075642, 0.0395871890405749, 0.0414574606741673, 0.0444462614069956,
                    0.0473998525042686, 0.0516677370374349, 0.0551037519001622, 0.058265005347493,
                    0.0614510857304343, 0.0649164408053978, 0.0689178762425442, 0.0739249074078291,
                    0.08147913793901269, 0.0881689143126666, 0.0960564383013644, 0.101478558893837,
                    0.10650487144103, 0.112724636524262, 0.117164140184417, 0.121425859908987, 0.126733051889401,
                    0.131198578897542, 0.133691739483444, 0.137831637950694, 0.141557509624351, 0.163833046059817],
                   [0.0218544781364545, 0.0314400501999916, 0.0329008160470834, 0.0353023819040016,
                    0.0377279973102482, 0.0410699984399582, 0.0437704598622665, 0.0462925642671299,
                    0.048851155289608, 0.0516145897865757, 0.0548121932066019, 0.0588230482851366,
                    0.06491363240467669, 0.0702737877191269, 0.07670958860791791, 0.0811998415355918,
                    0.0852854646662134, 0.09048478274902939, 0.0940930106666244, 0.0974904344916743,
                    0.102284204283997, 0.104680624334611, 0.107496694235039, 0.11140887547015, 0.113536607717411,
                    0.117886716865312],
                   [0.0164852597438403, 0.022831985803043, 0.0238917486442849, 0.0256559537977579,
                    0.0273987414570948, 0.0298109370830153, 0.0317771496530253, 0.0336073821590387,
                    0.0354621760592113, 0.0374805844550272, 0.0398046179116599, 0.0427283846799166,
                    0.047152783315718, 0.0511279442868827, 0.0558022052195208, 0.059024132304226,
                    0.0620425065165146, 0.06580160114660991, 0.0684479731118028, 0.0709169443994193,
                    0.0741183486081263, 0.0762579402903838, 0.0785735967934979, 0.08134583568891331,
                    0.0832963013755522, 0.09267804230967371],
                   [0.0111236388849688, 0.0165017735429825, 0.0172594157992489, 0.0185259426032926,
                    0.0197917612637521, 0.0215233745778454, 0.0229259769870428, 0.024243848341112,
                    0.025584358256487, 0.0270252129816288, 0.0286920262150517, 0.0308006766341406,
                    0.0339967814293504, 0.0368418413878307, 0.0402729850316397, 0.0426864799777448,
                    0.044958959158761, 0.0477643873749449, 0.0497198001867437, 0.0516114611801451,
                    0.0540543978864652, 0.0558704526182638, 0.0573877056330228, 0.0593365901653878,
                    0.0607646310473911, 0.0705309107882395],
                   [0.00755488597576196, 0.0106403461127515, 0.0111255573208294, 0.0119353655328931,
                    0.0127411306411808, 0.0138524542751814, 0.0147536004288476, 0.0155963185751048,
                    0.0164519238025286, 0.017383057902553, 0.0184503949887735, 0.0198162679782071,
                    0.0218781313182203, 0.0237294742633411, 0.025919578977657, 0.0274518022761997,
                    0.0288986369564301, 0.0306813505050163, 0.0320170996823189, 0.0332452747332959,
                    0.0348335698576168, 0.0359832389317461, 0.0369051995840645, 0.0387221159256424,
                    0.03993025905765, 0.0431448163617178],
                   [0.00541658127872122, 0.00760286745300187, 0.007949878346447991, 0.008521651834359399,
                    0.00909775605533253, 0.009889245210140779, 0.0105309297090482, 0.0111322726797384,
                    0.0117439009052552, 0.012405033293814, 0.0131684179320803, 0.0141377942603047,
                    0.0156148055023058, 0.0169343970067564, 0.018513067368104, 0.0196080260483234,
                    0.0206489568587364, 0.0219285176765082, 0.0228689168972669, 0.023738710122235,
                    0.0248334158891432, 0.0256126573433596, 0.0265491336936829, 0.027578430100536, 0.0284430733108,
                    0.0313640941982108],
                   [0.00390439997450557, 0.00541664181796583, 0.00566171386252323, 0.00607120971135229,
                    0.0064762535755248, 0.00703573098590029, 0.00749421254589299, 0.007920878896017331,
                    0.008355737247680061, 0.00882439333812351, 0.00936785820717061, 0.01005604603884,
                    0.0111019116837591, 0.0120380990328341, 0.0131721010552576, 0.0139655122281969,
                    0.0146889122204488, 0.0156076779647454, 0.0162685615996248, 0.0168874937789415,
                    0.0176505093388153, 0.0181944265400504, 0.0186226037818523, 0.0193001796565433,
                    0.0196241518040617, 0.0213081254074584],
                   [0.00245657785440433, 0.00344809282233326, 0.00360473943713036, 0.00386326548010849,
                    0.00412089506752692, 0.00447640050137479, 0.00476555693102276, 0.00503704029750072,
                    0.00531239247408213, 0.00560929919359959, 0.00595352728377949, 0.00639092280563517,
                    0.00705566126234625, 0.0076506368153935, 0.00836821687047215, 0.008863578928549141,
                    0.00934162787186159, 0.009932186363240289, 0.0103498795291629, 0.0107780907076862,
                    0.0113184316868283, 0.0117329446468571, 0.0119995948968375, 0.0124410052027886,
                    0.0129467396733128, 0.014396063834027],
                   [0.00174954269199566, 0.00244595133885302, 0.00255710802275612, 0.00273990955227265,
                    0.0029225480567908, 0.00317374638422465, 0.00338072258533527, 0.00357243876535982,
                    0.00376734715752209, 0.00397885007249132, 0.00422430013176233, 0.00453437508148542,
                    0.00500178808402368, 0.00542372242836395, 0.00592656681022859, 0.00628034732880374,
                    0.00661030641550873, 0.00702254699967648, 0.00731822628156458, 0.0076065423418208,
                    0.00795640367207482, 0.008227052458435399, 0.00852240989786251, 0.00892863905540303,
                    0.009138539330002131, 0.009522345795667729],
                   [0.00119458814106091, 0.00173435346896287, 0.00181194434584681, 0.00194259470485893,
                    0.00207173719623868, 0.00224993202086955, 0.00239520831473419, 0.00253036792824665,
                    0.00266863168718114, 0.0028181999035216, 0.00299137548142077, 0.00321024899920135,
                    0.00354362220314155, 0.00384330190244679, 0.00420258799378253, 0.00445774902155711,
                    0.00469461513212743, 0.00499416069129168, 0.00520917757743218, 0.00540396235924372,
                    0.00564540201704594, 0.00580460792299214, 0.00599774739593151, 0.00633099254378114,
                    0.00656987109386762, 0.00685829448046227],
                   [0.000852415648011777, 0.00122883479310665, 0.00128469304457018, 0.00137617650525553,
                    0.00146751502006323, 0.00159376453672466, 0.00169668445506151, 0.00179253418337906,
                    0.00189061261635977, 0.00199645471886179, 0.00211929748381704, 0.00227457698703581,
                    0.00250999080890397, 0.00272375073486223, 0.00298072958568387, 0.00315942194040388,
                    0.0033273652798148, 0.00353988965698579, 0.00369400045486625, 0.00383345715372182,
                    0.00400793469634696, 0.00414892737222885, 0.0042839159079761, 0.00441870104432879,
                    0.00450818604569179, 0.00513477467565583],
                   [0.000644400053256997, 0.000916872204484283, 0.000957932946765532, 0.00102641863872347,
                    0.00109495154218002, 0.00118904090369415, 0.00126575197699874, 0.00133750966361506,
                    0.00141049709228472, 0.00148936709298802, 0.00158027541945626, 0.00169651643860074,
                    0.00187306184725826, 0.00203178401610555, 0.00222356097506054, 0.00235782814777627,
                    0.00248343580127067, 0.00264210826339498, 0.0027524322157581, 0.0028608570740143,
                    0.00298695044508003, 0.00309340092038059, 0.00319932767198801, 0.00332688234611187,
                    0.00339316094477355, 0.00376331697005859]])
    return N, SIG, CV


"""
Autoencoder stuff
"""


def detect_device():
    """Automatically detects if you have a cuda enabled GPU"""
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    return device


def encode_batchwise(dataloader, model, device):
    """ Utility function for embedding the whole data set in a mini-batch fashion
    """
    embeddings = []
    for batch in dataloader:
        batch_data = batch.to(device)
        embeddings.append(model.encode(batch_data).detach().cpu())
    return torch.cat(embeddings, dim=0).numpy()


def get_trained_autoencoder(trainloader, learning_rate, n_epochs, device, optimizer_class, loss_fn,
                            input_dim, embedding_size, autoencoder_class):
    if embedding_size > input_dim:
        print(
            "WARNING: embedding_size is larger than the dimensionality of the input dataset. Setting embedding_size to",
            input_dim)
        embedding_size = input_dim
    # Pretrain Autoencoder
    autoencoder = autoencoder_class(input_dim=input_dim, embedding_size=embedding_size).to(device)
    optimizer = optimizer_class(autoencoder.parameters(), lr=learning_rate)
    autoencoder.start_training(trainloader, n_epochs, device, optimizer, loss_fn)
    return autoencoder


def squared_euclidean_distance(centers, embedded, weights=None):
    ta = centers.unsqueeze(0)
    tb = embedded.unsqueeze(1)
    squared_diffs = (ta - tb)
    if weights is not None:
        weights_unsqueezed = weights.unsqueeze(0).unsqueeze(1)
        squared_diffs = squared_diffs * weights_unsqueezed
    squared_diffs = squared_diffs.pow(2).sum(2)  # .mean(2) # TODO Evaluate this change
    return squared_diffs


def int_to_one_hot(label_tensor, n_labels):
    onehot = torch.zeros([label_tensor.shape[0], n_labels], dtype=torch.float, device=label_tensor.device)
    onehot.scatter_(1, label_tensor.unsqueeze(1).long(), 1.0)
    return onehot

"""
DipDECK stuff
"""


def _dip_deck(X, n_clusters_start, dip_merge_threshold, cluster_loss_weight, n_clusters_max, n_clusters_min, batch_size,
              learning_rate, pretrain_epochs, dedc_epochs, optimizer_class, loss_fn, autoencoder, embedding_size,
              max_cluster_size_diff_factor, debug):
    if n_clusters_max < n_clusters_min:
        raise Exception("n_clusters_max can not be smaller than n_clusters_min")
    if n_clusters_min <= 0:
        raise Exception("n_clusters_min must be greater than zero")
    if n_clusters_start < n_clusters_min:
        raise Exception("n_clusters can not be smaller than n_clusters_min")
    device = detect_device()
    trainloader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(*(torch.from_numpy(X).float(), torch.arange(0, X.shape[0]))),
        batch_size=batch_size,
        # sample random mini-batches from the data
        shuffle=True,
        drop_last=False)
    
    # create a Dataloader to test the autoencoder in mini-batch fashion (Important for validation)
    testloader = torch.utils.data.DataLoader(torch.from_numpy(X).float(),
                                             batch_size=batch_size,
                                             # Note that we deactivate the shuffling
                                             shuffle=False,
                                             drop_last=False)
    if autoencoder is None:
        autoencoder = get_trained_autoencoder(trainloader, learning_rate, pretrain_epochs, device,
                                              optimizer_class, loss_fn, X.shape[1], embedding_size,
                                              _DipDECK_Autoencoder)
        
    # Execute kmeans in embedded space - initial clustering
    embedded_data = encode_batchwise(testloader, autoencoder, device)
    kmeans = KMeans(n_clusters=n_clusters_start)
    kmeans.fit(embedded_data)
    init_centers = kmeans.cluster_centers_
    cluster_labels_cpu = kmeans.labels_

    # Get nearest points to optimal centers
    centers_cpu, embedded_centers_cpu = _get_nearest_points_to_optimal_centers(X, init_centers, embedded_data)
    # Initial dip values
    dip_matrix_cpu = _get_dip_matrix(embedded_data, embedded_centers_cpu, cluster_labels_cpu, n_clusters_start,
                                     max_cluster_size_diff_factor)
    
    # Reduce learning_rate from pretraining by a magnitude of 10
    dedc_learning_rate = learning_rate * 0.1
    optimizer = optimizer_class(autoencoder.parameters(), lr=dedc_learning_rate)
    # Start training
    cluster_labels_cpu, n_clusters_current, centers_cpu, autoencoder = _dip_deck_training(X, n_clusters_start,
                                                                                          dip_merge_threshold,
                                                                                          cluster_loss_weight,
                                                                                          centers_cpu,
                                                                                          cluster_labels_cpu,
                                                                                          dip_matrix_cpu,
                                                                                          n_clusters_max,
                                                                                          n_clusters_min, dedc_epochs,
                                                                                          optimizer, loss_fn,
                                                                                          autoencoder,
                                                                                          device, trainloader,
                                                                                          testloader,
                                                                                          max_cluster_size_diff_factor,
                                                                                          debug)
    # Return results
    return cluster_labels_cpu, n_clusters_current, centers_cpu, autoencoder


def _dip_deck_training(X, n_clusters_current, dip_merge_threshold, cluster_loss_weight, centers_cpu, cluster_labels_cpu,
                       dip_matrix_cpu, n_clusters_max, n_clusters_min, dedc_epochs, optimizer, loss_fn, autoencoder,
                       device, trainloader, testloader, max_cluster_size_diff_factor, debug):
    i = 0
    while i < dedc_epochs:
        cluster_labels_torch = torch.from_numpy(cluster_labels_cpu).long().to(device)
        centers_torch = torch.from_numpy(centers_cpu).float().to(device)
        dip_matrix_torch = torch.from_numpy(dip_matrix_cpu).float().to(device)

        # Get dip costs matrix
        dip_matrix_eye = dip_matrix_torch + torch.eye(n_clusters_current, device=device)
        dip_matrix_final = dip_matrix_eye / dip_matrix_eye.sum(1).reshape((-1, 1))

        # Iterate over batches
        for batch, ids in trainloader:
            batch_data = batch.to(device)
            embedded = autoencoder.encode(batch_data)
            reconstruction = autoencoder.decode(embedded)
            embedded_centers_torch = autoencoder.encode(centers_torch)

            # Reconstruction Loss
            ae_loss = loss_fn(reconstruction, batch_data)
            # Get distances between points and centers. Get nearest center
            squared_diffs = squared_euclidean_distance(embedded_centers_torch, embedded)

            # Update labels? Pause is needed, so cluster labels can adjust to the new structure
            if i != 0:
                # Update labels
                current_labels = squared_diffs.argmin(1)
                # cluster_labels_torch[ids] = current_labels
            else:
                current_labels = cluster_labels_torch[ids]
            onehot_labels = int_to_one_hot(current_labels, n_clusters_current).float()
            cluster_relationships = torch.matmul(onehot_labels, dip_matrix_final)
            escaped_diffs = cluster_relationships * squared_diffs

            # Normalize loss by cluster distances
            squared_center_diffs = squared_euclidean_distance(embedded_centers_torch, embedded_centers_torch)

            # Ignore zero values (diagonal)
            mask = torch.where(squared_center_diffs != 0)
            masked_center_diffs = squared_center_diffs[mask[0], mask[1]]
            sqrt_masked_center_diffs = masked_center_diffs.sqrt()
            masked_center_diffs_std = sqrt_masked_center_diffs.std() if len(sqrt_masked_center_diffs) > 2 else 0

            # Loss function
            cluster_loss = escaped_diffs.sum(1).mean() * (1 + masked_center_diffs_std) / sqrt_masked_center_diffs.mean()
            cluster_loss *= cluster_loss_weight
            loss = ae_loss + cluster_loss

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # cluster_labels_cpu = cluster_labels_torch.detach().cpu().numpy()
        # Update centers
        embedded_data = encode_batchwise(testloader, autoencoder, device)
        embedded_centers_cpu = autoencoder.encode(centers_torch).detach().cpu().numpy()
        cluster_labels_cpu = np.argmin(cdist(embedded_centers_cpu, embedded_data), axis=0)
        optimal_centers = np.array([np.mean(embedded_data[cluster_labels_cpu == cluster_id], axis=0) for cluster_id in
                                    range(n_clusters_current)])
        centers_cpu, embedded_centers_cpu = _get_nearest_points_to_optimal_centers(X, optimal_centers, embedded_data)
        
        # Update Dips
        dip_matrix_cpu = _get_dip_matrix(embedded_data, embedded_centers_cpu, cluster_labels_cpu, n_clusters_current,
                                         max_cluster_size_diff_factor)

        if debug:
            print(
                "Iteration {0}  (n_clusters = {4}) - reconstruction loss: {1} / cluster loss: {2} / total loss: {3}".format(
                    i, ae_loss.item(), cluster_loss.item(), loss.item(), n_clusters_current))
            print("max dip", np.max(dip_matrix_cpu), " at ",
                  np.unravel_index(np.argmax(dip_matrix_cpu, axis=None), dip_matrix_cpu.shape))
        # i is increased here. Else next iteration will start with i = 1 instead of 0 after a merge
        i += 1
        # Start merging procedure
        dip_argmax = np.unravel_index(np.argmax(dip_matrix_cpu, axis=None), dip_matrix_cpu.shape)
        # Is merge possible?
        if i != 0:
            while dip_matrix_cpu[dip_argmax] >= dip_merge_threshold and n_clusters_current > n_clusters_min:
                if debug:
                    print("Start merging in iteration {0}.\nMerging clusters {1} with dip value {2}.".format(i,
                                                                                                             dip_argmax,
                                                                                                             dip_matrix_cpu[
                                                                                                                 dip_argmax]))
                # Reset iteration and reduce number of cluster
                i = 0
                n_clusters_current -= 1
                cluster_labels_cpu, centers_cpu, embedded_centers_cpu, dip_matrix_cpu = \
                    _merge_by_dip_value(X, embedded_data, cluster_labels_cpu, dip_argmax, n_clusters_current,
                                        centers_cpu,
                                        embedded_centers_cpu, max_cluster_size_diff_factor)
                dip_argmax = np.unravel_index(np.argmax(dip_matrix_cpu, axis=None), dip_matrix_cpu.shape)
        if n_clusters_current == 1:
            if debug:
                print("Only one cluster left")
            break
    return cluster_labels_cpu, n_clusters_current, centers_cpu, autoencoder


def _merge_by_dip_value(X, embedded_data, cluster_labels_cpu, dip_argmax, n_clusters_current, centers_cpu,
                        embedded_centers_cpu, max_cluster_size_diff_factor):
    # Get points in clusters
    points_in_center_1 = len(cluster_labels_cpu[cluster_labels_cpu == dip_argmax[0]])
    points_in_center_2 = len(cluster_labels_cpu[cluster_labels_cpu == dip_argmax[1]])
    # update labels
    for j, l in enumerate(cluster_labels_cpu):
        if l == dip_argmax[0] or l == dip_argmax[1]:
            cluster_labels_cpu[j] = n_clusters_current - 1
        elif l < dip_argmax[0] and l < dip_argmax[1]:
            cluster_labels_cpu[j] = l
        elif l > dip_argmax[0] and l > dip_argmax[1]:
            cluster_labels_cpu[j] = l - 2
        else:
            cluster_labels_cpu[j] = l - 1
    # Find new center position
    optimal_new_center = (embedded_centers_cpu[dip_argmax[0]] * points_in_center_1 +
                          embedded_centers_cpu[dip_argmax[1]] * points_in_center_2) / (
                                 points_in_center_1 + points_in_center_2)
    new_center_cpu, new_embedded_center_cpu = _get_nearest_points_to_optimal_centers(X, [optimal_new_center],
                                                                                     embedded_data)
    # Remove the two old centers and add the new one
    centers_cpu_tmp = np.delete(centers_cpu, dip_argmax, axis=0)
    centers_cpu = np.append(centers_cpu_tmp, new_center_cpu, axis=0)
    embedded_centers_cpu_tmp = np.delete(embedded_centers_cpu, dip_argmax, axis=0)
    embedded_centers_cpu = np.append(embedded_centers_cpu_tmp, new_embedded_center_cpu, axis=0)
    # Update dip values
    dip_matrix_cpu = _get_dip_matrix(embedded_data, embedded_centers_cpu, cluster_labels_cpu,
                                     n_clusters_current, max_cluster_size_diff_factor)
    return cluster_labels_cpu, centers_cpu, embedded_centers_cpu, dip_matrix_cpu


def _get_nearest_points_to_optimal_centers(X, optimal_centers, embedded_data):
    best_center_points = np.argmin(cdist(optimal_centers, embedded_data), axis=1)
    centers_cpu = X[best_center_points, :]
    embedded_centers_cpu = embedded_data[best_center_points, :]
    return centers_cpu, embedded_centers_cpu


def _get_nearest_points(points_in_larger_cluster, center, size_smaller_cluster, max_cluster_size_diff_factor,
                        min_sample_size):
    distances = cdist(points_in_larger_cluster, [center])
    nearest_points = np.argsort(distances, axis=0)
    # Check if more points should be taken because the other cluster is too small
    sample_size = size_smaller_cluster * max_cluster_size_diff_factor
    if size_smaller_cluster + sample_size < min_sample_size:
        sample_size = min(min_sample_size - size_smaller_cluster, len(points_in_larger_cluster))
    subset_all_points = points_in_larger_cluster[nearest_points[:sample_size, 0]]
    return subset_all_points


def _get_dip_matrix(data, dip_centers, dip_labels, n_clusters, max_cluster_size_diff_factor=2, min_sample_size=100):
    dip_matrix = np.zeros((n_clusters, n_clusters))
    # Loop over all combinations of centers
    for i in range(0, n_clusters - 1):
        for j in range(i + 1, n_clusters):
            center_diff = dip_centers[i] - dip_centers[j]
            points_in_i = data[dip_labels == i]
            points_in_j = data[dip_labels == j]
            points_in_i_or_j = np.append(points_in_i, points_in_j, axis=0)
            proj_points = np.dot(points_in_i_or_j, center_diff)
            _, dip_p_value = dip_test(proj_points)
            # Check if clusters sizes differ heavily
            if points_in_i.shape[0] > points_in_j.shape[0] * max_cluster_size_diff_factor or \
                    points_in_j.shape[0] > points_in_i.shape[0] * max_cluster_size_diff_factor:
                if points_in_i.shape[0] > points_in_j.shape[0] * max_cluster_size_diff_factor:
                    points_in_i = _get_nearest_points(points_in_i, dip_centers[j], points_in_j.shape[0],
                                                      max_cluster_size_diff_factor, min_sample_size)
                elif points_in_j.shape[0] > points_in_i.shape[0] * max_cluster_size_diff_factor:
                    points_in_j = _get_nearest_points(points_in_j, dip_centers[i], points_in_i.shape[0],
                                                      max_cluster_size_diff_factor, min_sample_size)
                points_in_i_or_j = np.append(points_in_i, points_in_j, axis=0)
                proj_points = np.dot(points_in_i_or_j, center_diff)
                _, dip_p_value_2 = dip_test(proj_points)
                dip_p_value = min(dip_p_value, dip_p_value_2)
            # Add pval to dip matrix
            dip_matrix[i][j] = dip_p_value
            dip_matrix[j][i] = dip_p_value
    return dip_matrix


class _DipDECK_Autoencoder(torch.nn.Module):

    def __init__(self, input_dim: int, embedding_size: int):
        super(_DipDECK_Autoencoder, self).__init__()

        # make a sequential list of all operations you want to apply for encoding a data point
        self.encoder = torch.nn.Sequential(
            # Linear layer (just a matrix multiplication)
            torch.nn.Linear(input_dim, 500),
            # apply an elementwise non-linear function
            torch.nn.LeakyReLU(inplace=True),
            torch.nn.Linear(500, 500),
            torch.nn.LeakyReLU(inplace=True),
            torch.nn.Linear(500, 2000),
            torch.nn.LeakyReLU(inplace=True),
            torch.nn.Linear(2000, embedding_size))

        # make a sequential list of all operations you want to apply for decoding a data point
        # In our case this is a symmetric version of the encoder
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(embedding_size, 2000),
            torch.nn.LeakyReLU(inplace=True),
            torch.nn.Linear(2000, 500),
            torch.nn.LeakyReLU(inplace=True),
            torch.nn.Linear(500, 500),
            torch.nn.LeakyReLU(inplace=True),
            torch.nn.Linear(500, input_dim),
        )

    def encode(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: input data point, can also be a mini-batch of points

        Returns:
            embedded: the embedded data point with dimensionality embedding_size
        """
        return self.encoder(x)

    def decode(self, embedded: torch.Tensor) -> torch.Tensor:
        """
        Args:
            embedded: embedded data point, can also be a mini-batch of embedded points

        Returns:
            reconstruction: returns the reconstruction of a data point
        """
        return self.decoder(embedded)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """ Applies both encode and decode function.
        The forward function is automatically called if we call self(x).
        Args:
            x: input data point, can also be a mini-batch of embedded points

        Returns:
            reconstruction: returns the reconstruction of a data point
        """
        embedded = self.encode(x)
        reconstruction = self.decode(embedded)
        return reconstruction

    def start_training(self, trainloader, n_epochs, device, optimizer, loss_fn):
        for _ in range(n_epochs):
            for batch, _ in trainloader:
                # load batch on device
                batch_data = batch.to(device)
                reconstruction = self.forward(batch_data)
                loss = loss_fn(reconstruction, batch_data)
                # reset gradients from last iteration
                optimizer.zero_grad()
                # calculate gradients and reset the computation graph
                loss.backward()
                # update the internal params (weights, etc.)
                optimizer.step()


class DipDECK():

    def __init__(self, n_clusters_start=35, dip_merge_threshold=0.9, cluster_loss_weight=1, n_clusters_max=np.inf,
                 n_clusters_min=1, batch_size=256, learning_rate=1e-3, pretrain_epochs=100, dedc_epochs=50,
                 optimizer_class=torch.optim.Adam, loss_fn=torch.nn.MSELoss(), autoencoder=None, embedding_size=5,
                 max_cluster_size_diff_factor=2, debug=False):
        self.n_clusters_start = n_clusters_start
        self.dip_merge_threshold = dip_merge_threshold
        self.cluster_loss_weight = cluster_loss_weight
        self.n_clusters_max = n_clusters_max
        self.n_clusters_min = n_clusters_min
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.pretrain_epochs = pretrain_epochs
        self.dedc_epochs = dedc_epochs
        self.optimizer_class = optimizer_class
        self.loss_fn = loss_fn
        self.autoencoder = autoencoder
        self.embedding_size = embedding_size
        self.max_cluster_size_diff_factor = max_cluster_size_diff_factor
        self.debug = debug

    def fit(self, X):
        labels, n_clusters, centers, autoencoder = _dip_deck(X, self.n_clusters_start, self.dip_merge_threshold,
                                                             self.cluster_loss_weight, self.n_clusters_max,
                                                             self.n_clusters_min, self.batch_size, self.learning_rate,
                                                             self.pretrain_epochs, self.dedc_epochs,
                                                             self.optimizer_class,
                                                             self.loss_fn, self.autoencoder, self.embedding_size,
                                                             self.max_cluster_size_diff_factor, self.debug)
        self.labels_ = labels
        self.n_clusters_ = n_clusters
        self.cluster_centers_ = centers
        self.autoencoder = autoencoder


"""
Data loader
"""

def _get_download_dir():
    downloads_path = str(Path.home() / "Downloads/dipDECK_data")
    if not os.path.isdir(downloads_path):
        os.makedirs(downloads_path)
    return downloads_path

def _download_file(download_path, filename):
    ssl._create_default_https_context = ssl._create_unverified_context
    urllib.request.urlretrieve(download_path, filename)
    ssl._create_default_https_context = ssl._create_default_https_context


def _load_data_file(filename, download_path, delimiter=",", last_column_are_labels=True):
    if not os.path.isfile(filename):
        _download_file(download_path, filename)
    datafile = np.genfromtxt(filename, delimiter=delimiter)
    if last_column_are_labels:
        data = datafile[:, :-1]
        labels = datafile[:, -1]
    else:
        data = datafile[:, 1:]
        labels = datafile[:, 0]
    return data, labels

def _load_torch_image_data(data_source, add_testdata):
    # Get data from source
    ssl._create_default_https_context = ssl._create_unverified_context
    dataset = data_source(root=_get_download_dir(), train=True, download=True)
    data = dataset.data
    labels = dataset.targets
    if add_testdata:
        testset = data_source(root=_get_download_dir(), train=False, download=True)
        data = torch.cat([data, testset.data], dim=0)
        labels = torch.cat([labels, testset.targets], dim=0)
    ssl._create_default_https_context = ssl._create_default_https_context
    # Flatten shape
    if data.dim() == 3:
        data = data.reshape(-1, data.shape[1] * data.shape[2])
    else:
        data = data.reshape(-1, data.shape[1] * data.shape[2] * data.shape[3])
    # Move data to CPU
    data_cpu = data.detach().cpu().numpy()
    labels_cpu = labels.detach().cpu().numpy()
    return data_cpu, labels_cpu

def _image_z_transformation(data):
    return (data - np.mean(data)) / np.std(data)


def load_usps(add_testdata=True, normalize=True):
    dataset = torchvision.datasets.USPS(root=_get_download_dir(), train=True, download=True)
    data = dataset.data
    labels = dataset.targets
    if add_testdata:
        test_dataset = torchvision.datasets.USPS(root=_get_download_dir(), train=False, download=True)
        data = np.r_[data, test_dataset.data]
        labels = np.r_[labels, test_dataset.targets]
    data = data.reshape(-1, 256)
    if normalize:
        data = _image_z_transformation(data)
    return data, labels

def load_optdigits(add_testdata=True, normalize=True):
    filename = _get_download_dir() + "/optdigits.tra"
    data, labels = _load_data_file(filename,
                                   "https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tra")
    if add_testdata:
        filename = _get_download_dir() + "/optdigits.tes"
        test_data, test_labels = _load_data_file(filename,
                                                 "https://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes")
        data = np.r_[data, test_data]
        labels = np.r_[labels, test_labels]
    if normalize:
        data = _image_z_transformation(data)
    return data, labels
"""
Start DipDECK
"""

if __name__ == "__main__":
    # === Choose data set ===
    # data, labels = load_usps()
    # data, labels = load_mnist()
    # data, labels = load_fmnist()
    # data, labels = load_kmnist()
    data, labels = load_optdigits()
    # data, labels = load_pendigits()
    # data, labels = load_letterrecognition()
    # GTSRB can be downloaded at https://benchmark.ini.rub.de/gtsrb_news.html

    # === Create DipDECK object ===
    dipdeck = DipDECK()
    dipdeck.fit(data)

    # === Print results ===
    print("K:", dipdeck.n_clusters_)
    print("NMI:", nmi(labels, dipdeck.labels_))

## **External Run of BERTweet -- NOT REQD TO RUN**

In [ ]:
from emoji import demojize

In [ ]:
# tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/TwiCSv2/data/tweets_3k_annotated.csv",sep =',',keep_default_na=False)
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('tweets_3k_annotated.csv')

# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('venezuela.csv')

# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('billdeblasio.csv')
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('pikapika.csv')
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('ripcity.csv')
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('billnye.csv')
# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('roevwade.csv')

# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('wnut17test.csv')

# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('broad_twitter_corpus.csv')


# testset, tokenizedtestset, tweet_to_sentences_w_annotation = preprocess('wnut17test_ner.csv')

In [ ]:
# tokenized_datasets = testset.map(tokenize_and_align_labels)
print(len(testset))
print(testset[:5])

1864
['& gt ; * The soldier was killed when another avalanche hit an army barracks in the northern area of Sonmarg , said a military spokesman .', '& gt ; * Police last week evacuated 80 villagers from Waltengoo Nar where dozens were killed after a series of avalanches hit the area in 2005 in the south of the territory .', '& gt ; * The army on Thursday recovered the bodies of ten of its men who were killed in an avalanche the previous day .', '& gt ; * The four civilians killed included two children of a family whose house was hit by a separate avalanche , also on Wednesday , a police spokesman said .', 'The bodies of the soldiers were recovered after the concerted efforts of the Avalanche Rescue Teams ( ART ) , which is equipped to work in inhospitable terrain and weather conditions .']


In [ ]:
# Set device
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

cuda


In [ ]:
def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):
    tokens = tweetTokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return normTweet

In [ ]:
tweetTokenizer = TweetTokenizer()
predictions=[]
tokenized_sentences=[]
count=0
with torch.no_grad():
    for test_record in testset:
        # print(test_record)
        # test_record=test_record.lower()
        # print(test_record)
        sentence = normalizeTweet(test_record)
        # tweetWordList=self.getWords(sentence)
        tweetWordList = sentence.split()

        tokenized_input=tokenizer(sentence)
        initial_input_ids = torch.tensor([tokenizer.encode(sentence)])
        initial_input_ids = initial_input_ids[:,:128]
        token_dict = {x : tokenizer.encode(x, add_special_tokens=False) for x in sentence.split()}
        input_ids = initial_input_ids.to(device)
        tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
        
        # output = model(input_ids)

        output = alt_model(input_ids)
        
        
        prediction = (torch.argmax(output.logits, axis=2))
        prediction = prediction.cpu().numpy().reshape(-1)
        if(count<=5):
            print(prediction)
        # prediction_labels=[label_list[l].split('-')[0] for l in prediction]
        prediction_labels=[label_list[l] for l in prediction]
        prediction_labels=collate_token_labels(tweetWordList, token_dict, prediction_labels[1:-1])
        predictions.append(prediction_labels)
        tokenized_sentences.append(tweetWordList)
        assert (len(prediction_labels)==len(tweetWordList))

        if(count<=5):
            print(len(output.hidden_states))
            print(token_dict.keys())
            print(initial_input_ids)
            print(input_ids)
            print(prediction_labels)
            print('===============')
        count+=1

print(len(predictions),len(tokenized_sentences))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 5 0 0 5 0 5 5 5 5 0 0 0 0 0 0 0]
13
dict_keys(['&', 'gt', ';', '*', 'The', 'soldier', 'was', 'killed', 'when', 'another', 'avalanche', 'hit', 'an', 'army', 'barracks', 'in', 'the', 'northern', 'area', 'of', 'Sonmarg', ',', 'said', 'a', 'military', 'spokesman', '.'])
tensor([[    0,    55,  8083,   208,   110,    47, 10998,    38,  1294,    64,
           347, 36791, 25313,   512,    74,  3466,  2149, 18549,    16,     6,
          7322,  1149,    15,  7725,  2350,   714,     7,   193,    11,  2492,
         34196,     4,     2]])
tensor([[    0,    55,  8083,   208,   110,    47, 10998,    38,  1294,    64,
           347, 36791, 25313,   512,    74,  3466,  2149, 18549,    16,     6,
          7322,  1149,    15,  7725,  2350,   714,     7,   193,    11,  2492,
         34196,     4,     2]], device='cuda:0')
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'B-LOC', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O

In [ ]:
print(tokenized_sentences[:5])
print(predictions[:5])

[['&', 'gt', ';', '*', 'The', 'soldier', 'was', 'killed', 'when', 'another', 'avalanche', 'hit', 'an', 'army', 'barracks', 'in', 'the', 'northern', 'area', 'of', 'Sonmarg', ',', 'said', 'a', 'military', 'spokesman', '.'], ['&', 'gt', ';', '*', 'Police', 'last', 'week', 'evacuated', '80', 'villagers', 'from', 'Waltengoo', 'Nar', 'where', 'dozens', 'were', 'killed', 'after', 'a', 'series', 'of', 'avalanches', 'hit', 'the', 'area', 'in', '2005', 'in', 'the', 'south', 'of', 'the', 'territory', '.'], ['&', 'gt', ';', '*', 'The', 'army', 'on', 'Thursday', 'recovered', 'the', 'bodies', 'of', 'ten', 'of', 'its', 'men', 'who', 'were', 'killed', 'in', 'an', 'avalanche', 'the', 'previous', 'day', '.'], ['&', 'gt', ';', '*', 'The', 'four', 'civilians', 'killed', 'included', 'two', 'children', 'of', 'a', 'family', 'whose', 'house', 'was', 'hit', 'by', 'a', 'separate', 'avalanche', ',', 'also', 'on', 'Wednesday', ',', 'a', 'police', 'spokesman', 'said', '.'], ['The', 'bodies', 'of', 'the', 'soldiers

In [ ]:
# calculate_f1(tweet_to_sentences_w_annotation, tokenized_sentences, predictions)
calculate_f1_ner(tweet_to_sentences_w_annotation, tokenized_sentences, predictions)